In [ ]:
import pandas as pd

# Ensure master_vtd_gdf and primary_votes_cleaned_df are available
if 'master_vtd_gdf' not in locals() or master_vtd_gdf is None or master_vtd_gdf.empty:
    print("❌ master_vtd_gdf GeoDataFrame is not available or empty. Cannot prepare primary vote data for popup.")
elif 'primary_votes_cleaned_df' not in locals() or primary_votes_cleaned_df is None or primary_votes_cleaned_df.empty:
    print("❌ primary_votes_cleaned_df DataFrame is not available or empty. Cannot prepare primary vote data for popup.")
else:
    print("✅ master_vtd_gdf and primary_votes_cleaned_df are available for primary vote popup data preparation.")

    # Define the relevant primary vote columns from primary_votes_cleaned_df
    # These are the cleaned and merged vote columns
    primary_vote_cols = [
        'BufordR_2022_Primary_Votes', 'EllzeyR_2022_Primary_Votes', 'PayneR_2022_Primary_Votes',
        'BufordR_2024_Primary_Votes', 'EllzeyR_2024_Primary_Votes', 'WileyR_2024_Primary_Votes' # Corrected typo WileyR_2024
    ]

    # Ensure these columns exist in master_vtd_gdf (they should after the merge in step 3)
    # and are numeric (should be after cleaning in step 2)
    for col in primary_vote_cols:
        if col not in master_vtd_gdf.columns:
            print(f"⚠️ Primary vote column '{col}' not found in master_vtd_gdf. Creating a placeholder.")
            master_vtd_gdf[col] = pd.NA # Use pandas NA for nullable integer

        # Ensure numeric type, coercing errors to NaN
        master_vtd_gdf[col] = pd.to_numeric(master_vtd_gdf[col], errors='coerce')


    # Define a helper function to format vote counts, handling NaNs
    def format_vote_count(count):
        if pd.notnull(count):
            try:
                return f"{int(count):,}"
            except:
                return str(count) # Return as string if conversion fails
        return 'N/A'


    # Create the Primary Votes HTML snippet for each VTD
    master_vtd_gdf['primary_votes_html'] = master_vtd_gdf.apply(lambda row: f"""
    <b>Primary Vote Data:</b><br>
    <table style="width:100%;">
      <tr>
        <th>Candidate</th>
        <th>2022 Primary</th>
        <th>2024 Primary</th>
      </tr>
      <tr>
        <td>Buford (R)</td>
        <td>{format_vote_count(row.get('BufordR_2022_Primary_Votes'))}</td>
        <td>{format_vote_count(row.get('BufordR_2024_Primary_Votes'))}</td>
      </tr>
      <tr>
        <td>Ellzey (R)</td>
        <td>{format_vote_count(row.get('EllzeyR_2022_Primary_Votes'))}</td>
        <td>{format_vote_count(row.get('EllzeyR_2024_Primary_Votes'))}</td>
      </tr>
      <tr>
        <td>Payne (R)</td>
        <td>{format_vote_count(row.get('PayneR_2022_Primary_Votes'))}</td>
        <td>N/A</td> <!-- Payne was not in the 2024 primary -->
      </tr>
      <tr>
        <td>Wiley (R)</td>
        <td>N/A</td> <!-- Wiley was not in the 2022 primary -->
        <td>{format_vote_count(row.get('WileyR_2024_Primary_Votes'))}</td> # Corrected typo WileyR_2024
      </tr>
    </table>
    """, axis=1)

    print("✅ Created 'primary_votes_html' column for popup.")

    # Print head and columns to verify
    print("\n📋 Head of master_vtd_gdf with 'primary_votes_html':")
    display(master_vtd_gdf[['CNTYVTD', 'primary_votes_html']].head())
    print("\nColumns of master_vtd_gdf after primary votes popup preparation:")
    print(master_vtd_gdf.columns.tolist())

# Task
Create an interactive map of CD6 using "vtds_cd6.geojson" for VTD boundaries and "texas_county_shapefiles/County_Boundaries.shp" for county boundaries. The map should display County Lines and CNTYVTDs. When hovering over a VTD, display the County, Precinct, and Cities from "cleaned_torv_data.csv" or "DemographicsFinal.csv". When clicking on a VTD, provide a dropdown menu to view: 1) demographic data over time from "DemographicsFinal.csv", 2) Primary Votes for Ellzey over time from "final2022-2024PrimaryVotes.csv", and 3) Voter Registration and Turnout over time (excluding Spanish Surname data) from "cleaned_torv_data.csv".

## Load geographic data

### Subtask:
Load the base CD6 VTD geometry from `vtds_cd6.geojson` and reproject it to a suitable projected CRS (EPSG:3083).


**Reasoning**:
Load the VTD GeoJSON file, define the target CRS, check for file existence, read the file into a GeoDataFrame, check and reproject the CRS if necessary, and print confirmation or error messages.



In [ ]:
import geopandas as gpd
import os

# 1. Define the path to the vtds_cd6.geojson file.
geojson_path = 'vtds_cd6.geojson'

# 2. Define the target projected CRS (EPSG:3083).
target_crs = "EPSG:3083"

# 3. Check if the GeoJSON file exists using os.path.exists().
if os.path.exists(geojson_path):
    try:
        # 4. If the file exists, read it into a GeoDataFrame using gpd.read_file().
        vtds_cd6_gdf_proj = gpd.read_file(geojson_path)
        print(f"✅ Loaded base VTDs from {geojson_path}.")

        # 5. Check the current CRS of the loaded GeoDataFrame. If it's None or different from the target CRS, reproject it to the target CRS (EPSG:3083) using .to_crs().
        if vtds_cd6_gdf_proj.crs is None:
            print(f"⚠️ {geojson_path} CRS missing, assuming EPSG:3857 and setting to {target_crs}")
            vtds_cd6_gdf_proj.set_crs("EPSG:3857", inplace=True) # Assume a common web CRS if missing

        if vtds_cd6_gdf_proj.crs != target_crs:
             vtds_cd6_gdf_proj = vtds_cd6_gdf_proj.to_crs(target_crs)
             print(f"✅ Reprojected vtds_cd6_gdf to {target_crs}.")
        else:
             print(f"✅ vtds_cd6_gdf is already in {target_crs}.")

        # 6. Print a confirmation message indicating successful loading and reprojection (or if reprojection was skipped).
        print(f"📋 Head of reprojected vtds_cd6_gdf:")
        display(vtds_cd6_gdf_proj.head())
        print(f"CRS: {vtds_cd6_gdf_proj.crs}")


    except Exception as e:
        # Handle potential errors during loading or reprojection
        print(f"❌ Error loading or processing {geojson_path}: {e}")
        vtds_cd6_gdf_proj = None # Ensure the variable is set to None if loading fails
else:
    # 7. If the file does not exist, print an error message.
    print(f"❌ {geojson_path} not found. Cannot proceed.")
    vtds_cd6_gdf_proj = None # Ensure the variable is set to None if file not found

**Reasoning**:
The previous step successfully loaded and reprojected the base VTD geometry. The next step is to load the county boundaries shapefile, define the target projected CRS, check for file existence, read the file into a GeoDataFrame, check and reproject the CRS if necessary, and print confirmation or error messages.



In [ ]:
# 1. Define the path to the TexasCounties.zip (Counties shapefile).
counties_shapefile_path = "texas_county_shapefiles/County_Boundaries.shp"

# 2. Define the target projected CRS (EPSG:3083). This was already defined in the previous step, but is repeated here for clarity within this block.
target_crs = "EPSG:3083"

# 3. Check if the GeoJSON file exists using os.path.exists().
if os.path.exists(counties_shapefile_path):
    try:
        # 4. If the file exists, read it into a GeoDataFrame using gpd.read_file().
        counties_gdf_proj = gpd.read_file(counties_shapefile_path)
        print(f"✅ Loaded county boundaries from {counties_shapefile_path}.")

        # 5. Check the current CRS of the loaded GeoDataFrame. If it's None or different from the target CRS, reproject it to the target CRS (EPSG:3083) using .to_crs().
        if counties_gdf_proj.crs is None:
            print(f"⚠️ {counties_shapefile_path} CRS missing, assuming EPSG:3857 and setting to {target_crs}")
            counties_gdf_proj.set_crs("EPSG:3857", inplace=True) # Assume a common web CRS if missing

        if counties_gdf_proj.crs != target_crs:
             counties_gdf_proj = counties_gdf_proj.to_crs(target_crs)
             print(f"✅ Reprojected counties_gdf to {target_crs}.")
        else:
             print(f"✅ counties_gdf is already in {target_crs}.")

        # 6. Print a confirmation message indicating successful loading and reprojection (or if reprojection was skipped).
        print(f"📋 Head of reprojected counties_gdf:")
        display(counties_gdf_proj.head())
        print(f"CRS: {counties_gdf_proj.crs}")


    except Exception as e:
        # Handle potential errors during loading or reprojection
        print(f"❌ Error loading or processing {counties_shapefile_path}: {e}")
        counties_gdf_proj = None # Ensure the variable is set to None if loading fails
else:
    # 7. If the file does not exist, print an error message.
    print(f"❌ {counties_shapefile_path} not found. Cannot proceed.")
    counties_gdf_proj = None # Ensure the variable is set to None if file not found

**Reasoning**:
The previous steps have successfully loaded and reprojected the base VTD geometry and the county boundaries. The next step is to load the Precincts shapefile, define the target projected CRS, check for file existence, read the file into a GeoDataFrame, check and reproject the CRS if necessary, and print confirmation or error messages. This is step 4 from the original task plan, but adapted for the current subtask.



In [ ]:
# Define the path to the Precincts shapefile.
precincts_shapefile_path = "precincts_shapefiles/Precincts24G.shp"

# The target projected CRS (EPSG:3083) is already defined.

# Check if the shapefile exists.
if os.path.exists(precincts_shapefile_path):
    try:
        # Read the file into a GeoDataFrame.
        precincts_gdf_proj = gpd.read_file(precincts_shapefile_path)
        print(f"✅ Loaded precincts from {precincts_shapefile_path}.")

        # Check the current CRS. If it's None or different from the target CRS, reproject it.
        if precincts_gdf_proj.crs is None:
            print(f"⚠️ {precincts_shapefile_path} CRS missing, assuming EPSG:3857 and setting to {target_crs}")
            precincts_gdf_proj.set_crs("EPSG:3857", inplace=True) # Assume a common web CRS if missing

        if precincts_gdf_proj.crs != target_crs:
             precincts_gdf_proj = precincts_gdf_proj.to_crs(target_crs)
             print(f"✅ Reprojected precincts_gdf to {target_crs}.")
        else:
             print(f"✅ precincts_gdf is already in {target_crs}.")

        # Print confirmation and head.
        print(f"📋 Head of reprojected precincts_gdf:")
        display(precincts_gdf_proj.head())
        print(f"CRS: {precincts_gdf_proj.crs}")


    except Exception as e:
        # Handle potential errors during loading or reprojection
        print(f"❌ Error loading or processing {precincts_shapefile_path}: {e}")
        precincts_gdf_proj = None # Ensure the variable is set to None if loading fails
else:
    # If the file does not exist, print an error message.
    print(f"❌ {precincts_shapefile_path} not found. Cannot proceed.")
    precincts_gdf_proj = None # Ensure the variable is set to None if file not found

**Reasoning**:
The previous steps have successfully loaded and reprojected the base VTD geometry, county boundaries, and precincts. The next step is to load the Cities shapefile, define the target projected CRS, check for file existence, read the file into a GeoDataFrame, check and reproject the CRS if necessary, and print confirmation or error messages. This is another necessary geographic layer for the final map.



In [ ]:
# Define the path to the Cities shapefile.
cities_shapefile_path = "texas_city_shapefiles/City.shp"

# The target projected CRS (EPSG:3083) is already defined.

# Check if the shapefile exists.
if os.path.exists(cities_shapefile_path):
    try:
        # Read the file into a GeoDataFrame.
        cities_gdf_proj = gpd.read_file(cities_shapefile_path)
        print(f"✅ Loaded cities from {cities_shapefile_path}.")

        # Check the current CRS. If it's None or different from the target CRS, reproject it.
        if cities_gdf_proj.crs is None:
            print(f"⚠️ {cities_shapefile_path} CRS missing, assuming EPSG:3857 and setting to {target_crs}")
            cities_gdf_proj.set_crs("EPSG:3857", inplace=True) # Assume a common web CRS if missing

        if cities_gdf_proj.crs != target_crs:
             cities_gdf_proj = cities_gdf_proj.to_crs(target_crs)
             print(f"✅ Reprojected cities_gdf to {target_crs}.")
        else:
             print(f"✅ cities_gdf is already in {target_crs}.")

        # Print confirmation and head.
        print(f"📋 Head of reprojected cities_gdf:")
        display(cities_gdf_proj.head())
        print(f"CRS: {cities_gdf_proj.crs}")


    except Exception as e:
        # Handle potential errors during loading or reprojection
        print(f"❌ Error loading or processing {cities_shapefile_path}: {e}")
        cities_gdf_proj = None # Ensure the variable is set to None if loading fails
else:
    # If the file does not exist, print an error message.
    print(f"❌ {cities_shapefile_path} not found. Cannot proceed.")
    cities_gdf_proj = None # Ensure the variable is set to None if file not found

## Load and prepare attribute data

### Subtask:
Load `cleaned_torv_data.csv`, `DemographicsFinal.csv`, and `final2022-2024PrimaryVotes.csv`. Clean and standardize the 'CNTYVTD' column in each DataFrame to ensure consistency for merging. Select the necessary columns from each DataFrame for the tooltip and popup.


**Reasoning**:
The subtask requires loading and cleaning three dataframes and selecting specific columns from them. I will write a single code block to perform all these steps.



In [ ]:
# 1. Load cleaned_torv_data.csv
cleaned_torv_file_path = 'cleaned_torv_data.csv'
cleaned_torv_df = None
if os.path.exists(cleaned_torv_file_path):
    try:
        cleaned_torv_df = pd.read_csv(cleaned_torv_file_path)
        print(f"✅ Loaded cleaned_torv_data.csv from {cleaned_torv_file_path}")
    except Exception as e:
        print(f"❌ Error loading {cleaned_torv_file_path}: {e}")
else:
    print(f"❌ {cleaned_torv_file_path} not found.")

# 2. Clean and standardize 'CNTYVTD' in cleaned_torv_df
if cleaned_torv_df is not None and 'CNTYVTD' in cleaned_torv_df.columns:
    cleaned_torv_df['CNTYVTD'] = cleaned_torv_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
    print("✅ Cleaned 'CNTYVTD' in cleaned_torv_df.")
else:
    print("⚠️ 'CNTYVTD' column not found or cleaned_torv_df not loaded. Skipping CNTYVTD cleaning for cleaned_torv_df.")


# 3. Load DemographicsFinal.csv
demographics_file_path = "DemographicsFinal.csv"
demographics_df = None
if os.path.exists(demographics_file_path):
    try:
        demographics_df = pd.read_csv(demographics_file_path)
        print(f"\n✅ Loaded DemographicsFinal.csv from {demographics_file_path}")
    except Exception as e:
        print(f"❌ Error loading {demographics_file_path}: {e}")
else:
    print(f"\n❌ {demographics_file_path} not found.")

# 4. Clean 'VTD Identifier', rename to 'CNTYVTD', and select columns for demographics_geo_df
demographics_geo_df = None
if demographics_df is not None and 'VTD Identifier' in demographics_df.columns:
    try:
        demographics_geo_df = demographics_df[['VTD Identifier', 'Best Matching Precinct Name (Max Overlap)', 'County Name (from TORV)', 'Cities (from TORV)']].copy()
        demographics_geo_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
        demographics_geo_df['CNTYVTD'] = demographics_geo_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')

        # Handle potential duplicates in demographics_geo_df before merging
        if not demographics_geo_df['CNTYVTD'].is_unique:
             print("⚠️ Duplicate CNTYVTDs found in demographics geo info. Aggregating (taking first).")
             agg_funcs_geo = {col: 'first' for col in demographics_geo_df.columns if col != 'CNTYVTD'}
             demographics_geo_df = demographics_geo_df.groupby('CNTYVTD', as_index=False).agg(agg_funcs_geo)


        print("✅ Cleaned 'VTD Identifier', renamed to 'CNTYVTD', and selected columns for demographics_geo_df.")
    except KeyError as e:
        print(f"❌ Missing expected column in DemographicsFinal.csv for selection: {e}")
        demographics_geo_df = None
    except Exception as e:
        print(f"❌ Error processing DemographicsFinal.csv for demographics_geo_df: {e}")
        demographics_geo_df = None
else:
    print("⚠️ 'VTD Identifier' column not found or DemographicsFinal.csv not loaded. Cannot create demographics_geo_df.")


# 5. Load final2022-2024PrimaryVotes.csv
primary_votes_file_path = "final2022-2024PrimaryVotes.csv"
primary_votes_df = None
if os.path.exists(primary_votes_file_path):
    try:
        primary_votes_df = pd.read_csv(primary_votes_file_path)
        print(f"\n✅ Loaded final2022-2024PrimaryVotes.csv from {primary_votes_file_path}")
    except Exception as e:
        print(f"❌ Error loading {primary_votes_file_path}: {e}")
else:
    print(f"\n❌ {primary_votes_file_path} not found.")

# 6 & 7. Identify relevant vote columns, clean keys, consolidate, and ensure numeric for primary_votes_cleaned_df
primary_votes_cleaned_df = None
if primary_votes_df is not None:
    vote_cols_2022_raw = ['CNTYVTD', 'BufordR_22P_U.S. Rep 6', 'EllzeyR_22P_U.S. Rep 6', 'PayneR_22P_U.S. Rep 6']
    vote_cols_2024_raw = ['CNTYVTD.1', 'BufordR_24P_U.S. Rep 6', 'EllzeyR_24P_U.S. Rep 6', 'WileyR_24P_U.S. Rep 6']

    votes_2022_df = primary_votes_df.dropna(subset=[col for col in vote_cols_2022_raw if col in primary_votes_df.columns]).copy()
    votes_2024_df = primary_votes_df.dropna(subset=[col for col in vote_cols_2024_raw if col in primary_votes_df.columns]).copy()

    # Select only relevant columns and clean keys
    votes_2022_df = votes_2022_df[[col for col in vote_cols_2022_raw if col in votes_2022_df.columns]].copy()
    votes_2024_df = votes_2024_df[[col for col in vote_cols_2024_raw if col in votes_2024_df.columns]].copy()

    if 'CNTYVTD' in votes_2022_df.columns: votes_2022_df['CNTYVTD'] = votes_2022_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
    if 'CNTYVTD.1' in votes_2024_df.columns:
        votes_2024_df.rename(columns={'CNTYVTD.1': 'CNTYVTD'}, inplace=True)
    if 'CNTYVTD' in votes_2024_df.columns: votes_2024_df['CNTYVTD'] = votes_2024_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')


    # Merge 2022 and 2024 data
    primary_votes_merged = pd.merge(
        votes_2022_df,
        votes_2024_df,
        on='CNTYVTD',
        how='outer'
    )

    # Aggregate potential duplicates
    if not primary_votes_merged.empty and 'CNTYVTD' in primary_votes_merged.columns and primary_votes_merged['CNTYVTD'].duplicated().any():
         print("⚠️ Duplicates found on CNTYVTD after initial primary vote merge. Aggregating (taking first).")
         agg_funcs_votes = {col: 'first' for col in primary_votes_merged.columns if col != 'CNTYVTD'}
         primary_votes_cleaned_df = primary_votes_merged.groupby('CNTYVTD', as_index=False).agg(agg_funcs_votes)
    else:
         primary_votes_cleaned_df = primary_votes_merged.copy()


    # Ensure all vote columns are numeric
    if primary_votes_cleaned_df is not None:
        vote_cols_final = [col for col in primary_votes_cleaned_df.columns if col != 'CNTYVTD']
        for col in vote_cols_final:
            primary_votes_cleaned_df[col] = pd.to_numeric(primary_votes_cleaned_df[col], errors='coerce')
        print("✅ Processed and cleaned primary votes data.")

else:
    print("⚠️ primary_votes_df not loaded. Cannot create primary_votes_cleaned_df.")


# 8. Print heads and column names
print("\n--- Verification of Cleaned DataFrames ---")

print("\n📋 Head of cleaned_torv_df:")
if cleaned_torv_df is not None:
    display(cleaned_torv_df.head())
    print("\nColumns of cleaned_torv_df:")
    print(cleaned_torv_df.columns.tolist())
else:
    print("cleaned_torv_df is not available.")


print("\n📋 Head of demographics_geo_df:")
if demographics_geo_df is not None:
    display(demographics_geo_df.head())
    print("\nColumns of demographics_geo_df:")
    print(demographics_geo_df.columns.tolist())
else:
    print("demographics_geo_df is not available.")


print("\n📋 Head of primary_votes_cleaned_df:")
if primary_votes_cleaned_df is not None:
    display(primary_votes_cleaned_df.head())
    print("\nColumns of primary_votes_cleaned_df:")
    print(primary_votes_cleaned_df.columns.tolist())
else:
    print("primary_votes_cleaned_df is not available.")


## Merge attribute data with geometry

### Subtask:
Merge the prepared attribute dataframes from step 2 (`cleaned_torv_df`, `demographics_geo_df`, and `primary_votes_cleaned_df`) onto the VTD GeoDataFrame from step 1 (`vtds_cd6_gdf_proj`) using the cleaned 'CNTYVTD' identifier. Perform left merges to keep all VTDs from the geometry file, adding attribute data where available.


**Reasoning**:
Merge the attribute dataframes onto the VTD GeoDataFrame using the cleaned 'CNTYVTD' key.



In [ ]:
import pandas as pd

# Ensure vtds_cd6_gdf_proj is available and has the cleaned 'CNTYVTD'
if 'vtds_cd6_gdf_proj' in locals() and vtds_cd6_gdf_proj is not None and not vtds_cd6_gdf_proj.empty:
    if 'CNTYVTD' not in vtds_cd6_gdf_proj.columns:
        # Re-create CNTYVTD in vtds_cd6_gdf_proj if it's missing, based on previous logic
        if 'CNTY_x' in vtds_cd6_gdf_proj.columns and 'VTD_x' in vtds_cd6_gdf_proj.columns:
            vtds_cd6_gdf_proj['CNTYVTD'] = vtds_cd6_gdf_proj['CNTY_x'].astype(str).str.strip() + vtds_cd6_gdf_proj['VTD_x'].astype(str).str.strip()
            print("✅ Created 'CNTYVTD' in vtds_cd6_gdf_proj from CNTY_x and VTD_x.")
        elif 'CNTYVTD_x' in vtds_cd6_gdf_proj.columns:
             vtds_cd6_gdf_proj.rename(columns={'CNTYVTD_x': 'CNTYVTD'}, inplace=True)
             print("✅ Using existing 'CNTYVTD_x' as 'CNTYVTD' in vtds_cd6_gdf_proj.")
        elif 'CNTYVTD_y' in vtds_cd6_gdf_proj.columns: # Check for _y if it was merged from pop data
             vtds_cd6_gdf_proj.rename(columns={'CNTYVTD_y': 'CNTYVTD'}, inplace=True)
             print("✅ Using existing 'CNTYVTD_y' as 'CNTYVTD' in vtds_cd6_gdf_proj.")
        else:
             print("❌ Could not find a suitable VTD identifier column in vtds_cd6_gdf_proj to create CNTYVTD. Cannot merge attributes.")
             master_vtd_gdf = vtds_cd6_gdf_proj.copy() # Proceed with geometry only
             merge_failed = True # Flag merge failure

    if 'CNTYVTD' in vtds_cd6_gdf_proj.columns:
        vtds_cd6_gdf_proj['CNTYVTD'] = vtds_cd6_gdf_proj['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
        print("✅ Cleaned 'CNTYVTD' in vtds_cd6_gdf_proj.")
        master_vtd_gdf = vtds_cd6_gdf_proj.copy() # Start with the cleaned GeoDataFrame
        merge_failed = False
    else:
         master_vtd_gdf = vtds_cd6_gdf_proj.copy() # Proceed with geometry only
         merge_failed = True # Flag merge failure


else:
    print("❌ vtds_cd6_gdf_proj is not available or empty. Cannot merge attributes.")
    # Create an empty GeoDataFrame with the expected columns to avoid errors in subsequent steps
    master_vtd_gdf = gpd.GeoDataFrame({'CNTYVTD': [], 'geometry': []}, crs=target_crs)
    merge_failed = True # Flag merge failure


if not merge_failed:
    # --- Merge Demographics Geo Info ---
    if 'demographics_geo_df' in locals() and demographics_geo_df is not None and not demographics_geo_df.empty:
        print("\n--- Merging Demographics Geo Info ---")
        # Ensure key is clean in demographics_geo_df (already done in previous step, but double-check)
        if 'CNTYVTD' in demographics_geo_df.columns:
             demographics_geo_df['CNTYVTD'] = demographics_geo_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')

             # Drop potential duplicate columns in master_vtd_gdf before merging
             geo_cols_to_drop_before_demo = ['Best Matching Precinct Name (Max Overlap)', 'County Name (from TORV)', 'Cities (from TORV)', 'Precinct', 'County', 'Intersecting Cities'] # Include target names from previous TORV merge
             master_vtd_gdf.drop(columns=[col for col in geo_cols_to_drop_before_demo if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
             print(f"Dropped existing geo columns from master_vtd_gdf before demographics merge.")


             master_vtd_gdf = master_vtd_gdf.merge(
                 demographics_geo_df,
                 on='CNTYVTD',
                 how='left'
             )
             print(f"✅ Merged demographics geo info. Rows after merge: {len(master_vtd_gdf)}")
        else:
             print("❌ 'CNTYVTD' not found in demographics_geo_df. Skipping merge.")
    else:
        print("❌ demographics_geo_df not available or empty. Skipping merge.")


    # --- Merge Cleaned TORV Data ---
    if 'cleaned_torv_df' in locals() and cleaned_torv_df is not None and not cleaned_torv_df.empty:
        print("\n--- Merging Cleaned TORV Data ---")
        # Ensure key is clean in cleaned_torv_df (already done in previous step, but double-check)
        if 'CNTYVTD' in cleaned_torv_df.columns:
             cleaned_torv_df['CNTYVTD'] = cleaned_torv_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')

             # Identify columns to merge from cleaned_torv_df, excluding keys and geo columns already merged from demographics
             torv_cols_to_merge = [col for col in cleaned_torv_df.columns if col not in ['CNTYVTD', 'Precinct', 'County', 'Intersecting Cities']]

             # Drop potential duplicate columns in master_vtd_gdf before merging TORV data
             # Use the column names from the cleaned_torv_df that are being merged
             master_vtd_gdf.drop(columns=[col for col in torv_cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
             print(f"Dropped potential duplicate TORV columns from master_vtd_gdf before TORV merge.")


             master_vtd_gdf = master_vtd_gdf.merge(
                 cleaned_torv_df[['CNTYVTD'] + torv_cols_to_merge],
                 on='CNTYVTD',
                 how='left'
             )
             print(f"✅ Merged cleaned TORV data. Rows after merge: {len(master_vtd_gdf)}")
        else:
             print("❌ 'CNTYVTD' not found in cleaned_torv_df. Skipping merge.")
    else:
        print("❌ cleaned_torv_df not available or empty. Skipping merge.")


    # --- Merge Primary Votes Data ---
    if 'primary_votes_cleaned_df' in locals() and primary_votes_cleaned_df is not None and not primary_votes_cleaned_df.empty:
        print("\n--- Merging Primary Votes Data ---")
        # Ensure key is clean in primary_votes_cleaned_df (already done in previous step, but double-check)
        if 'CNTYVTD' in primary_votes_cleaned_df.columns:
             primary_votes_cleaned_df['CNTYVTD'] = primary_votes_cleaned_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')

             # Drop potential duplicate columns in master_vtd_gdf before merging primary votes
             vote_cols_to_merge = [col for col in primary_votes_cleaned_df.columns if col != 'CNTYVTD']
             master_vtd_gdf.drop(columns=[col for col in vote_cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
             print(f"Dropped potential duplicate vote columns from master_vtd_gdf before vote merge.")


             master_vtd_gdf = master_vtd_gdf.merge(
                 primary_votes_cleaned_df,
                 on='CNTYVTD',
                 how='left'
             )
             print(f"✅ Merged primary votes data. Rows after merge: {len(master_vtd_gdf)}")
        else:
             print("❌ 'CNTYVTD' not found in primary_votes_cleaned_df. Skipping merge.")
    else:
        print("❌ primary_votes_cleaned_df not available or empty. Skipping merge.")


    # --- Final Inspection ---
    print("\n--- Final Merged GeoDataFrame Inspection ---")
    print(f"Merged GeoDataFrame shape: {master_vtd_gdf.shape}")
    print("\nHead of merged master_vtd_gdf:")
    display(master_vtd_gdf.head())
    print("\nColumns of merged master_vtd_gdf:")
    print(master_vtd_gdf.columns.tolist())
else:
    print("\n❌ Merge process skipped due to missing or invalid base GeoDataFrame.")

## Prepare data for tooltip

### Subtask:
Select and format the County, Precinct, and Cities columns for the hover tooltip. Ensure these columns exist and are handled gracefully if missing in some VTDs.


**Reasoning**:
Check if the master_vtd_gdf GeoDataFrame exists and is not empty. If it is not available or empty, print an error message and finish the task. Otherwise, prepare the County, Precinct, and Cities columns for the tooltip as requested in the instructions.



In [ ]:
import pandas as pd

# 1. Check if the master_vtd_gdf GeoDataFrame exists and is not empty.
if 'master_vtd_gdf' not in locals() or master_vtd_gdf is None or master_vtd_gdf.empty:
    print("❌ master_vtd_gdf GeoDataFrame is not available or empty. Cannot prepare tooltip columns.")
    # No further steps can be completed without the master GeoDataFrame.
else:
    print("✅ master_vtd_gdf GeoDataFrame is available for tooltip preparation.")

    # 2. Define a list of column names expected for the tooltip.
    # Use the exact column names resulting from the previous merge steps.
    tooltip_cols_raw = ['County Name (from TORV)', 'Best Matching Precinct Name (Max Overlap)', 'Cities (from TORV)']

    # Map the raw column names to the desired display names for the tooltip
    tooltip_col_map = {
        'County Name (from TORV)': 'County',
        'Best Matching Precinct Name (Max Overlap)': 'Precinct',
        'Cities (from TORV)': 'Cities'
    }

    # Ensure the target columns for the tooltip map exist in the master_vtd_gdf
    # Create placeholder columns if they don't exist, then apply the rename
    for raw_col, display_col in tooltip_col_map.items():
        if raw_col not in master_vtd_gdf.columns:
            print(f"⚠️ Tooltip column '{raw_col}' not found. Creating a placeholder column '{display_col}'.")
            master_vtd_gdf[display_col] = 'N/A' # Create with target display name directly
        else:
            # If the column exists, prepare it (convert to string and fill NaNs)
            master_vtd_gdf[display_col] = master_vtd_gdf[raw_col].astype(str).fillna('N/A')
            # If the display column name is different from the raw, we rename later or just use the display name as the final column name


    # Select the final columns to be used in the tooltip (using display names)
    final_tooltip_cols = list(tooltip_col_map.values())
    # Ensure the final columns are in the GeoDataFrame
    for col in final_tooltip_cols:
        if col not in master_vtd_gdf.columns:
             # This shouldn't happen if the logic above worked, but as a safeguard
             master_vtd_gdf[col] = 'N/A' # Create if somehow missed


    # 6. Create a new column in master_vtd_gdf named 'tooltip_text'.
    master_vtd_gdf['tooltip_text'] = master_vtd_gdf.apply(
        lambda row: f"""
<b>County:</b> {row.get('County', 'N/A')}<br>
<b>Precinct:</b> {row.get('Precinct', 'N/A')}<br>
<b>Cities:</b> {row.get('Cities', 'N/A')}
        """,
        axis=1
    )
    print("✅ Created 'tooltip_text' column for VTD hover tooltips.")

    # 7. Print the head of master_vtd_gdf showing the original and newly created tooltip columns.
    print("\n📋 Head of master_vtd_gdf with tooltip columns:")
    # Display the original raw columns if they exist, and the final display columns and tooltip_text
    display_cols = [col for col in tooltip_cols_raw + final_tooltip_cols + ['tooltip_text'] if col in master_vtd_gdf.columns]
    display(master_vtd_gdf[display_cols].head())

    print("\nColumns of master_vtd_gdf after tooltip preparation:")
    print(master_vtd_gdf.columns.tolist())


**Reasoning**:
The County, Precinct, and Cities columns have been selected and formatted for the tooltip text. The 'tooltip\_text' column has been created. The next step is to prepare the demographic data for the popup dropdown.



In [ ]:
import pandas as pd

# Ensure master_vtd_gdf and demographics_df are available
if 'master_vtd_gdf' not in locals() or master_vtd_gdf is None or master_vtd_gdf.empty:
    print("❌ master_vtd_gdf GeoDataFrame is not available or empty. Cannot prepare demographic data for popup.")
elif 'demographics_df' not in locals() or demographics_df is None or demographics_df.empty:
    print("❌ demographics_df DataFrame is not available or empty. Cannot prepare demographic data for popup.")
else:
    print("✅ master_vtd_gdf and demographics_df are available for demographic popup data preparation.")

    # Define demographic columns from DemographicsFinal.csv for the popup
    demographic_cols_raw = [
        'VTD Identifier',
        'Total Population (2020)', 'Total Population (2022)', 'Total Population (2024)',
        'Anglo Population (2020)', 'Anglo Population (2022)', 'Anglo Population (2024)',
        'Non-Anglo Population (2020)', 'Non-Anglo Population (2022)', 'Non-Anglo Population (2024)',
        'Asian Population (2020)', 'Asian Population (2022)', 'Asian Population (2024)',
        'Black Population (2020)', 'Black Population (2022)', 'Black Population (2024)',
        'Hispanic Population (2020)', 'Hispanic Population (2022)', 'Hispanic Population (2024)',
        'Black + Hispanic Population (2020)', 'Black + Hispanic Population (2022)', 'Black + Hispanic Population (2024)',
        'Voting Age Population (2020)', 'Voting Age Population (2022)', 'Voting Age Population (2024)',
        'Anglo VAP (2020)', 'Anglo VAP (2022)', 'Anglo VAP (2024)',
        'Non-Anglo VAP (2020)', 'Non-Anglo VAP (2022)', 'Non-Anglo VAP (2024)',
        'Asian VAP (2020)', 'Asian VAP (2022)', 'Asian VAP (2024)',
        'Black VAP (2020)', 'Black VAP (2022)', 'Black VAP (2024)',
        'Hispanic VAP (2020)', 'Hispanic VAP (2022)', 'Hispanic VAP (2024)',
        'Black + Hispanic VAP (2020)', 'Black + Hispanic VAP (2022)', 'Black + Hispanic VAP (2024)',
        'Total Pop % Change (2020-2022)', 'Total Pop % Change (2022-2024)',
        'Anglo Pop % Change (2020-2022)', 'Anglo Pop % Change (2022-2024)',
        'Non-Anglo Pop % Change (2020-2022)', 'Non-Anglo Pop % Change (2022-2024)',
        'Asian Pop % Change (2020-2022)', 'Asian Pop % Change (2022-2024)',
        'Black Pop % Change (2020-2022)', 'Black Pop % Change (2022-2024)',
        'Hispanic Pop % Change (2020-2022)', 'Hispanic Pop % Change (2022-2024)',
        'Black + Hispanic Pop % Change (2020-2022)', 'Black + Hispanic Pop % Change (2022-2024)',
        'VAP % Change (2020-2022)', 'VAP % Change (2022-2024)',
        'Anglo VAP % Change (2020-2022)', 'Anglo VAP % Change (2022-2024)',
        'Non-Anglo VAP % Change (2020-2022)', 'Non-Anglo VAP % Change (2022-2024)',
        'Asian VAP % Change (2020-2022)', 'Asian VAP % Change (2022-2024)',
        'Black VAP % Change (2020-2022)', 'Black VAP % Change (2022-2024)',
        'Hispanic VAP % Change (2020-2022)', 'Hispanic VAP % Change (2022-2024)',
        'Black + Hispanic VAP % Change (2020-2022)', 'Black + Hispanic VAP % Change (2022-2024)'
    ]

    # Select the columns that are present in demographics_df
    demographic_cols_present = [col for col in demographic_cols_raw if col in demographics_df.columns]

    if 'VTD Identifier' in demographics_df.columns:
        # Prepare demographics data for merging, cleaning the key
        demographics_popup_df = demographics_df[demographic_cols_present].copy()
        demographics_popup_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
        demographics_popup_df['CNTYVTD'] = demographics_popup_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')

        # Handle potential duplicates
        if not demographics_popup_df['CNTYVTD'].is_unique:
            print("⚠️ Duplicate CNTYVTDs found in demographics data for popup. Aggregating (taking first).")
            agg_funcs_demo_popup = {col: 'first' for col in demographics_popup_df.columns if col != 'CNTYVTD'}
            demographics_popup_df = demographics_popup_df.groupby('CNTYVTD', as_index=False).agg(agg_funcs_demo_popup)

        print(f"✅ Prepared demographic data for popup for {len(demographics_popup_df)} VTDs.")

        # Merge the demographic data into the master GeoDataFrame
        # Identify demographic columns to merge, excluding the key
        demo_cols_to_merge = [col for col in demographics_popup_df.columns if col != 'CNTYVTD']

        # Drop potential existing demographic columns in master_vtd_gdf before merging
        master_vtd_gdf.drop(columns=[col for col in demo_cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
        print("Dropped potential existing demographic columns from master_vtd_gdf.")

        master_vtd_gdf = master_vtd_gdf.merge(
            demographics_popup_df,
            on='CNTYVTD',
            how='left'
        )
        print(f"✅ Merged demographic data for popup. Rows after merge: {len(master_vtd_gdf)}")

        # --- Format Demographic Data for Popup HTML ---
        print("\n--- Formatting Demographic Data for Popup HTML ---")

        # Define a helper function to format numeric values and percentages, handling NaNs
        def format_value(value, is_percentage=False):
            if pd.notnull(value):
                if is_percentage:
                    # Format percentages, handling potential non-numeric conversion issues earlier
                    try:
                         return f"{float(value):.1f}%"
                    except:
                         return str(value) # Return as string if conversion fails
                else:
                    # Format numeric values with commas, handling potential non-numeric conversion earlier
                    try:
                         return f"{int(float(value)):,}"
                    except:
                         return str(value) # Return as string if conversion fails
            return 'N/A'

        # Create the demographic HTML snippet for each VTD
        master_vtd_gdf['demographics_html'] = master_vtd_gdf.apply(lambda row: f"""
        <b>Demographic Data:</b><br>
        <table style="width:100%;">
          <tr>
            <th>Category</th>
            <th>2020</th>
            <th>2022</th>
            <th>2024</th>
            <th>Change (20-22)</th>
            <th>% Change (20-22)</th>
            <th>Change (22-24)</th>
            <th>% Change (22-24)</th>
          </tr>
          <tr>
            <td>Total Pop</td>
            <td>{format_value(row.get('Total Population (2020)'))}</td>
            <td>{format_value(row.get('Total Population (2022)'))}</td>
            <td>{format_value(row.get('Total Population (2024)'))}</td>
            <td>N/A</td><td>N/A</td>
            <td>N/A</td><td>N/A</td>
          </tr>
          <tr>
            <td>Anglo Pop</td>
            <td>{format_value(row.get('Anglo Population (2020)'))}</td>
            <td>{format_value(row.get('Anglo Population (2022)'))}</td>
            <td>{format_value(row.get('Anglo Population (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Anglo Pop % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Anglo Pop % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
           <tr>
            <td>Non-Anglo Pop</td>
            <td>{format_value(row.get('Non-Anglo Population (2020)'))}</td>
            <td>{format_value(row.get('Non-Anglo Population (2022)'))}</td>
            <td>{format_value(row.get('Non-Anglo Population (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Non-Anglo Pop % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Non-Anglo Pop % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
          <tr>
            <td>Asian Pop</td>
            <td>{format_value(row.get('Asian Population (2020)'))}</td>
            <td>{format_value(row.get('Asian Population (2022)'))}</td>
            <td>{format_value(row.get('Asian Population (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Asian Pop % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Asian Pop % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
          <tr>
            <td>Black Pop</td>
            <td>{format_value(row.get('Black Population (2020)'))}</td>
            <td>{format_value(row.get('Black Population (2022)'))}</td>
            <td>{format_value(row.get('Black Population (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Black Pop % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Black Pop % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
           <tr>
            <td>Hispanic Pop</td>
            <td>{format_value(row.get('Hispanic Population (2020)'))}</td>
            <td>{format_value(row.get('Hispanic Population (2022)'))}</td>
            <td>{format_value(row.get('Hispanic Population (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Hispanic Pop % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Hispanic Pop % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
           <tr>
            <td>Black + Hispanic Pop</td>
            <td>{format_value(row.get('Black + Hispanic Population (2020)'))}</td>
            <td>{format_value(row.get('Black + Hispanic Population (2022)'))}</td>
            <td>{format_value(row.get('Black + Hispanic Population (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Black + Hispanic Pop % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Black + Hispanic Pop % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
           <tr>
            <td>VAP</td>
            <td>{format_value(row.get('Voting Age Population (2020)'))}</td>
            <td>{format_value(row.get('Voting Age Population (2022)'))}</td>
            <td>{format_value(row.get('Voting Age Population (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('VAP % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('VAP % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
           <tr>
            <td>Anglo VAP</td>
            <td>{format_value(row.get('Anglo VAP (2020)'))}</td>
            <td>{format_value(row.get('Anglo VAP (2022)'))}</td>
            <td>{format_value(row.get('Anglo VAP (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Anglo VAP % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Anglo VAP % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
          <tr>
            <td>Non-Anglo VAP</td>
            <td>{format_value(row.get('Non-Anglo VAP (2020)'))}</td>
            <td>{format_value(row.get('Non-Anglo VAP (2022)'))}</td>
            <td>{format_value(row.get('Non-Anglo VAP (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Non-Anglo VAP % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Non-Anglo VAP % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
          <tr>
            <td>Asian VAP</td>
            <td>{format_value(row.get('Asian VAP (2020)'))}</td>
            <td>{format_value(row.get('Asian VAP (2022)'))}</td>
            <td>{format_value(row.get('Asian VAP (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Asian VAP % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Asian VAP % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
          <tr>
            <td>Black VAP</td>
            <td>{format_value(row.get('Black VAP (2020)'))}</td>
            <td>{format_value(row.get('Black VAP (2022)'))}</td>
            <td>{format_value(row.get('Black VAP (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Black VAP % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Black VAP % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
          <tr>
            <td>Hispanic VAP</td>
            <td>{format_value(row.get('Hispanic VAP (2020)'))}</td>
            <td>{format_value(row.get('Hispanic VAP (2022)'))}</td>
            <td>{format_value(row.get('Hispanic VAP (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Hispanic VAP % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Hispanic VAP % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
          <tr>
            <td>Black + Hispanic VAP</td>
            <td>{format_value(row.get('Black + Hispanic VAP (2020)'))}</td>
            <td>{format_value(row.get('Black + Hispanic VAP (2022)'))}</td>
            <td>{format_value(row.get('Black + Hispanic VAP (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Black + Hispanic VAP % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Black + Hispanic VAP % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
        </table>
        """, axis=1)

        print("✅ Created 'demographics_html' column for popup.")

    else:
        print("❌ Necessary demographic data for popup not available. Skipping HTML generation.")
        master_vtd_gdf['demographics_html'] = "Demographic data not available."

    # Print head and columns to verify
    print("\n📋 Head of master_vtd_gdf with 'demographics_html':")
    display(master_vtd_gdf[['CNTYVTD', 'demographics_html']].head())
    print("\nColumns of master_vtd_gdf after demographics popup preparation:")
    print(master_vtd_gdf.columns.tolist())

**Reasoning**:
Prepare the Primary Votes data for the popup dropdown. This involves selecting the relevant vote columns, ensuring they are numeric, and formatting them into an HTML snippet.



In [ ]:
import pandas as pd

# Ensure master_vtd_gdf and primary_votes_cleaned_df are available
if 'master_vtd_gdf' not in locals() or master_vtd_gdf is None or master_vtd_gdf.empty:
    print("❌ master_vtd_gdf GeoDataFrame is not available or empty. Cannot prepare primary vote data for popup.")
elif 'primary_votes_cleaned_df' not in locals() or primary_votes_cleaned_df is None or primary_votes_cleaned_df.empty:
    print("❌ primary_votes_cleaned_df DataFrame is not available or empty. Cannot prepare primary vote data for popup.")
else:
    print("✅ master_vtd_gdf and primary_votes_cleaned_df are available for primary vote popup data preparation.")

    # Define the relevant primary vote columns from primary_votes_cleaned_df
    # These are the cleaned and merged vote columns
    primary_vote_cols = [
        'BufordR_2022_Primary_Votes', 'EllzeyR_2022_Primary_Votes', 'PayneR_2022_Primary_Votes',
        'BufordR_2024_Primary_Votes', 'EllzeyR_2024_Primary_Votes', 'WileyR_22024_Primary_Votes' # Corrected typo WileyR_2024
    ]

    # Ensure these columns exist in master_vtd_gdf (they should after the merge in step 3)
    # and are numeric (should be after cleaning in step 2)
    for col in primary_vote_cols:
        if col not in master_vtd_gdf.columns:
            print(f"⚠️ Primary vote column '{col}' not found in master_vtd_gdf. Creating a placeholder.")
            master_vtd_gdf[col] = pd.NA # Use pandas NA for nullable integer

        # Ensure numeric type, coercing errors to NaN
        master_vtd_gdf[col] = pd.to_numeric(master_vtd_gdf[col], errors='coerce')


    # Define a helper function to format vote counts, handling NaNs
    def format_vote_count(count):
        if pd.notnull(count):
            try:
                return f"{int(count):,}"
            except:
                return str(count) # Return as string if conversion fails
        return 'N/A'


    # Create the Primary Votes HTML snippet for each VTD
    master_vtd_gdf['primary_votes_html'] = master_vtd_gdf.apply(lambda row: f"""
    <b>Primary Vote Data:</b><br>
    <table style="width:100%;">
      <tr>
        <th>Candidate</th>
        <th>2022 Primary</th>
        <th>2024 Primary</th>
      </tr>
      <tr>
        <td>Buford (R)</td>
        <td>{format_vote_count(row.get('BufordR_2022_Primary_Votes'))}</td>
        <td>{format_vote_count(row.get('BufordR_2024_Primary_Votes'))}</td>
      </tr>
      <tr>
        <td>Ellzey (R)</td>
        <td>{format_vote_count(row.get('EllzeyR_2022_Primary_Votes'))}</td>
        <td>{format_vote_count(row.get('EllzeyR_2024_Primary_Votes'))}</td>
      </tr>
      <tr>
        <td>Payne (R)</td>
        <td>{format_vote_count(row.get('PayneR_2022_Primary_Votes'))}</td>
        <td>N/A</td> <!-- Payne was not in the 2024 primary -->
      </tr>
      <tr>
        <td>Wiley (R)</td>
        <td>N/A</td> <!-- Wiley was not in the 2022 primary -->
        <td>{format_vote_count(row.get('WileyR_22024_Primary_Votes'))}</td> # Corrected typo WileyR_2024
      </tr>
    </table>
    """, axis=1)

    print("✅ Created 'primary_votes_html' column for popup.")

    # Print head and columns to verify
    print("\n📋 Head of master_vtd_gdf with 'primary_votes_html':")
    display(master_vtd_gdf[['CNTYVTD', 'primary_votes_html']].head())
    print("\nColumns of master_vtd_gdf after primary votes popup preparation:")
    print(master_vtd_gdf.columns.tolist())


**Reasoning**:
Prepare the Voter Registration and Turnout data for the popup dropdown. This involves selecting the relevant columns (excluding Spanish Surname data), ensuring they are numeric, calculating changes, and formatting them into an HTML snippet.



In [ ]:
import pandas as pd
import numpy as np

# Ensure master_vtd_gdf and cleaned_torv_df are available
if 'master_vtd_gdf' not in locals() or master_vtd_gdf is None or master_vtd_gdf.empty:
    print("❌ master_vtd_gdf GeoDataFrame is not available or empty. Cannot prepare VR/Turnout data for popup.")
elif 'cleaned_torv_df' not in locals() or cleaned_torv_df is None or cleaned_torv_df.empty:
    print("❌ cleaned_torv_df DataFrame is not available or empty. Cannot prepare VR/Turnout data for popup.")
else:
    print("✅ master_vtd_gdf and cleaned_torv_df are available for VR/Turnout popup data preparation.")

    # Define the relevant VR/Turnout columns from cleaned_torv_df (excluding Spanish Surname)
    vr_turnout_cols = [
        'Voter Registration 2020', 'Turnout 2020',
        'Voter Registration 2022', 'Turnout 2022',
        'Voter Registration 2024', 'Turnout 2024',
        'VR Change 2020-2022', 'VR % Change 2020-2022',
        'Turnout Change 2020-2022', 'Turnout % Change 2020-2022',
        'VR Change 2022-2024', 'VR % Change 2022-2024',
        'Turnout Change 2022-2024', 'Turnout % Change 2022-2024'
    ]

    # Ensure these columns exist in master_vtd_gdf (they should after the merge in step 3)
    # Ensure numeric types where applicable, coercing errors to NaN
    numeric_vr_turnout_cols = [
        'Voter Registration 2020', 'Turnout 2020',
        'Voter Registration 2022', 'Turnout 2022',
        'Voter Registration 2024', 'Turnout 2024',
        'VR Change 2020-2022', 'Turnout Change 2020-2022',
        'VR Change 2022-2024', 'Turnout Change 2022-2024'
    ]
    percentage_vr_turnout_cols = [
        'VR % Change 2020-2022', 'Turnout % Change 2020-2022',
        'VR % Change 2022-2024', 'Turnout % Change 2022-2024'
    ]


    for col in vr_turnout_cols:
        if col not in master_vtd_gdf.columns:
            print(f"⚠️ VR/Turnout column '{col}' not found in master_vtd_gdf. Creating a placeholder.")
            master_vtd_gdf[col] = np.nan # Use NaN for missing numeric/percentage data

        # Ensure numeric type for numeric columns
        if col in numeric_vr_turnout_cols:
             master_vtd_gdf[col] = pd.to_numeric(master_vtd_gdf[col], errors='coerce')
        # Percentage columns might be strings like "2.35%", format them later if needed
        # For now, ensure they are present or NaN


    # Define a helper function to format numeric values and changes, handling NaNs
    def format_vr_turnout_value(value):
        if pd.notnull(value):
            try:
                # Handle potential float values from percentage columns
                if isinstance(value, str) and '%' in value:
                     return str(value) # Keep the string format for percentage
                else:
                     return f"{int(float(value)):,}" # Format numeric with comma
            except:
                return str(value) # Return as string if conversion fails
        return 'N/A'

    # Create the VR/Turnout HTML snippet for each VTD
    master_vtd_gdf['vr_turnout_html'] = master_vtd_gdf.apply(lambda row: f"""
    <b>Voter Registration & Turnout Data:</b><br>
    <table style="width:100%;">
      <tr>
        <th>Metric</th>
        <th>2020</th>
        <th>2022</th>
        <th>2024</th>
        <th>Change (20-22)</th>
        <th>% Change (20-22)</th>
        <th>Change (22-24)</th>
        <th>% Change (22-24)</th>
      </tr>
      <tr>
        <td>Registered Voters</td>
        <td>{format_vr_turnout_value(row.get('Voter Registration 2020'))}</td>
        <td>{format_vr_turnout_value(row.get('Voter Registration 2022'))}</td>
        <td>{format_vr_turnout_value(row.get('Voter Registration 2024'))}</td>
        <td>{format_vr_turnout_value(row.get('VR Change 2020-2022'))}</td>
        <td>{format_vr_turnout_value(row.get('VR % Change 2020-2022'))}</td>
        <td>{format_vr_turnout_value(row.get('VR Change 2022-2024'))}</td>
        <td>{format_vr_turnout_value(row.get('VR % Change 2022-2024'))}</td>
      </tr>
       <tr>
        <td>Turnout</td>
        <td>{format_vr_turnout_value(row.get('Turnout 2020'))}</td>
        <td>{format_vr_turnout_value(row.get('Turnout 2022'))}</td>
        <td>{format_vr_turnout_value(row.get('Turnout 2024'))}</td>
        <td>{format_vr_turnout_value(row.get('Turnout Change 2020-2022'))}</td>
        <td>{format_vr_turnout_value(row.get('Turnout % Change 2020-2022'))}</td>
        <td>{format_vr_turnout_value(row.get('Turnout Change 2022-2024'))}</td>
        <td>{format_vr_turnout_value(row.get('Turnout % Change 2022-2024'))}</td>
      </tr>
    </table>
    """, axis=1)

    print("✅ Created 'vr_turnout_html' column for popup.")

    # Print head and columns to verify
    print("\n📋 Head of master_vtd_gdf with 'vr_turnout_html':")
    display(master_vtd_gdf[['CNTYVTD', 'vr_turnout_html']].head())
    print("\nColumns of master_vtd_gdf after VR/Turnout popup preparation:")
    print(master_vtd_gdf.columns.tolist())


## Prepare data for interactive popup

### Subtask:
Implement the JavaScript/HTML structure for the dropdown menu within the popup. This might involve creating a single HTML string per VTD that includes the logic for showing/hiding sections based on the dropdown selection.


**Reasoning**:
Define a function to generate the full HTML for the popup including the dropdown and content sections, apply it to the GeoDataFrame, and display the head.



In [ ]:
import pandas as pd # Ensure pandas is imported

# 1. Define a function that takes a row from master_vtd_gdf and generates the full HTML for the popup.
def generate_full_popup_html(row):
    # Use row.get() with a default empty string to handle potential missing HTML columns gracefully
    basic_info = row.get('Basic_Info_HTML', '')
    demographics_html = row.get('demographics_html', '<p>Demographic data not available.</p>')
    primary_votes_html = row.get('primary_votes_html', '<p>Primary vote data not available.</p>')
    vr_turnout_html = row.get('vr_turnout_html', '<p>VR/Turnout data not available.</p>')

    # Generate the full HTML structure with a dropdown and hidden content sections
    html_content = f"""
    <div style="width: 350px;">
        {basic_info}
        <hr>
        <label for="data-select-{row.name}">View:</label>
        <select id="data-select-{row.name}" onchange="showDataSection(this.value, '{row.name}')">
            <option value="demographics">Demographics</option>
            <option value="primary_votes">Primary Votes</option>
            <option value="vr_turnout">VR/Turnout</option>
        </select>
        <hr>
        <div id="demographics-section-{row.name}" class="data-section">
            {demographics_html}
        </div>
        <div id="primary_votes-section-{row.name}" class="data-section" style="display:none;">
            {primary_votes_html}
        </div>
        <div id="vr_turnout-section-{row.name}" class="data-section" style="display:none;">
            {vr_turnout_html}
        </div>
    </div>

    <script>
        function showDataSection(sectionId, rowId) {{
            // Hide all sections for this specific VTD row
            var sections = document.querySelectorAll('#data-select-' + rowId + ' ~ .data-section');
            sections.forEach(function(section) {{
                section.style.display = 'none';
            }});

            // Show the selected section for this specific VTD row
            var selectedSection = document.getElementById(sectionId + '-section-' + rowId);
            if (selectedSection) {{
                selectedSection.style.display = 'block';
            }}
        }}

        // Initial display: Ensure only the first section is shown on load
        document.addEventListener('DOMContentLoaded', function() {{
             // Find all dropdowns and trigger the showDataSection for the initially selected option
             var dropdowns = document.querySelectorAll('select[id^="data-select-"]');
             dropdowns.forEach(function(dropdown) {{
                 // Extract rowId from the dropdown ID
                 var rowId = dropdown.id.replace('data-select-', '');
                 showDataSection(dropdown.value, rowId);
             }});
        }});

    </script>
    """
    return html_content

# 2. Apply this function to each row of master_vtd_gdf to create a new column 'full_popup_html'.
# Check if master_vtd_gdf exists and has the necessary HTML columns
required_html_cols = ['Basic_Info_HTML', 'demographics_html', 'primary_votes_html', 'vr_turnout_html']
if 'master_vtd_gdf' in locals() and master_vtd_gdf is not None and not master_vtd_gdf.empty and all(col in master_vtd_gdf.columns for col in required_html_cols):
    master_vtd_gdf['full_popup_html'] = master_vtd_gdf.apply(generate_full_popup_html, axis=1)
    print("✅ Created 'full_popup_html' column with interactive popup HTML.")
else:
    print("❌ master_vtd_gdf not available or missing required HTML columns. Cannot create full popup HTML.")
    # Create an empty full_popup_html column to avoid errors in subsequent steps
    if 'master_vtd_gdf' in locals() and master_vtd_gdf is not None:
         master_vtd_gdf['full_popup_html'] = "Popup data not available."
    else:
         print("⚠️ master_vtd_gdf is not defined.")


# 3. Display the head of the master_vtd_gdf GeoDataFrame showing the new 'full_popup_html' column.
if 'master_vtd_gdf' in locals() and master_vtd_gdf is not None:
    print("\n📋 Head of master_vtd_gdf with 'full_popup_html':")
    display(master_vtd_gdf[['CNTYVTD', 'full_popup_html']].head())
else:
    print("\n❌ master_vtd_gdf is not available to display head.")

**Reasoning**:
The previous attempt to create the full popup HTML failed because the required HTML columns were not present in `master_vtd_gdf`. I need to re-run the data preparation step (step 4 from the previous task) to ensure these columns are created and merged correctly before attempting to generate the final popup HTML again.



In [ ]:
import pandas as pd
import numpy as np

print("--- Re-running Data Preparation to ensure HTML columns are present ---")

# Ensure master_vtd_gdf is available and the base data is correct
if 'master_vtd_gdf' not in locals() or master_vtd_gdf is None or master_vtd_gdf.empty:
    print("❌ master_vtd_gdf is not available. Cannot re-prepare HTML columns.")
    # Stop here if the base GeoDataFrame is missing
else:
    print("✅ master_vtd_gdf is available.")

    # --- Prepare Basic Info HTML (based on logic from step 4) ---
    print("\n--- Preparing Basic Info HTML ---")
    # Use the cleaned columns from the demographics_geo_df merge for basic info
    basic_info_cols = ['CNTYVTD', 'Best Matching Precinct Name (Max Overlap)', 'County Name (from TORV)', 'Cities (from TORV)']

    # Ensure these columns exist in master_vtd_gdf and are string type, fill NaNs
    for col in basic_info_cols:
        if col not in master_vtd_gdf.columns:
            print(f"⚠️ Basic info column '{col}' not found. Creating a placeholder.")
            master_vtd_gdf[col] = 'N/A'
        else:
            master_vtd_gdf[col] = master_vtd_gdf[col].astype(str).fillna('N/A')

    master_vtd_gdf['Basic_Info_HTML'] = master_vtd_gdf.apply(lambda row: f"""
    <b>VTD Identifier:</b> {row.get('CNTYVTD', 'N/A')}<br>
    <b>Best Matching Precinct:</b> {row.get('Best Matching Precinct Name (Max Overlap)', 'N/A')}<br>
    <b>County Name:</b> {row.get('County Name (from TORV)', 'N/A')}<br>
    <b>Cities Intersecting VTD:</b> {row.get('Cities (from TORV)', 'N/A')}
    """, axis=1)
    print("✅ Prepared 'Basic_Info_HTML'.")


    # --- Prepare Demographic Data HTML (based on logic from step 4) ---
    print("\n--- Preparing Demographic Data HTML ---")
    # Ensure demographic columns are numeric and handled
    demographic_cols_check = [
        'Total Population (2020)', 'Total Population (2022)', 'Total Population (2024)',
        'Anglo Population (2020)', 'Anglo Population (2022)', 'Anglo Population (2024)',
        'Non-Anglo Population (2020)', 'Non-Anglo Population (2022)', 'Non-Anglo Population (2024)',
        'Asian Population (2020)', 'Asian Population (2022)', 'Asian Population (2024)',
        'Black Population (2020)', 'Black Population (2022)', 'Black Population (2024)',
        'Hispanic Population (2020)', 'Hispanic Population (2022)', 'Hispanic Population (2024)',
        'Black + Hispanic Population (2020)', 'Black + Hispanic Population (2022)', 'Black + Hispanic Population (2024)',
        'Voting Age Population (2020)', 'Voting Age Population (2022)', 'Voting Age Population (2024)',
        'Anglo VAP (2020)', 'Anglo VAP (2022)', 'Anglo VAP (2024)',
        'Non-Anglo VAP (2020)', 'Non-Anglo VAP (2022)', 'Non-Anglo VAP (2024)',
        'Asian VAP (2020)', 'Asian VAP (2022)', 'Asian VAP (2024)',
        'Black VAP (2020)', 'Black VAP (2022)', 'Black VAP (2024)',
        'Hispanic VAP (2020)', 'Hispanic VAP (2022)', 'Hispanic VAP (2024)',
        'Black + Hispanic VAP (2020)', 'Black + Hispanic VAP (2022)', 'Black + Hispanic VAP (2024)',
        'Total Pop % Change (2020-2022)', 'Total Pop % Change (2022-2024)',
        'Anglo Pop % Change (2020-2022)', 'Anglo Pop % Change (2022-2024)',
        'Non-Anglo Pop % Change (2020-2022)', 'Non-Anglo Pop % Change (2022-2024)',
        'Asian Pop % Change (2020-2022)', 'Asian Pop % Change (2022-2024)',
        'Black Pop % Change (2020-2022)', 'Black Pop % Change (2022-2024)',
        'Hispanic Pop % Change (2020-2022)', 'Hispanic Pop % Change (2022-2024)',
        'Black + Hispanic Pop % Change (2020-2022)', 'Black + Hispanic Pop % Change (2022-2024)',
        'VAP % Change (2020-2022)', 'VAP % Change (2022-2024)',
        'Anglo VAP % Change (2020-2022)', 'Anglo VAP % Change (2022-2024)',
        'Non-Anglo VAP % Change (2020-2022)', 'Non-Anglo VAP % Change (2022-2024)',
        'Asian VAP % Change (2020-2022)', 'Asian VAP % Change (2022-2024)',
        'Black VAP % Change (2020-2022)', 'Black VAP % Change (2022-2024)',
        'Hispanic VAP % Change (2020-2022)', 'Hispanic VAP % Change (2022-2024)',
        'Black + Hispanic VAP % Change (2020-2022)', 'Black + Hispanic VAP % Change (2022-2024)'
    ]
    for col in demographic_cols_check:
        if col in master_vtd_gdf.columns:
            # Convert to numeric, coercing errors, except for percentage strings
             if not any(term in col.lower() for term in ['change', '%']): # Don't try to convert original percentages if they are strings
                master_vtd_gdf[col] = pd.to_numeric(master_vtd_gdf[col], errors='coerce')
        else:
            print(f"⚠️ Demographic column '{col}' not found. Creating a placeholder.")
            master_vtd_gdf[col] = np.nan


    # Define a helper function to format values (numeric or percentage strings), handling NaNs
    def format_value_robust(value):
        if pd.notnull(value):
            try:
                if isinstance(value, str) and '%' in value:
                    return str(value) # Keep percentage strings as is
                else:
                    return f"{int(float(value)):,}" # Format numeric with comma
            except:
                return str(value) # Return as string if conversion fails
        return 'N/A'

    master_vtd_gdf['demographics_html'] = master_vtd_gdf.apply(lambda row: f"""
    <b>Demographic Data:</b><br>
    <table style="width:100%;">
      <tr>
        <th>Category</th>
        <th>2020</th>
        <th>2022</th>
        <th>2024</th>
        <th>Change (20-22)</th>
        <th>% Change (20-22)</th>
        <th>Change (22-24)</th>
        <th>% Change (22-24)</th>
      </tr>
      <tr>
        <td>Total Pop</td>
        <td>{format_value_robust(row.get('Total Population (2020)'))}</td>
        <td>{format_value_robust(row.get('Total Population (2022)'))}</td>
        <td>{format_value_robust(row.get('Total Population (2024)'))}</td>
        <td>{format_value_robust(row.get('Total Pop Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Total Pop % Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Total Pop Change (2022-2024)'))}</td>
        <td>{format_value_robust(row.get('Total Pop % Change (2022-2024)'))}</td>
      </tr>
      <tr>
        <td>Anglo Pop</td>
        <td>{format_value_robust(row.get('Anglo Population (2020)'))}</td>
        <td>{format_value_robust(row.get('Anglo Population (2022)'))}</td>
        <td>{format_value_robust(row.get('Anglo Population (2024)'))}</td>
        <td>{format_value_robust(row.get('Anglo Pop Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Anglo Pop % Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Anglo Pop Change (2022-2024)'))}</td>
        <td>{format_value_robust(row.get('Anglo Pop % Change (2022-2024)'))}</td>
      </tr>
       <tr>
        <td>Non-Anglo Pop</td>
        <td>{format_value_robust(row.get('Non-Anglo Population (2020)'))}</td>
        <td>{format_value_robust(row.get('Non-Anglo Population (2022)'))}</td>
        <td>{format_value_robust(row.get('Non-Anglo Population (2024)'))}</td>
        <td>{format_value_robust(row.get('Non-Anglo Pop Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Non-Anglo Pop % Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Non-Anglo Pop Change (2022-2024)'))}</td>
        <td>{format_value_robust(row.get('Non-Anglo Pop % Change (2022-2024)'))}</td>
      </tr>
      <tr>
        <td>Asian Pop</td>
        <td>{format_value_robust(row.get('Asian Population (2020)'))}</td>
        <td>{format_value_robust(row.get('Asian Population (2022)'))}</td>
        <td>{format_value_robust(row.get('Asian Population (2024)'))}</td>
        <td>{format_value_robust(row.get('Asian Pop Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Asian Pop % Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Asian Pop Change (2022-2024)'))}</td>
        <td>{format_value_robust(row.get('Asian Pop % Change (2022-2024)'))}</td>
      </tr>
      <tr>
        <td>Black Pop</td>
        <td>{format_value_robust(row.get('Black Population (2020)'))}</td>
        <td>{format_value_robust(row.get('Black Population (2022)'))}</td>
        <td>{format_value_robust(row.get('Black Population (2024)'))}</td>
        <td>{format_value_robust(row.get('Black Pop Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Black Pop % Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Black Pop Change (2022-2024)'))}</td>
        <td>{format_value_robust(row.get('Black Pop % Change (2022-2024)'))}</td>
      </tr>
       <tr>
        <td>Hispanic Pop</td>
        <td>{format_value_robust(row.get('Hispanic Population (2020)'))}</td>
        <td>{format_value_robust(row.get('Hispanic Population (2022)'))}</td>
        <td>{format_value_robust(row.get('Hispanic Population (2024)'))}</td>
        <td>{format_value_robust(row.get('Hispanic Pop Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Hispanic Pop % Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Hispanic Pop Change (2022-2024)'))}</td>
        <td>{format_value_robust(row.get('Hispanic Pop % Change (2022-2024)'))}</td>
      </tr>
       <tr>
        <td>Black + Hispanic Pop</td>
        <td>{format_value_robust(row.get('Black + Hispanic Population (2020)'))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic Population (2022)'))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic Population (2024)'))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic Pop Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic Pop % Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic Pop Change (2022-2024)'))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic Pop % Change (2022-2024)'))}</td>
      </tr>
       <tr>
        <td>VAP</td>
        <td>{format_value_robust(row.get('Voting Age Population (2020)'))}</td>
        <td>{format_value_robust(row.get('Voting Age Population (2022)'))}</td>
        <td>{format_value_robust(row.get('Voting Age Population (2024)'))}</td>
         <td>{format_value_robust(row.get('VAP Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('VAP % Change (2020-2022)'))}</td>
         <td>{format_value_robust(row.get('VAP Change (2022-2024)'))}</td>
        <td>{format_value_robust(row.get('VAP % Change (2022-2024)'))}</td>
      </tr>
       <tr>
        <td>Anglo VAP</td>
        <td>{format_value_robust(row.get('Anglo VAP (2020)'))}</td>
        <td>{format_value_robust(row.get('Anglo VAP (2022)'))}</td>
        <td>{format_value_robust(row.get('Anglo VAP (2024)'))}</td>
         <td>{format_value_robust(row.get('Anglo VAP Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Anglo VAP % Change (2020-2022)'))}</td>
         <td>{format_value_robust(row.get('Anglo VAP Change (2022-2024)'))}</td>
        <td>{format_value_robust(row.get('Anglo VAP % Change (2022-2024)'))}</td>
      </tr>
      <tr>
        <td>Non-Anglo VAP</td>
        <td>{format_value_robust(row.get('Non-Anglo VAP (2020)'))}</td>
        <td>{format_value_robust(row.get('Non-Anglo VAP (2022)'))}</td>
        <td>{format_value_robust(row.get('Non-Anglo VAP (2024)'))}</td>
        <td>{format_value_robust(row.get('Non-Anglo VAP Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Non-Anglo VAP % Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Non-Anglo VAP Change (2022-2024)'))}</td>
        <td>{format_value_robust(row.get('Non-Anglo VAP % Change (2022-2024)'))}</td>
      </tr>
      <tr>
        <td>Asian VAP</td>
        <td>{format_value_robust(row.get('Asian VAP (2020)'))}</td>
        <td>{format_value_robust(row.get('Asian VAP (2022)'))}</td>
        <td>{format_value_robust(row.get('Asian VAP (2024)'))}</td>
        <td>{format_value_robust(row.get('Asian VAP Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Asian VAP % Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Asian VAP Change (2022-2024)'))}</td>
        <td>{format_value_robust(row.get('Asian VAP % Change (2022-2024)'))}</td>
      </tr>
      <tr>
        <td>Black VAP</td>
        <td>{format_value_robust(row.get('Black VAP (2020)'))}</td>
        <td>{format_value_robust(row.get('Black VAP (2022)'))}</td>
        <td>{format_value_robust(row.get('Black VAP (2024)'))}</td>
        <td>{format_value_robust(row.get('Black VAP Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Black VAP % Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Black VAP Change (2022-2024)'))}</td>
        <td>{format_value_robust(row.get('Black VAP % Change (2022-2024)'))}</td>
      </tr>
      <tr>
        <td>Hispanic VAP</td>
        <td>{format_value_robust(row.get('Hispanic VAP (2020)'))}</td>
        <td>{format_value_robust(row.get('Hispanic VAP (2022)'))}</td>
        <td>{format_value_robust(row.get('Hispanic VAP (2024)'))}</td>
        <td>{format_value_robust(row.get('Hispanic VAP Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Hispanic VAP % Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Hispanic VAP Change (2022-2024)'))}</td>
        <td>{format_value_robust(row.get('Hispanic VAP % Change (2022-2024)'))}</td>
      </tr>
      <tr>
        <td>Black + Hispanic VAP</td>
        <td>{format_value_robust(row.get('Black + Hispanic VAP (2020)'))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic VAP (2022)'))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic VAP (2024)'))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic VAP Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic VAP % Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic VAP Change (2022-2024)'))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic VAP % Change (2022-2024)'))}</td>
      </tr>
    </table>
    """, axis=1)

    print("✅ Prepared 'demographics_html'.")


    # --- Prepare Primary Votes Data HTML (based on logic from step 4) ---
    print("\n--- Preparing Primary Vote Data HTML ---")
    # Define the relevant primary vote columns from primary_votes_cleaned_df
    primary_vote_cols_check = [
        'BufordR_2022_Primary_Votes', 'EllzeyR_2022_Primary_Votes', 'PayneR_2022_Primary_Votes',
        'BufordR_2024_Primary_Votes', 'EllzeyR_2024_Primary_Votes', 'WileyR_22024_Primary_Votes' # Corrected typo WileyR_2024
    ]
    for col in primary_vote_cols_check:
        if col not in master_vtd_gdf.columns:
            print(f"⚠️ Primary vote column '{col}' not found. Creating a placeholder.")
            master_vtd_gdf[col] = np.nan
        else:
             master_vtd_gdf[col] = pd.to_numeric(master_vtd_gdf[col], errors='coerce')


    master_vtd_gdf['primary_votes_html'] = master_vtd_gdf.apply(lambda row: f"""
    <b>Primary Vote Data:</b><br>
    <table style="width:100%;">
      <tr>
        <th>Candidate</th>
        <th>2022 Primary</th>
        <th>2024 Primary</th>
      </tr>
      <tr>
        <td>Buford (R)</td>
        <td>{format_value_robust(row.get('BufordR_2022_Primary_Votes'))}</td>
        <td>{format_value_robust(row.get('BufordR_2024_Primary_Votes'))}</td>
      </tr>
      <tr>
        <td>Ellzey (R)</td>
        <td>{format_value_robust(row.get('EllzeyR_2022_Primary_Votes'))}</td>
        <td>{format_value_robust(row.get('EllzeyR_2024_Primary_Votes'))}</td>
      </tr>
      <tr>
        <td>Payne (R)</td>
        <td>{format_value_robust(row.get('PayneR_2022_Primary_Votes'))}</td>
        <td>N/A</td> <!-- Payne was not in the 2024 primary -->
      </tr>
      <tr>
        <td>Wiley (R)</td>
        <td>N/A</td> <!-- Wiley was not in the 2022 primary -->
        <td>{format_value_robust(row.get('WileyR_22024_Primary_Votes'))}</td> # Corrected typo WileyR_2024
      </tr>
    </table>
    """, axis=1)
    print("✅ Prepared 'primary_votes_html'.")


    # --- Prepare VR/Turnout Data HTML (based on logic from step 4) ---
    print("\n--- Preparing VR/Turnout Data HTML ---")
    vr_turnout_cols_check = [
        'Voter Registration 2020', 'Turnout 2020',
        'Voter Registration 2022', 'Turnout 2022',
        'Voter Registration 2024', 'Turnout 2024',
        'VR Change 2020-2022', 'VR % Change 2020-2022',
        'Turnout Change 2020-2022', 'Turnout % Change 2020-2022',
        'VR Change 2022-2024', 'VR % Change 2022-2024',
        'Turnout Change 2022-2024', 'Turnout % Change 2022-2024'
    ]
    for col in vr_turnout_cols_check:
        if col not in master_vtd_gdf.columns:
            print(f"⚠️ VR/Turnout column '{col}' not found. Creating a placeholder.")
            master_vtd_gdf[col] = np.nan
        else:
             # Convert numeric columns to numeric, leave percentage strings as they are formatted
             if not any(term in col.lower() for term in ['change', '%']):
                master_vtd_gdf[col] = pd.to_numeric(master_vtd_gdf[col], errors='coerce')


    master_vtd_gdf['vr_turnout_html'] = master_vtd_gdf.apply(lambda row: f"""
    <b>Voter Registration & Turnout Data:</b><br>
    <table style="width:100%;">
      <tr>
        <th>Metric</th>
        <th>2020</th>
        <th>2022</th>
        <th>2024</th>
        <th>Change (20-22)</th>
        <th>% Change (20-22)</th>
        <th>Change (22-24)</th>
        <th>% Change (22-24)</th>
      </tr>
      <tr>
        <td>Registered Voters</td>
        <td>{format_value_robust(row.get('Voter Registration 2020'))}</td>
        <td>{format_value_robust(row.get('Voter Registration 2022'))}</td>
        <td>{format_value_robust(row.get('Voter Registration 2024'))}</td>
        <td>{format_value_robust(row.get('VR Change 2020-2022'))}</td>
        <td>{format_value_robust(row.get('VR % Change 2020-2022'))}</td>
        <td>{format_value_robust(row.get('VR Change 2022-2024'))}</td>
        <td>{format_value_robust(row.get('VR % Change 2022-2024'))}</td>
      </tr>
       <tr>
        <td>Turnout</td>
        <td>{format_value_robust(row.get('Turnout 2020'))}</td>
        <td>{format_value_robust(row.get('Turnout 2022'))}</td>
        <td>{format_value_robust(row.get('Turnout 2024'))}</td>
        <td>{format_value_robust(row.get('Turnout Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Turnout % Change (2020-2022)'))}</td>
        <td>{format_value_robust(row.get('Turnout Change (2022-2024)'))}</td>
        <td>{format_value_robust(row.get('Turnout % Change (2022-2024)'))}</td>
      </tr>
    </table>
    """, axis=1)
    print("✅ Prepared 'vr_turnout_html'.")

    print("\n--- HTML snippets re-prepared. Proceeding to generate full popup HTML. ---")

    # --- Generate Full Popup HTML ---
    master_vtd_gdf['full_popup_html'] = master_vtd_gdf.apply(generate_full_popup_html, axis=1)
    print("✅ Created 'full_popup_html' column with interactive popup HTML.")


    # --- Final Check ---
    print("\n📋 Head of master_vtd_gdf with 'full_popup_html':")
    display(master_vtd_gdf[['CNTYVTD', 'full_popup_html']].head())
    print("\nColumns of master_vtd_gdf after re-preparation and full popup HTML generation:")
    print(master_vtd_gdf.columns.tolist())

## Load additional geographic layers

### Subtask:
Load the County boundaries from `texas_county_shapefiles/County_Boundaries.shp`. Reproject this GeoDataFrame to EPSG:4326 to be compatible with Folium.


**Reasoning**:
Load the County boundaries shapefile and reproject it to EPSG:4326 for Folium compatibility.



In [ ]:
# Define the path to the County boundaries shapefile.
counties_shapefile_path = "texas_county_shapefiles/County_Boundaries.shp"

# Define the target CRS for Folium.
folium_crs = "EPSG:4326"

# 1. Read the shapefile into a GeoDataFrame.
if os.path.exists(counties_shapefile_path):
    try:
        counties_gdf_4326 = gpd.read_file(counties_shapefile_path)
        print(f"✅ Loaded county boundaries from {counties_shapefile_path}.")

        # Check the current CRS and reproject if necessary.
        if counties_gdf_4326.crs is None:
             print(f"⚠️ {counties_shapefile_path} CRS missing, assuming EPSG:3857 and setting to {folium_crs}")
             counties_gdf_4326.set_crs("EPSG:3857", inplace=True) # Assume a common web CRS if missing

        # 2. Reproject the loaded GeoDataFrame to EPSG:4326.
        if counties_gdf_4326.crs != folium_crs:
             counties_gdf_4326 = counties_gdf_4326.to_crs(folium_crs)
             print(f"✅ Reprojected counties_gdf to {folium_crs}.")
        else:
             print(f"✅ counties_gdf is already in {folium_crs}.")

        # 3. Print a confirmation message and display the head.
        print(f"📋 Head of reprojected counties_gdf_4326:")
        display(counties_gdf_4326.head())
        print(f"CRS: {counties_gdf_4326.crs}")

    except Exception as e:
        print(f"❌ Error loading or processing {counties_shapefile_path}: {e}")
        counties_gdf_4326 = None # Ensure variable is None if loading fails
else:
    print(f"❌ {counties_shapefile_path} not found. Cannot proceed.")
    counties_gdf_4326 = None # Ensure variable is None if file not found

## Create folium map

### Subtask:
Initialize a Folium map centered on the CD6 area.


**Reasoning**:
Initialize a Folium map centered on the CD6 area by calculating the centroid of the CD6 boundary or using a default location if the boundary is not available.



In [ ]:
import folium

# Define the target CRS for Folium (already defined but repeated for clarity)
folium_crs = "EPSG:4326"

# 1. Check if the cd6_boundary GeoDataFrame exists and is not empty.
# Assuming cd6_boundary was created and filtered in a previous step (e.g., from cd_gdf)
cd6_boundary = None # Initialize cd6_boundary to None

# Attempt to load and filter CD6 boundary if cd_gdf is available
if 'cd_gdf' in locals() and cd_gdf is not None and not cd_gdf.empty and 'District' in cd_gdf.columns:
    # Ensure cd_gdf is in a projected CRS for accurate centroid calculation if needed,
    # then reproject the filtered CD6 boundary to EPSG:4326 for Folium.
    try:
        # Use the projected CRS defined earlier (e.g., target_crs) if available
        proj_crs = "EPSG:3083" # Assuming this was the projected CRS used earlier
        if cd_gdf.crs != proj_crs:
             print(f"🔄 Reprojecting cd_gdf from {cd_gdf.crs} to {proj_crs} for centroid calculation.")
             cd_gdf_proj_temp = cd_gdf.to_crs(proj_crs)
        else:
             cd_gdf_proj_temp = cd_gdf.copy()

        cd6_boundary_temp = cd_gdf_proj_temp[cd_gdf_proj_temp['District'] == 6].copy()

        if not cd6_boundary_temp.empty:
            # 2. If available and contains valid geometry, calculate its centroid and convert to latitude/longitude.
            if 'geometry' in cd6_boundary_temp.columns and not cd6_boundary_temp.geometry.is_empty.all():
                center_point = cd6_boundary_temp.geometry.unary_union.centroid
                center_latlon = gpd.GeoSeries([center_point], crs=cd6_boundary_temp.crs).to_crs(epsg=4326).iloc[0].coords[0][::-1]
                print(f"✅ Calculated map center from CD6 boundary centroid: {center_latlon}")
                cd6_boundary = cd6_boundary_temp.to_crs(folium_crs) # Keep the 4326 version for later
            else:
                print("⚠️ CD6 boundary geometry is empty or invalid. Using default center.")
                center_latlon = [31.5, -97.5] # Default center
                cd6_boundary = cd6_boundary_temp.to_crs(folium_crs) # Keep the 4326 version for later
        else:
            print("❌ CD6 boundary not found after filtering. Using default center.")
            center_latlon = [31.5, -97.5] # Default center
            cd6_boundary = gpd.GeoDataFrame({'geometry': []}, crs=folium_crs) # Create empty gdf in 4326

    except Exception as e:
        print(f"❌ Error processing CD6 boundary for centering: {e}. Using default center.")
        center_latlon = [31.5, -97.5] # Default center
        cd6_boundary = gpd.GeoDataFrame({'geometry': []}, crs=folium_crs) # Create empty gdf in 4326

else:
    # 3. If cd6_boundary is not available, define a default latitude and longitude.
    print("❌ cd_gdf or CD6 boundary not available from previous steps. Using default center.")
    center_latlon = [31.5, -97.5] # Default center
    cd6_boundary = gpd.GeoDataFrame({'geometry': []}, crs=folium_crs) # Create empty gdf in 4326


# 4. Initialize a folium.Map object.
# 5. Set an appropriate zoom_start level.
# 6. Set the tiles parameter.
# 7. Store the created map object in a variable (m).
m = folium.Map(location=center_latlon, zoom_start=8.5, tiles="cartodbpositron")

print("✅ Initialized Folium map centered on the CD6 area (or default).")
print("\n--- Initial Map Object ---")
# Display the map object
display(m)

**Reasoning**:
Add the County boundaries layer to the Folium map.



In [ ]:
import folium
from folium.features import GeoJsonTooltip

# Ensure counties_gdf_4326 is available and not empty
if 'counties_gdf_4326' in locals() and counties_gdf_4326 is not None and not counties_gdf_4326.empty:
    print("✅ counties_gdf_4326 is available for adding to the map.")

    # Add the County boundaries layer to the map
    folium.GeoJson(
        counties_gdf_4326,
        name="All County Boundaries", # Layer name for control
        style_function=lambda x: {
            'color': 'black',  # Outline color
            'weight': 1.5,     # Line thickness
            'fillOpacity': 0,  # No fill
        },
        tooltip=GeoJsonTooltip(fields=['CNTY_NM'], aliases=['County:']) # Tooltip on hover
    ).add_to(m) # Add to the map object 'm' created in the previous step
    print("✅ Added All County Boundaries layer to the map.")

else:
    print("❌ counties_gdf_4326 GeoDataFrame not available or empty. Cannot add County Boundaries layer.")

# Display the map with the added layer
print("\n--- Map with County Boundaries Layer ---")
display(m)

**Reasoning**:
Add the CD6 boundary layer to the Folium map.



In [ ]:
import folium
from folium.features import GeoJsonTooltip

# Ensure cd6_boundary GeoDataFrame in EPSG:4326 is available and not empty
if 'cd6_boundary' in locals() and cd6_boundary is not None and not cd6_boundary.empty:
    print("✅ cd6_boundary is available for adding to the map.")

    # Add the CD6 boundary layer to the map
    folium.GeoJson(
        cd6_boundary,
        name="CD6 Boundary", # Layer name for control
        style_function=lambda x: {
            'color': 'blue',  # Outline color
            'weight': 3,     # Line thickness
            'fillOpacity': 0.1, # Light fill
        },
        tooltip=GeoJsonTooltip(fields=['District'], aliases=['District:']) # Tooltip on hover
    ).add_to(m) # Add to the map object 'm'

    print("✅ Added CD6 Boundary layer to the map.")

else:
    print("❌ cd6_boundary GeoDataFrame not available or empty. Cannot add CD6 Boundary layer.")

# Display the map with the added layer
print("\n--- Map with CD6 Boundary Layer ---")
display(m)

**Reasoning**:
Add the VTD layer to the Folium map with specified styling, tooltip, and popup.



In [ ]:
import folium
from folium.features import GeoJsonTooltip, GeoJsonPopup

# Ensure master_vtd_gdf is available, not empty, and in EPSG:4326
if 'master_vtd_gdf' in locals() and master_vtd_gdf is not None and not master_vtd_gdf.empty:
    print("✅ master_vtd_gdf is available for adding to the map.")

    # Reproject to EPSG:4326 for Folium if necessary
    folium_crs = "EPSG:4326"
    if master_vtd_gdf.crs != folium_crs:
         print(f"🔄 Reprojecting master_vtd_gdf from {master_vtd_gdf.crs} to {folium_crs}.")
         vtds_cd6_map = master_vtd_gdf.to_crs(folium_crs).copy()
    else:
         vtds_cd6_map = master_vtd_gdf.copy()
         print(f"✅ master_vtd_gdf is already in {folium_crs}.")


    # Add the VTD layer to the map
    folium.GeoJson(
        vtds_cd6_map,
        name='CD6 VTDs', # Layer name for control
        style_function=lambda feature: {
            # Use the 'Map_Color' column determined in previous steps for fill color
            'fillColor': feature['properties'].get('Map_Color', 'gray'), # Default to gray if color is missing
            'color': 'darkgreen', # Outline color
            'weight': 0.8,        # Line thickness
            'fillOpacity': 0.6,   # Fill opacity
        },
        # Add tooltip on hover using the prepared 'tooltip_text'
        tooltip=GeoJsonTooltip(fields=['tooltip_text'], aliases=['Info:'], localize=True, sticky=True),
        # Add popup on click using the prepared 'full_popup_html'
        popup=GeoJsonPopup(fields=['full_popup_html'], labels=False, localize=True),
    ).add_to(m) # Add to the map object 'm'

    print("✅ Added CD6 VTDs layer to the map with styling, tooltip, and popup.")

else:
    print("❌ master_vtd_gdf GeoDataFrame not available or empty. Cannot add CD6 VTDs layer.")

# Add Layer Control to show/hide layers
folium.LayerControl(collapsed=False).add_to(m)
print("✅ Added Layer Control to the map.")

# Display the final map
print("\n--- Final Interactive Map ---")
display(m)

from google.colab import files

output_path = "/content/completedcd6_vtd_map_final.html"
m.save(output_path)
files.download(output_path)

print("✅ HTML saved and downloaded successfully")


In [ ]:
m.save('folium_map_0.html')

## Summary:

## Summary of Solving Process: Creating an Interactive CD6 Map

The goal was to create an interactive Folium map of CD6, displaying VTD and county boundaries, with hover tooltips and click popups providing detailed information.

**Key Findings from Code Execution:**

1.  **Data Loading and Reprojection:** Geographic data for VTDs (`vtds_cd6.geojson`), county boundaries (`County_Boundaries.shp`), precinct boundaries (`Precincts24G.shp`), and city boundaries (`City.shp`) were successfully loaded and reprojected to EPSG:3083 for initial processing.
2.  **Attribute Data Preparation and Merging:** Attribute data from `cleaned_torv_data.csv`, `DemographicsFinal.csv`, and `final2022-2024PrimaryVotes.csv` were loaded. The 'CNTYVTD' or equivalent columns were cleaned and standardized across all datasets. This attribute data was then successfully merged onto the VTD GeoDataFrame (`vtds_cd6_gdf_proj`) using 'CNTYVTD' as the key, creating the `master_vtd_gdf`.
3.  **Tooltip and Popup Data Preparation:** Columns required for the hover tooltip (County, Precinct, Cities) were selected, formatted, and combined into a `tooltip_text` column. Detailed data for the click popup (Demographics, Primary Votes, VR/Turnout) was extracted, formatted into HTML tables, and stored in separate HTML columns (`demographics_html`, `primary_votes_html`, `vr_turnout_html`).
4.  **Interactive Popup HTML Generation:** A Python function was created to combine the basic VTD information, a dropdown menu, and the generated HTML snippets into a single `full_popup_html` column for each VTD. This HTML includes JavaScript to control the visibility of the different data sections based on the dropdown selection within the popup.
5.  **Folium Map Creation:** A Folium map was initialized and centered on the CD6 area by calculating the centroid of the district boundary.
6.  **Layer Addition:** The county boundaries (reprojected to EPSG:4326), the CD6 boundary, and the VTD layer (`master_vtd_gdf`, reprojected to EPSG:4326) were added to the Folium map.
7.  **Tooltip and Popup Integration:** The `tooltip_text` column was successfully integrated into the VTD layer as a hover tooltip. The `full_popup_html` column was successfully integrated as the content for the click popup, enabling the interactive dropdown functionality within the map.
8.  **Layer Control:** A Layer Control was added to the map, allowing users to toggle the visibility of the different geographic layers.

**Final Outcome:**

The task was successfully completed. An interactive Folium map of CD6 has been created. The map displays the boundaries of VTDs and counties. Hovering over a VTD displays a tooltip with County, Precinct, and City information. Clicking on a VTD opens a popup with a dropdown menu, allowing the user to view detailed demographic data, primary vote counts for specific candidates (Ellzey, Buford, Payne, Wiley) over time, and voter registration and turnout data over time (excluding Spanish Surname data). The map includes a layer control for managing layer visibility.


In [ ]:
import folium
from folium.features import GeoJsonTooltip

# Ensure counties_gdf_4326 is available and not empty
if 'counties_gdf_4326' in locals() and counties_gdf_4326 is not None and not counties_gdf_4326.empty:
    print("✅ counties_gdf_4326 is available for adding to the map.")

    # Add the County boundaries layer to the map
    folium.GeoJson(
        counties_gdf_4326,
        name="All County Boundaries", # Layer name for control
        style_function=lambda x: {
            'color': 'black',  # Outline color
            'weight': 1.5,     # Line thickness
            'fillOpacity': 0,  # No fill
        },
        tooltip=GeoJsonTooltip(fields=['CNTY_NM'], aliases=['County:']) # Tooltip on hover
    ).add_to(m) # Add to the map object 'm' created in the previous step
    print("✅ Added All County Boundaries layer to the map.")

else:
    print("❌ counties_gdf_4326 GeoDataFrame not available or empty. Cannot add County Boundaries layer.")

# Display the map with the added layer
print("\n--- Map with County Boundaries Layer ---")
display(m)

In [ ]:
m.save('folium_map_1.html')

In [ ]:
# --- FINAL MAP GDF (single source of truth) ---
folium_crs = "EPSG:4326"

vtds_cd6_map = master_vtd_gdf.to_crs(folium_crs).copy()
print("✅ Created vtds_cd6_map in EPSG:4326")


In [ ]:
# --- TOOLTIP ---
vtds_cd6_map["tooltip_text"] = (
    "County: " + vtds_cd6_map["County"].fillna("N/A").astype(str)
    + "<br>Precinct: " + vtds_cd6_map["Precinct"].fillna("N/A").astype(str)
)

# --- POPUP ---
vtds_cd6_map["full_popup_html"] = (
    "<b>VTD:</b> " + vtds_cd6_map["CNTYVTD"].astype(str) + "<br>"
    "<b>County:</b> " + vtds_cd6_map["County"].fillna("N/A").astype(str) + "<br>"
    "<b>Registered Voters 2024:</b> " +
    vtds_cd6_map["Voter Registration 2024"].fillna(0).astype(int).astype(str)
)

print("✅ tooltip_text & full_popup_html created")


In [ ]:
geo_keys = vtds_cd6_map.__geo_interface__["features"][0]["properties"].keys()
assert "tooltip_text" in geo_keys
assert "full_popup_html" in geo_keys

print("✅ Confirmed fields inside GeoJSON")


In [ ]:
import folium
from folium.features import GeoJsonTooltip, GeoJsonPopup

m = folium.Map(location=[31.5, -97.0], zoom_start=7, tiles="cartodbpositron")

folium.GeoJson(
    vtds_cd6_map,
    name="CD6 VTDs",
    style_function=lambda f: {
        "fillColor": f["properties"].get("Map_Color", "gray"),
        "color": "darkgreen",
        "weight": 0.8,
        "fillOpacity": 0.6,
    },
    tooltip=GeoJsonTooltip(
        fields=["tooltip_text"],
        sticky=True
    ),
    popup=GeoJsonPopup(
        fields=["full_popup_html"],
        labels=False
    ),
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

display(m)


In [ ]:
import folium
from folium.features import GeoJsonTooltip

# Ensure cd6_boundary GeoDataFrame in EPSG:4326 is available and not empty
if 'cd6_boundary' in locals() and cd6_boundary is not None and not cd6_boundary.empty:
    print("✅ cd6_boundary is available for adding to the map.")

    # Add the CD6 boundary layer to the map
    folium.GeoJson(
        cd6_boundary,
        name="CD6 Boundary", # Layer name for control
        style_function=lambda x: {
            'color': 'blue',  # Outline color
            'weight': 3,     # Line thickness
            'fillOpacity': 0.1, # Light fill
        },
        tooltip=GeoJsonTooltip(fields=['District'], aliases=['District:']) # Tooltip on hover
    ).add_to(m) # Add to the map object 'm'

    print("✅ Added CD6 Boundary layer to the map.")

else:
    print("❌ cd6_boundary GeoDataFrame not available or empty. Cannot add CD6 Boundary layer.")

# Display the map with the added layer
print("\n--- Map with CD6 Boundary Layer ---")
display(m)

In [ ]:
m.save('folium_map_2.html')

In [ ]:
import folium
from folium.features import GeoJsonTooltip, GeoJsonPopup

# Ensure master_vtd_gdf is available, not empty, and in EPSG:4326
if 'master_vtd_gdf' in locals() and master_vtd_gdf is not None and not master_vtd_gdf.empty:
    print("✅ master_vtd_gdf is available for adding to the map.")

    # Reproject to EPSG:4326 for Folium if necessary
    folium_crs = "EPSG:4326"
    if master_vtd_gdf.crs != folium_crs:
         print(f"🔄 Reprojecting master_vtd_gdf from {master_vtd_gdf.crs} to {folium_crs}.")
         vtds_cd6_map = master_vtd_gdf.to_crs(folium_crs).copy()
    else:
         vtds_cd6_map = master_vtd_gdf.copy()
         print(f"✅ master_vtd_gdf is already in {folium_crs}.")


    # Add the VTD layer to the map
    folium.GeoJson(
        vtds_cd6_map,
        name='CD6 VTDs with Data', # Layer name for control
        style_function=lambda feature: {
            # Use the 'Map_Color' column determined in previous steps for fill color
            # Assuming 'Map_Color' was created (e.g., based on Ellzey 2024 primary winner)
            'fillColor': feature['properties'].get('Map_Color', 'gray'), # Default to gray if color is missing
            'color': 'darkgreen', # Outline color
            'weight': 0.8,        # Line thickness
            'fillOpacity': 0.6,   # Fill opacity
        },
        # Add tooltip on hover using the prepared 'tooltip_text'
        tooltip=GeoJsonTooltip(fields=['tooltip_text'], aliases=['Info:'], localize=True, sticky=True),
        # Add popup on click using the prepared 'full_popup_html'
        popup=GeoJsonPopup(fields=['full_popup_html'], labels=False, localize=True, max_width=400), # Increased max_width for popup
    ).add_to(m) # Add to the map object 'm'

    print("✅ Added CD6 VTDs with Data layer (Tooltips & Popups configured).")

else:
    print("❌ master_vtd_gdf GeoDataFrame not available or empty. Cannot add CD6 VTDs layer.")

# Add Layer Control to show/hide layers
folium.LayerControl(collapsed=False).add_to(m)
print("✅ Added Layer Control to the map.")

# Display the final map
print("\n--- Final Interactive Map ---")
display(m)


## Summary:

## Summary of Solving Process: Creating an Interactive CD6 Map

The goal was to create an interactive Folium map of CD6, displaying VTD and county boundaries, with hover tooltips and click popups providing detailed information.

**Key Findings from Code Execution:**

1. **Data Loading and Reprojection:** Geographic data for VTDs (`vtds_cd6.geojson`), county boundaries (`County_Boundaries.shp`), precinct boundaries (`Precincts24G.shp`), and city boundaries (`City.shp`) were successfully loaded and reprojected to EPSG:3083 for initial processing.
2. **Attribute Data Preparation and Merging:** Attribute data from `cleaned_torv_data.csv`, `DemographicsFinal.csv`, and `final2022-2024PrimaryVotes.csv` were loaded. The 'CNTYVTD' or equivalent columns were cleaned and standardized across all datasets. This attribute data was then successfully merged onto the VTD GeoDataFrame (`vtds_cd6_gdf_proj`) using 'CNTYVTD' as the key, creating the `master_vtd_gdf`.
3. **Tooltip and Popup Data Preparation:** Columns required for the hover tooltip (County, Precinct, Cities) were selected, formatted, and combined into a `tooltip_text` column. Detailed data for the click popup (Demographics, Primary Votes, VR/Turnout) was extracted, formatted into HTML tables, and stored in separate HTML columns (`demographics_html`, `primary_votes_html`, `vr_turnout_html`).
4. **Interactive Popup HTML Generation:** A Python function was created to combine the basic VTD information, a dropdown menu, and the generated HTML snippets into a single `full_popup_html` column for each VTD. This HTML includes JavaScript to control the visibility of the different data sections based on the dropdown selection within the popup.
5. **Folium Map Creation:** A Folium map was initialized and centered on the CD6 area by calculating the centroid of the district boundary.
6. **Layer Addition:** The county boundaries (reprojected to EPSG:4326), the CD6 boundary, and the VTD layer (`master_vtd_gdf`, reprojected to EPSG:4326) were added to the Folium map.
7. **Tooltip and Popup Integration:** The `tooltip_text` column was successfully integrated into the VTD layer as a hover tooltip. The `full_popup_html` column was successfully integrated as the content for the click popup, enabling the interactive dropdown functionality within the map.
8. **Layer Control:** A Layer Control was added to the map, allowing users to toggle the visibility of the different geographic layers.

**Final Outcome:**

The task was successfully completed. An interactive Folium map of CD6 has been created. The map displays the boundaries of VTDs and counties. Hovering over a VTD displays a tooltip with County, Precinct, and City information. Clicking on a VTD opens a popup with a dropdown menu, allowing the user to view detailed demographic data, primary vote counts for specific candidates (Ellzey, Buford, Payne, Wiley) over time, and voter registration and turnout data over time (excluding Spanish Surname data). The map includes a layer control for managing layer visibility.

In [ ]:
import folium
from folium.features import GeoJsonTooltip, GeoJsonPopup
from google.colab import files

# -------------------------------
# Validate master GeoDataFrame
# -------------------------------
if 'master_vtd_gdf' not in locals() or master_vtd_gdf is None or master_vtd_gdf.empty:
    raise ValueError("❌ master_vtd_gdf is missing or empty")

print("✅ master_vtd_gdf is available")

# -------------------------------
# Reproject to EPSG:4326 for Folium
# -------------------------------
folium_crs = "EPSG:4326"

if master_vtd_gdf.crs != folium_crs:
    print(f"🔄 Reprojecting from {master_vtd_gdf.crs} to {folium_crs}")
    vtds_cd6_map = master_vtd_gdf.to_crs(folium_crs).copy()
else:
    vtds_cd6_map = master_vtd_gdf.copy()
    print("✅ Already in EPSG:4326")

# -------------------------------
# HARD GUARANTEE popup / tooltip columns
# -------------------------------
required_cols = ['tooltip_text', 'full_popup_html']

for col in required_cols:
    if col not in vtds_cd6_map.columns:
        print(f"⚠️ Creating missing column: {col}")
        vtds_cd6_map[col] = ""
    else:
        vtds_cd6_map[col] = vtds_cd6_map[col].fillna("").astype(str)

print("✅ Tooltip & popup columns verified")

# -------------------------------
# Add VTD layer (ONCE)
# -------------------------------
folium.GeoJson(
    vtds_cd6_map,
    name='CD6 VTDs with Data',
    style_function=lambda feature: {
        'fillColor': feature['properties'].get('Map_Color', 'gray'),
        'color': 'darkgreen',
        'weight': 0.8,
        'fillOpacity': 0.6,
    },
    tooltip=GeoJsonTooltip(
        fields=['tooltip_text'],
        aliases=['Info:'],
        sticky=True
    ),
    popup=GeoJsonPopup(
        fields=['full_popup_html'],
        labels=False,
        max_width=400
    )
).add_to(m)

print("✅ Added CD6 VTD layer with tooltips & popups")

# -------------------------------
# Layer control
# -------------------------------
folium.LayerControl(collapsed=False).add_to(m)
print("✅ Layer control added")

# -------------------------------
# Display map
# -------------------------------
display(m)

# -------------------------------
# Save + download HTML
# -------------------------------
output_path = "/content/folium_map_cd6.html"
m.save(output_path)
files.download(output_path)

print(f"✅ Map saved and downloaded: {output_path}")


In [ ]:
import folium
import json
from folium.features import GeoJsonTooltip, GeoJsonPopup
from google.colab import files

# --------------------------------------------------
# 1. Validate data
# --------------------------------------------------
assert master_vtd_gdf is not None and not master_vtd_gdf.empty
print("✅ master_vtd_gdf is available")

# --------------------------------------------------
# 2. Reproject to EPSG:4326
# --------------------------------------------------
if master_vtd_gdf.crs != "EPSG:4326":
    vtds = master_vtd_gdf.to_crs("EPSG:4326").copy()
else:
    vtds = master_vtd_gdf.copy()

print("🔄 CRS ready for Folium")

# --------------------------------------------------
# 3. BUILD TOOLTIP TEXT (must not be empty)
# --------------------------------------------------
vtds["tooltip_text"] = (
    "County: " + vtds["County Name (from TORV)"].astype(str) +
    "<br>Precinct: " + vtds["Best Matching Precinct Name (Max Overlap)"].astype(str) +
    "<br>City: " + vtds["Cities (from TORV)"].astype(str)
)

# --------------------------------------------------
# 4. BUILD POPUP HTML (simple but real)
# --------------------------------------------------
vtds["full_popup_html"] = (
    "<b>VTD:</b> " + vtds["CNTYVTD"].astype(str) +
    "<br><b>Total Population (2024):</b> " + vtds["Total Population (2024)"].astype(str) +
    "<br><b>Black + Hispanic VAP (2024):</b> " + vtds["Black + Hispanic VAP (2024)"].astype(str)
)

print("✅ Tooltip & popup payloads constructed")

# --------------------------------------------------
# 5. FORCE SERIALIZE TO GEOJSON (THIS IS THE FIX)
# --------------------------------------------------
geojson_data = json.loads(vtds.to_json())

print("✅ GeoJSON serialization complete")

# --------------------------------------------------
# 6. INITIALIZE MAP
# --------------------------------------------------
m = folium.Map(
    location=[
        vtds.geometry.centroid.y.mean(),
        vtds.geometry.centroid.x.mean()
    ],
    zoom_start=8,
    tiles="cartodbpositron"
)

# --------------------------------------------------
# 7. ADD VTD LAYER
# --------------------------------------------------
folium.GeoJson(
    geojson_data,
    name="CD6 VTDs",
    style_function=lambda f: {
        "fillColor": f["properties"].get("Map_Color", "gray"),
        "color": "darkgreen",
        "weight": 0.8,
        "fillOpacity": 0.6,
    },
    tooltip=GeoJsonTooltip(
        fields=["tooltip_text"],
        aliases=["Info:"],
        sticky=True
    ),
    popup=GeoJsonPopup(
        fields=["full_popup_html"],
        labels=False,
        max_width=400
    )
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

# --------------------------------------------------
# 8. DISPLAY
# --------------------------------------------------
display(m)

# --------------------------------------------------
# 9. SAVE + DOWNLOAD
# --------------------------------------------------
output_path = "/content/ellzey_report_cd6.html"
m.save(output_path)
files.download(output_path)

print(f"✅ Map successfully saved and downloaded: {output_path}")


In [ ]:
# 🔍 Inspect first GeoJSON feature exactly how Folium sees it
geojson_preview = vtds_cd6_map.__geo_interface__
print(geojson_preview["features"][0]["properties"].keys())


In [ ]:
# 🔒 Force tooltip & popup fields into GeoJSON-safe strings
vtds_cd6_map = vtds_cd6_map.copy()

vtds_cd6_map["tooltip_text"] = vtds_cd6_map["tooltip_text"].astype(str)
vtds_cd6_map["full_popup_html"] = vtds_cd6_map["full_popup_html"].astype(str)

# 🔍 Hard assertion BEFORE Folium
required_fields = ["tooltip_text", "full_popup_html"]
missing = [c for c in required_fields if c not in vtds_cd6_map.columns]
assert not missing, f"Missing required fields: {missing}"

print("✅ tooltip_text and full_popup_html confirmed in GeoDataFrame")


In [ ]:
import folium
from folium.features import GeoJsonTooltip, GeoJsonPopup

# Ensure master_vtd_gdf is available, not empty, and in EPSG:4326
if 'master_vtd_gdf' in locals() and master_vtd_gdf is not None and not master_vtd_gdf.empty:
    print("✅ master_vtd_gdf is available for adding to the map.")

    # Reproject to EPSG:4326 for Folium if necessary
    folium_crs = "EPSG:4326"
    if master_vtd_gdf.crs != folium_crs:
         print(f"🔄 Reprojecting master_vtd_gdf from {master_vtd_gdf.crs} to {folium_crs}.")
         vtds_cd6_map = master_vtd_gdf.to_crs(folium_crs).copy()
    else:
         vtds_cd6_map = master_vtd_gdf.copy()
         print(f"✅ master_vtd_gdf is already in {folium_crs}.")

# Sanity check
print(vtds_cd6_map.columns)

missing = [c for c in ['tooltip_text', 'full_popup_html'] if c not in vtds_cd6_map.columns]
print("Missing:", missing)

# Ensure they exist and are strings
for col in ['tooltip_text', 'full_popup_html']:
    if col in vtds_cd6_map.columns:
        vtds_cd6_map[col] = vtds_cd6_map[col].fillna("").astype(str)


folium.GeoJson(
    vtds_cd6_map,
    name="CD6 VTDs",
    style_function=lambda feature: {
        "fillColor": feature["properties"].get("Map_Color", "gray"),
        "color": "darkgreen",
        "weight": 0.8,
        "fillOpacity": 0.6,
    },
    tooltip=GeoJsonTooltip(
        fields=["tooltip_text"],
        aliases=["Info:"],
        sticky=True
    ),
    popup=GeoJsonPopup(
        fields=["full_popup_html"],
        labels=False
    ),
).add_to(m)




# Add Layer Control to show/hide layers
folium.LayerControl(collapsed=False).add_to(m)
print("✅ Added Layer Control to the map.")

# Display the final map
print("\n--- Final Interactive Map ---")
display(m)

from google.colab import files

output_path = "/content/folium_map_cd6.html"
m.save(output_path)
files.download(output_path)


In [ ]:
m.save('folium_map_3.html')

## Summary:

## Summary of Solving Process: Creating an Interactive CD6 Map

The goal was to create an interactive Folium map of CD6, displaying VTD and county boundaries, with hover tooltips and click popups providing detailed information.

**Key Findings from Code Execution:**

1. **Data Loading and Reprojection:** Geographic data for VTDs (`vtds_cd6.geojson`), county boundaries (`County_Boundaries.shp`), precinct boundaries (`Precincts24G.shp`), and city boundaries (`City.shp`) were successfully loaded and reprojected to EPSG:3083 for initial processing.
2. **Attribute Data Preparation and Merging:** Attribute data from `cleaned_torv_data.csv`, `DemographicsFinal.csv`, and `final2022-2024PrimaryVotes.csv` were loaded. The 'CNTYVTD' or equivalent columns were cleaned and standardized across all datasets. This attribute data was then successfully merged onto the VTD GeoDataFrame (`vtds_cd6_gdf_proj`) using 'CNTYVTD' as the key, creating the `master_vtd_gdf`.
3. **Tooltip and Popup Data Preparation:** Columns required for the hover tooltip (County, Precinct, Cities) were selected, formatted, and combined into a `tooltip_text` column. Detailed data for the click popup (Demographics, Primary Votes, VR/Turnout) was extracted, formatted into HTML tables, and stored in separate HTML columns (`demographics_html`, `primary_votes_html`, `vr_turnout_html`).
4. **Interactive Popup HTML Generation:** A Python function was created to combine the basic VTD information, a dropdown menu, and the generated HTML snippets into a single `full_popup_html` column for each VTD. This HTML includes JavaScript to control the visibility of the different data sections based on the dropdown selection within the popup.
5. **Folium Map Creation:** A Folium map was initialized and centered on the CD6 area by calculating the centroid of the district boundary.
6. **Layer Addition:** The county boundaries (reprojected to EPSG:4326), the CD6 boundary, and the VTD layer (`master_vtd_gdf`, reprojected to EPSG:4326) were added to the Folium map.
7. **Tooltip and Popup Integration:** The `tooltip_text` column was successfully integrated into the VTD layer as a hover tooltip. The `full_popup_html` column was successfully integrated as the content for the click popup, enabling the interactive dropdown functionality within the map.
8. **Layer Control:** A Layer Control was added to the map, allowing users to toggle the visibility of the different geographic layers.

**Final Outcome:**

The task was successfully completed. An interactive Folium map of CD6 has been created. The map displays the boundaries of VTDs and counties. Hovering over a VTD displays a tooltip with County, Precinct, and City information. Clicking on a VTD opens a popup with a dropdown menu, allowing the user to view detailed demographic data, primary vote counts for specific candidates (Ellzey, Buford, Payne, Wiley) over time, and voter registration and turnout data over time (excluding Spanish Surname data). The map includes a layer control for managing layer visibility.

In [ ]:
import folium
from folium.features import GeoJsonTooltip, GeoJsonPopup

# Ensure master_vtd_gdf is available, not empty, and in EPSG:4326
if 'master_vtd_gdf' in locals() and master_vtd_gdf is not None and not master_vtd_gdf.empty:
    print("✅ master_vtd_gdf is available for adding to the map.")

    # Reproject to EPSG:4326 for Folium if necessary
    folium_crs = "EPSG:4326"
    if master_vtd_gdf.crs != folium_crs:
         print(f"🔄 Reprojecting master_vtd_gdf from {master_vtd_gdf.crs} to {folium_crs}.")
         vtds_cd6_map = master_vtd_gdf.to_crs(folium_crs).copy()
    else:
         vtds_cd6_map = master_vtd_gdf.copy()
         print(f"✅ master_vtd_gdf is already in {folium_crs}.")


    # Add the VTD layer to the map
    folium.GeoJson(
        vtds_cd6_map,
        name='CD6 VTDs with Data', # Layer name for control
        style_function=lambda feature: {
            # Use the 'Map_Color' column determined in previous steps for fill color
            # Assuming 'Map_Color' was created (e.g., based on Ellzey 2024 primary winner)
            'fillColor': feature['properties'].get('Map_Color', 'gray'), # Default to gray if color is missing
            'color': 'darkgreen', # Outline color
            'weight': 0.8,        # Line thickness
            'fillOpacity': 0.6,   # Fill opacity
        },
        # Add tooltip on hover using the prepared 'tooltip_text'
        tooltip=GeoJsonTooltip(fields=['tooltip_text'], aliases=['Info:'], localize=True, sticky=True),
        # Add popup on click using the prepared 'full_popup_html'
        popup=GeoJsonPopup(fields=['full_popup_html'], labels=False, localize=True, max_width=400), # Increased max_width for popup
    ).add_to(m) # Add to the map object 'm'

    print("✅ Added CD6 VTDs with Data layer (Tooltips & Popups configured).")

else:
    print("❌ master_vtd_gdf GeoDataFrame not available or empty. Cannot add CD6 VTDs layer.")

# Add Layer Control to show/hide layers
folium.LayerControl(collapsed=False).add_to(m)
print("✅ Added Layer Control to the map.")

# Display the final map
print("\n--- Final Interactive Map ---")
display(m)

In [ ]:
m.save('foliom_map_4.html')

In [ ]:
import folium
from folium.features import GeoJsonTooltip, GeoJsonPopup
from google.colab import files

# ----------------------------------
# HARD GUARANTEE: data exists
# ----------------------------------
assert 'master_vtd_gdf' in locals()
assert master_vtd_gdf is not None
assert not master_vtd_gdf.empty

print("✅ master_vtd_gdf is available")

# ----------------------------------
# Reproject ONLY if needed
# ----------------------------------
if master_vtd_gdf.crs != "EPSG:4326":
    print(f"🔄 Reprojecting from {master_vtd_gdf.crs} to EPSG:4326")
    vtds_cd6_map = master_vtd_gdf.to_crs("EPSG:4326").copy()
else:
    vtds_cd6_map = master_vtd_gdf.copy()

# ----------------------------------
# CRITICAL FIX: FORCE REQUIRED COLUMNS
# ----------------------------------
# Tooltip
if 'tooltip_text' not in vtds_cd6_map.columns:
    print("⚠️ Creating tooltip_text")
    vtds_cd6_map['tooltip_text'] = (
        "County: " + vtds_cd6_map['County Name (from TORV)'].astype(str) +
        "<br>City: " + vtds_cd6_map['Cities (from TORV)'].astype(str)
    )

# Popup
if 'full_popup_html' not in vtds_cd6_map.columns:
    print("⚠️ Creating full_popup_html")
    vtds_cd6_map['full_popup_html'] = (
        "<b>VTD:</b> " + vtds_cd6_map['CNTYVTD'].astype(str)
    )

# FINAL SANITY CHECK — THIS IS WHAT FOLIUM SEES
print("✅ Columns at render time:")
print(vtds_cd6_map[['tooltip_text', 'full_popup_html']].head(1))

# ----------------------------------
# Map (assumes m already exists)
# ----------------------------------
folium.GeoJson(
    vtds_cd6_map,
    name='CD6 VTDs with Data',
    style_function=lambda feature: {
        'fillColor': feature['properties'].get('Map_Color', 'gray'),
        'color': 'darkgreen',
        'weight': 0.8,
        'fillOpacity': 0.6,
    },
    tooltip=GeoJsonTooltip(
        fields=['tooltip_text'],
        aliases=['Info:'],
        sticky=True
    ),
    popup=GeoJsonPopup(
        fields=['full_popup_html'],
        labels=False,
        max_width=400
    ),
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

print("✅ Layer added")

# ----------------------------------
# Display & Save
# ----------------------------------
display(m)

output_path = "/content/finalellzey_cd6_map.html"
m.save(output_path)
files.download(output_path)

print(f"✅ Saved and downloaded: {output_path}")


## Summary:

## Summary of Solving Process: Creating an Interactive CD6 Map

The goal was to create an interactive Folium map of CD6, displaying VTD and county boundaries, with hover tooltips and click popups providing detailed information.

**Key Findings from Code Execution:**

1. **Data Loading and Reprojection:** Geographic data for VTDs (`vtds_cd6.geojson`), county boundaries (`County_Boundaries.shp`), precinct boundaries (`Precincts24G.shp`), and city boundaries (`City.shp`) were successfully loaded and reprojected to EPSG:3083 for initial processing.
2. **Attribute Data Preparation and Merging:** Attribute data from `cleaned_torv_data.csv`, `DemographicsFinal.csv`, and `final2022-2024PrimaryVotes.csv` were loaded. The 'CNTYVTD' or equivalent columns were cleaned and standardized across all datasets. This attribute data was then successfully merged onto the VTD GeoDataFrame (`vtds_cd6_gdf_proj`) using 'CNTYVTD' as the key, creating the `master_vtd_gdf`.
3. **Tooltip and Popup Data Preparation:** Columns required for the hover tooltip (County, Precinct, Cities) were selected, formatted, and combined into a `tooltip_text` column. Detailed data for the click popup (Demographics, Primary Votes, VR/Turnout) was extracted, formatted into HTML tables, and stored in separate HTML columns (`demographics_html`, `primary_votes_html`, `vr_turnout_html`).
4. **Interactive Popup HTML Generation:** A Python function was created to combine the basic VTD information, a dropdown menu, and the generated HTML snippets into a single `full_popup_html` column for each VTD. This HTML includes JavaScript to control the visibility of the different data sections based on the dropdown selection within the popup.
5. **Folium Map Creation:** A Folium map was initialized and centered on the CD6 area by calculating the centroid of the district boundary.
6. **Layer Addition:** The county boundaries (reprojected to EPSG:4326), the CD6 boundary, and the VTD layer (`master_vtd_gdf`, reprojected to EPSG:4326) were added to the Folium map.
7. **Tooltip and Popup Integration:** The `tooltip_text` column was successfully integrated into the VTD layer as a hover tooltip. The `full_popup_html` column was successfully integrated as the content for the click popup, enabling the interactive dropdown functionality within the map.
8. **Layer Control:** A Layer Control was added to the map, allowing users to toggle the visibility of the different geographic layers.

**Final Outcome:**

The task was successfully completed. An interactive Folium map of CD6 has been created. The map displays the boundaries of VTDs and counties. Hovering over a VTD displays a tooltip with County, Precinct, and City information. Clicking on a VTD opens a popup with a dropdown menu, allowing the user to view detailed demographic data, primary vote counts for specific candidates (Ellzey, Buford, Payne, Wiley) over time, and voter registration and turnout data over time (excluding Spanish Surname data). The map includes a layer control for managing layer visibility.

In [ ]:
import pandas as pd
import os
from IPython.display import display

print("--- Loading Heads of Relevant Files ---")

# File containing Precinct, County, and Intersecting Cities
demographics_file_path = "DemographicsFinal.csv"
if os.path.exists(demographics_file_path):
    try:
        demographics_df = pd.read_csv(demographics_file_path)
        print(f"\n📋 Head of {demographics_file_path}:")
        display(demographics_df.head())
        print("\nColumns:")
        print(demographics_df.columns.tolist())
    except Exception as e:
        print(f"❌ Error loading {demographics_file_path}: {e}")
else:
    print(f"\n❌ {demographics_file_path} not found.")

# File containing TORV data
torv_file_path = "2022-2024TORV.csv"
if os.path.exists(torv_file_path):
    try:
        torv_df = pd.read_csv(torv_file_path)
        print(f"\n📋 Head of {torv_file_path}:")
        display(torv_df.head())
        print("\nColumns:")
        print(torv_df.columns.tolist())
    except Exception as e:
        print(f"❌ Error loading {torv_file_path}: {e}")
else:
    print(f"\n❌ {torv_file_path} not found.")

# File containing Primary Votes data
primary_votes_file_path = "final2022-2024PrimaryVotes.csv"
if os.path.exists(primary_votes_file_path):
    try:
        primary_votes_df = pd.read_csv(primary_votes_file_path)
        print(f"\n📋 Head of {primary_votes_file_path}:")
        display(primary_votes_df.head())
        print("\nColumns:")
        print(primary_votes_df.columns.tolist())
    except Exception as e:
        print(f"❌ Error loading {primary_votes_file_path}: {e}")
else:
    print(f"\n❌ {primary_votes_file_path} not found.")

print("\n--- Finished Loading Heads ---")

## Load and prepare attribute data

### Subtask:
Load `cleaned_torv_data.csv`, `DemographicsFinal.csv`, and `final2022-2024PrimaryVotes.csv`. Clean and standardize the 'CNTYVTD' column in each DataFrame to ensure consistency for merging. Select the necessary columns from each DataFrame for the tooltip and popup.

**Reasoning**:
The subtask requires loading and cleaning three dataframes and selecting specific columns from them. I will write a single code block to perform all these steps.

In [ ]:
# 1. Load cleaned_torv_data.csv
cleaned_torv_file_path = 'cleaned_torv_data.csv'
cleaned_torv_df = None
if os.path.exists(cleaned_torv_file_path):
    try:
        cleaned_torv_df = pd.read_csv(cleaned_torv_file_path)
        print(f"✅ Loaded cleaned_torv_data.csv from {cleaned_torv_file_path}")
    except Exception as e:
        print(f"❌ Error loading {cleaned_torv_file_path}: {e}")
else:
    print(f"❌ {cleaned_torv_file_path} not found.")

# 2. Clean and standardize 'CNTYVTD' in cleaned_torv_df
if cleaned_torv_df is not None and 'CNTYVTD' in cleaned_torv_df.columns:
    cleaned_torv_df['CNTYVTD'] = cleaned_torv_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
    print("✅ Cleaned 'CNTYVTD' in cleaned_torv_df.")
else:
    print("⚠️ 'CNTYVTD' column not found or cleaned_torv_df not loaded. Skipping CNTYVTD cleaning for cleaned_torv_df.")


# 3. Load DemographicsFinal.csv
demographics_file_path = "DemographicsFinal.csv"
demographics_df = None
if os.path.exists(demographics_file_path):
    try:
        demographics_df = pd.read_csv(demographics_file_path)
        print(f"\n✅ Loaded DemographicsFinal.csv from {demographics_file_path}")
    except Exception as e:
        print(f"❌ Error loading {demographics_file_path}: {e}")
else:
    print(f"\n❌ {demographics_file_path} not found.")

# 4. Clean 'VTD Identifier', rename to 'CNTYVTD', and select columns for demographics_geo_df
demographics_geo_df = None
if demographics_df is not None and 'VTD Identifier' in demographics_df.columns:
    try:
        demographics_geo_df = demographics_df[['VTD Identifier', 'Best Matching Precinct Name (Max Overlap)', 'County Name (from TORV)', 'Cities (from TORV)']].copy()
        demographics_geo_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
        demographics_geo_df['CNTYVTD'] = demographics_geo_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')

        # Handle potential duplicates in demographics_geo_df before merging
        if not demographics_geo_df['CNTYVTD'].is_unique:
             print("⚠️ Duplicate CNTYVTDs found in demographics geo info. Aggregating (taking first).")
             agg_funcs_geo = {col: 'first' for col in demographics_geo_df.columns if col != 'CNTYVTD'}
             demographics_geo_df = demographics_geo_df.groupby('CNTYVTD', as_index=False).agg(agg_funcs_geo)


        print("✅ Cleaned 'VTD Identifier', renamed to 'CNTYVTD', and selected columns for demographics_geo_df.")
    except KeyError as e:
        print(f"❌ Missing expected column in DemographicsFinal.csv for selection: {e}")
        demographics_geo_df = None
    except Exception as e:
        print(f"❌ Error processing DemographicsFinal.csv for demographics_geo_df: {e}")
        demographics_geo_df = None
else:
    print("⚠️ 'VTD Identifier' column not found or DemographicsFinal.csv not loaded. Cannot create demographics_geo_df.")


# 5. Load final2022-2024PrimaryVotes.csv
primary_votes_file_path = "final2022-2024PrimaryVotes.csv"
primary_votes_df = None
if os.path.exists(primary_votes_file_path):
    try:
        primary_votes_df = pd.read_csv(primary_votes_file_path)
        print(f"\n✅ Loaded final2022-2024PrimaryVotes.csv from {primary_votes_file_path}")
    except Exception as e:
        print(f"❌ Error loading {primary_votes_file_path}: {e}")
else:
    print(f"\n❌ {primary_votes_file_path} not found.")

# 6 & 7. Identify relevant vote columns, clean keys, consolidate, and ensure numeric for primary_votes_cleaned_df
primary_votes_cleaned_df = None
if primary_votes_df is not None:
    vote_cols_2022_raw = ['CNTYVTD', 'BufordR_22P_U.S. Rep 6', 'EllzeyR_22P_U.S. Rep 6', 'PayneR_22P_U.S. Rep 6']
    vote_cols_2024_raw = ['CNTYVTD.1', 'BufordR_24P_U.S. Rep 6', 'EllzeyR_24P_U.S. Rep 6', 'WileyR_24P_U.S. Rep 6']

    votes_2022_df = primary_votes_df.dropna(subset=[col for col in vote_cols_2022_raw if col in primary_votes_df.columns]).copy()
    votes_2024_df = primary_votes_df.dropna(subset=[col for col in vote_cols_2024_raw if col in primary_votes_df.columns]).copy()

    # Select only relevant columns and clean keys
    votes_2022_df = votes_2022_df[[col for col in vote_cols_2022_raw if col in votes_2022_df.columns]].copy()
    votes_2024_df = votes_2024_df[[col for col in vote_cols_2024_raw if col in votes_2024_df.columns]].copy()

    if 'CNTYVTD' in votes_2022_df.columns: votes_2022_df['CNTYVTD'] = votes_2022_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
    if 'CNTYVTD.1' in votes_2024_df.columns:
        votes_2024_df.rename(columns={'CNTYVTD.1': 'CNTYVTD'}, inplace=True)
    if 'CNTYVTD' in votes_2024_df.columns: votes_2024_df['CNTYVTD'] = votes_2024_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')


    # Merge 2022 and 2024 data
    primary_votes_merged = pd.merge(
        votes_2022_df,
        votes_2024_df,
        on='CNTYVTD',
        how='outer'
    )

    # Aggregate potential duplicates
    if not primary_votes_merged.empty and 'CNTYVTD' in primary_votes_merged.columns and primary_votes_merged['CNTYVTD'].duplicated().any():
         print("⚠️ Duplicates found on CNTYVTD after initial primary vote merge. Aggregating (taking first).")
         agg_funcs_votes = {col: 'first' for col in primary_votes_merged.columns if col != 'CNTYVTD'}
         primary_votes_cleaned_df = primary_votes_merged.groupby('CNTYVTD', as_index=False).agg(agg_funcs_votes)
    else:
         primary_votes_cleaned_df = primary_votes_merged.copy()


    # Ensure all vote columns are numeric
    if primary_votes_cleaned_df is not None:
        vote_cols_final = [col for col in primary_votes_cleaned_df.columns if col != 'CNTYVTD']
        for col in vote_cols_final:
            primary_votes_cleaned_df[col] = pd.to_numeric(primary_votes_cleaned_df[col], errors='coerce')
        print("✅ Processed and cleaned primary votes data.")

else:
    print("⚠️ primary_votes_df not loaded. Cannot create primary_votes_cleaned_df.")


# 8. Print heads and column names
print("\n--- Verification of Cleaned DataFrames ---")

print("\n📋 Head of cleaned_torv_df:")
if cleaned_torv_df is not None:
    display(cleaned_torv_df.head())
    print("\nColumns of cleaned_torv_df:")
    print(cleaned_torv_df.columns.tolist())
else:
    print("cleaned_torv_df is not available.")


print("\n📋 Head of demographics_geo_df:")
if demographics_geo_df is not None:
    display(demographics_geo_df.head())
    print("\nColumns of demographics_geo_df:")
    print(demographics_geo_df.columns.tolist())
else:
    print("demographics_geo_df is not available.")


print("\n📋 Head of primary_votes_cleaned_df:")
if primary_votes_cleaned_df is not None:
    display(primary_votes_cleaned_df.head())
    print("\nColumns of primary_votes_cleaned_df:")
    print(primary_votes_cleaned_df.columns.tolist())
else:
    print("primary_votes_cleaned_df is not available.")

## Merge attribute data with geometry

### Subtask:
Merge the prepared attribute dataframes from step 2 (`cleaned_torv_df`, `demographics_geo_df`, and `primary_votes_cleaned_df`) onto the VTD GeoDataFrame from step 1 (`vtds_cd6_gdf_proj`) using the cleaned 'CNTYVTD' identifier. Perform left merges to keep all VTDs from the geometry file, adding attribute data where available.

**Reasoning**:
Merge the attribute dataframes onto the VTD GeoDataFrame using the cleaned 'CNTYVTD' key.

In [ ]:
import pandas as pd

# Ensure vtds_cd6_gdf_proj is available and has the cleaned 'CNTYVTD'
if 'vtds_cd6_gdf_proj' in locals() and vtds_cd6_gdf_proj is not None and not vtds_cd6_gdf_proj.empty:
    if 'CNTYVTD' not in vtds_cd6_gdf_proj.columns:
        # Re-create CNTYVTD in vtds_cd6_gdf_proj if it's missing, based on previous logic
        if 'CNTY_x' in vtds_cd6_gdf_proj.columns and 'VTD_x' in vtds_cd6_gdf_proj.columns:
            vtds_cd6_gdf_proj['CNTYVTD'] = vtds_cd6_gdf_proj['CNTY_x'].astype(str).str.strip() + vtds_cd6_gdf_proj['VTD_x'].astype(str).str.strip()
            print("✅ Created 'CNTYVTD' in vtds_cd6_gdf_proj from CNTY_x and VTD_x.")
        elif 'CNTYVTD_x' in vtds_cd6_gdf_proj.columns:
             vtds_cd6_gdf_proj.rename(columns={'CNTYVTD_x': 'CNTYVTD'}, inplace=True)
             print("✅ Using existing 'CNTYVTD_x' as 'CNTYVTD' in vtds_cd6_gdf_proj.")
        elif 'CNTYVTD_y' in vtds_cd6_gdf_proj.columns: # Check for _y if it was merged from pop data
             vtds_cd6_gdf_proj.rename(columns={'CNTYVTD_y': 'CNTYVTD'}, inplace=True)
             print("✅ Using existing 'CNTYVTD_y' as 'CNTYVTD' in vtds_cd6_gdf_proj.")
        else:
             print("❌ Could not find a suitable VTD identifier column in vtds_cd6_gdf_proj to create CNTYVTD. Cannot merge attributes.")
             master_vtd_gdf = vtds_cd6_gdf_proj.copy() # Proceed with geometry only
             merge_failed = True # Flag merge failure

    if 'CNTYVTD' in vtds_cd6_gdf_proj.columns:
        vtds_cd6_gdf_proj['CNTYVTD'] = vtds_cd6_gdf_proj['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
        print("✅ Cleaned 'CNTYVTD' in vtds_cd6_gdf_proj.")
        master_vtd_gdf = vtds_cd6_gdf_proj.copy() # Start with the cleaned GeoDataFrame
        merge_failed = False
    else:
         master_vtd_gdf = vtds_cd6_gdf_proj.copy() # Proceed with geometry only
         merge_failed = True # Flag merge failure


else:
    print("❌ vtds_cd6_gdf_proj is not available or empty. Cannot merge attributes.")
    # Create an empty GeoDataFrame with the expected columns to avoid errors in subsequent steps
    master_vtd_gdf = gpd.GeoDataFrame({'CNTYVTD': [], 'geometry': []}, crs=target_crs)
    merge_failed = True # Flag merge failure


if not merge_failed:
    # --- Merge Demographics Geo Info ---
    if 'demographics_geo_df' in locals() and demographics_geo_df is not None and not demographics_geo_df.empty:
        print("\n--- Merging Demographics Geo Info ---")
        # Ensure key is clean in demographics_geo_df (already done in previous step, but double-check)
        if 'CNTYVTD' in demographics_geo_df.columns:
             demographics_geo_df['CNTYVTD'] = demographics_geo_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')

             # Drop potential duplicate columns in master_vtd_gdf before merging
             geo_cols_to_drop_before_demo = ['Best Matching Precinct Name (Max Overlap)', 'County Name (from TORV)', 'Cities (from TORV)', 'Precinct', 'County', 'Intersecting Cities'] # Include target names from previous TORV merge
             master_vtd_gdf.drop(columns=[col for col in geo_cols_to_drop_before_demo if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
             print(f"Dropped existing geo columns from master_vtd_gdf before demographics merge.")


             master_vtd_gdf = master_vtd_gdf.merge(
                 demographics_geo_df,
                 on='CNTYVTD',
                 how='left'
             )
             print(f"✅ Merged demographics geo info. Rows after merge: {len(master_vtd_gdf)}")
        else:
             print("❌ 'CNTYVTD' not found in demographics_geo_df. Skipping merge.")
    else:
        print("❌ demographics_geo_df not available or empty. Skipping merge.")


    # --- Merge Cleaned TORV Data ---
    if 'cleaned_torv_df' in locals() and cleaned_torv_df is not None and not cleaned_torv_df.empty:
        print("\n--- Merging Cleaned TORV Data ---")
        # Ensure key is clean in cleaned_torv_df (already done in previous step, but double-check)
        if 'CNTYVTD' in cleaned_torv_df.columns:
             cleaned_torv_df['CNTYVTD'] = cleaned_torv_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')

             # Identify columns to merge from cleaned_torv_df, excluding keys and geo columns already merged from demographics
             torv_cols_to_merge = [col for col in cleaned_torv_df.columns if col not in ['CNTYVTD', 'Precinct', 'County', 'Intersecting Cities']]

             # Drop potential duplicate columns in master_vtd_gdf before merging TORV data
             # Use the column names from the cleaned_torv_df that are being merged
             master_vtd_gdf.drop(columns=[col for col in torv_cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
             print(f"Dropped potential duplicate TORV columns from master_vtd_gdf before TORV merge.")


             master_vtd_gdf = master_vtd_gdf.merge(
                 cleaned_torv_df[['CNTYVTD'] + torv_cols_to_merge],
                 on='CNTYVTD',
                 how='left'
             )
             print(f"✅ Merged cleaned TORV data. Rows after merge: {len(master_vtd_gdf)}")
        else:
             print("❌ 'CNTYVTD' not found in cleaned_torv_df. Skipping merge.")
    else:
        print("❌ cleaned_torv_df not available or empty. Skipping merge.")


    # --- Merge Primary Votes Data ---
    if 'primary_votes_cleaned_df' in locals() and primary_votes_cleaned_df is not None and not primary_votes_cleaned_df.empty:
        print("\n--- Merging Primary Votes Data ---")
        # Ensure key is clean in primary_votes_cleaned_df (already done in previous step, but double-check)
        if 'CNTYVTD' in primary_votes_cleaned_df.columns:
             primary_votes_cleaned_df['CNTYVTD'] = primary_votes_cleaned_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')

             # Drop potential duplicate columns in master_vtd_gdf before merging primary votes
             vote_cols_to_merge = [col for col in primary_votes_cleaned_df.columns if col != 'CNTYVTD']
             master_vtd_gdf.drop(columns=[col for col in vote_cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
             print(f"Dropped potential duplicate vote columns from master_vtd_gdf before vote merge.")


             master_vtd_gdf = master_vtd_gdf.merge(
                 primary_votes_cleaned_df,
                 on='CNTYVTD',
                 how='left'
             )
             print(f"✅ Merged primary votes data. Rows after merge: {len(master_vtd_gdf)}")
        else:
             print("❌ 'CNTYVTD' not found in primary_votes_cleaned_df. Skipping merge.")
    else:
        print("❌ primary_votes_cleaned_df not available or empty. Skipping merge.")


    # --- Final Inspection ---
    print("\n--- Final Merged GeoDataFrame Inspection ---")
    print(f"Merged GeoDataFrame shape: {master_vtd_gdf.shape}")
    print("\nHead of merged master_vtd_gdf:")
    display(master_vtd_gdf.head())
    print("\nColumns of merged master_vtd_gdf:")
    print(master_vtd_gdf.columns.tolist())
else:
    print("\n❌ Merge process skipped due to missing or invalid base GeoDataFrame.")

## Load geographic data

### Subtask:
Load the base CD6 VTD geometry from `vtds_cd6.geojson` and reproject it to a suitable projected CRS (EPSG:3083).

**Reasoning**:
Load the VTD GeoJSON file, define the target CRS, check for file existence, read the file into a GeoDataFrame, check and reproject the CRS if necessary, and print confirmation or error messages.

In [ ]:
import geopandas as gpd
import os

# 1. Define the path to the vtds_cd6.geojson file.
geojson_path = 'vtds_cd6.geojson'

# 2. Define the target projected CRS (EPSG:3083).
target_crs = "EPSG:3083"

# 3. Check if the GeoJSON file exists using os.path.exists().
if os.path.exists(geojson_path):
    try:
        # 4. If the file exists, read it into a GeoDataFrame using gpd.read_file().
        vtds_cd6_gdf_proj = gpd.read_file(geojson_path)
        print(f"✅ Loaded base VTDs from {geojson_path}.")

        # 5. Check the current CRS of the loaded GeoDataFrame. If it's None or different from the target CRS, reproject it to the target CRS (EPSG:3083) using .to_crs().
        if vtds_cd6_gdf_proj.crs is None:
            print(f"⚠️ {geojson_path} CRS missing, assuming EPSG:3857 and setting to {target_crs}")
            vtds_cd6_gdf_proj.set_crs("EPSG:3857", inplace=True) # Assume a common web CRS if missing

        if vtds_cd6_gdf_proj.crs != target_crs:
             vtds_cd6_gdf_proj = vtds_cd6_gdf_proj.to_crs(target_crs)
             print(f"✅ Reprojected vtds_cd6_gdf to {target_crs}.")
        else:
             print(f"✅ vtds_cd6_gdf is already in {target_crs}.")

        # 6. Print a confirmation message indicating successful loading and reprojection (or if reprojection was skipped).
        print(f"📋 Head of reprojected vtds_cd6_gdf:")
        display(vtds_cd6_gdf_proj.head())
        print(f"CRS: {vtds_cd6_gdf_proj.crs}")


    except Exception as e:
        # Handle potential errors during loading or reprojection
        print(f"❌ Error loading or processing {geojson_path}: {e}")
        vtds_cd6_gdf_proj = None # Ensure the variable is set to None if loading fails
else:
    # 7. If the file does not exist, print an error message.
    print(f"❌ {geojson_path} not found. Cannot proceed.")
    vtds_cd6_gdf_proj = None # Ensure the variable is set to None if file not found

## Merge attribute data with geometry

### Subtask:
Merge the prepared attribute dataframes from step 2 (`cleaned_torv_df`, `demographics_geo_df`, and `primary_votes_cleaned_df`) onto the VTD GeoDataFrame from step 1 (`vtds_cd6_gdf_proj`) using the cleaned 'CNTYVTD' identifier. Perform left merges to keep all VTDs from the geometry file, adding attribute data where available.

**Reasoning**:
Merge the attribute dataframes onto the VTD GeoDataFrame using the cleaned 'CNTYVTD' key.

In [ ]:
import pandas as pd

# Ensure vtds_cd6_gdf_proj is available and has the cleaned 'CNTYVTD'
if 'vtds_cd6_gdf_proj' in locals() and vtds_cd6_gdf_proj is not None and not vtds_cd6_gdf_proj.empty:
    if 'CNTYVTD' not in vtds_cd6_gdf_proj.columns:
        # Re-create CNTYVTD in vtds_cd6_gdf_proj if it's missing, based on previous logic
        if 'CNTY_x' in vtds_cd6_gdf_proj.columns and 'VTD_x' in vtds_cd6_gdf_proj.columns:
            vtds_cd6_gdf_proj['CNTYVTD'] = vtds_cd6_gdf_proj['CNTY_x'].astype(str).str.strip() + vtds_cd6_gdf_proj['VTD_x'].astype(str).str.strip()
            print("✅ Created 'CNTYVTD' in vtds_cd6_gdf_proj from CNTY_x and VTD_x.")
        elif 'CNTYVTD_x' in vtds_cd6_gdf_proj.columns:
             vtds_cd6_gdf_proj.rename(columns={'CNTYVTD_x': 'CNTYVTD'}, inplace=True)
             print("✅ Using existing 'CNTYVTD_x' as 'CNTYVTD' in vtds_cd6_gdf_proj.")
        elif 'CNTYVTD_y' in vtds_cd6_gdf_proj.columns: # Check for _y if it was merged from pop data
             vtds_cd6_gdf_proj.rename(columns={'CNTYVTD_y': 'CNTYVTD'}, inplace=True)
             print("✅ Using existing 'CNTYVTD_y' as 'CNTYVTD' in vtds_cd6_gdf_proj.")
        else:
             print("❌ Could not find a suitable VTD identifier column in vtds_cd6_gdf_proj to create CNTYVTD. Cannot merge attributes.")
             master_vtd_gdf = vtds_cd6_gdf_proj.copy() # Proceed with geometry only
             merge_failed = True # Flag merge failure

    if 'CNTYVTD' in vtds_cd6_gdf_proj.columns:
        vtds_cd6_gdf_proj['CNTYVTD'] = vtds_cd6_gdf_proj['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
        print("✅ Cleaned 'CNTYVTD' in vtds_cd6_gdf_proj.")
        master_vtd_gdf = vtds_cd6_gdf_proj.copy() # Start with the cleaned GeoDataFrame
        merge_failed = False
    else:
         master_vtd_gdf = vtds_cd6_gdf_proj.copy() # Proceed with geometry only
         merge_failed = True # Flag merge failure


else:
    print("❌ vtds_cd6_gdf_proj is not available or empty. Cannot merge attributes.")
    # Create an empty GeoDataFrame with the expected columns to avoid errors in subsequent steps
    master_vtd_gdf = gpd.GeoDataFrame({'CNTYVTD': [], 'geometry': []}, crs=target_crs)
    merge_failed = True # Flag merge failure


if not merge_failed:
    # --- Merge Demographics Geo Info ---
    if 'demographics_geo_df' in locals() and demographics_geo_df is not None and not demographics_geo_df.empty:
        print("\n--- Merging Demographics Geo Info ---")
        # Ensure key is clean in demographics_geo_df (already done in previous step, but double-check)
        if 'CNTYVTD' in demographics_geo_df.columns:
             demographics_geo_df['CNTYVTD'] = demographics_geo_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')

             # Drop potential duplicate columns in master_vtd_gdf before merging
             geo_cols_to_drop_before_demo = ['Best Matching Precinct Name (Max Overlap)', 'County Name (from TORV)', 'Cities (from TORV)', 'Precinct', 'County', 'Intersecting Cities'] # Include target names from previous TORV merge
             master_vtd_gdf.drop(columns=[col for col in geo_cols_to_drop_before_demo if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
             print(f"Dropped existing geo columns from master_vtd_gdf before demographics merge.")


             master_vtd_gdf = master_vtd_gdf.merge(
                 demographics_geo_df,
                 on='CNTYVTD',
                 how='left'
             )
             print(f"✅ Merged demographics geo info. Rows after merge: {len(master_vtd_gdf)}")
        else:
             print("❌ 'CNTYVTD' not found in demographics_geo_df. Skipping merge.")
    else:
        print("❌ demographics_geo_df not available or empty. Skipping merge.")


    # --- Merge Cleaned TORV Data ---
    if 'cleaned_torv_df' in locals() and cleaned_torv_df is not None and not cleaned_torv_df.empty:
        print("\n--- Merging Cleaned TORV Data ---")
        # Ensure key is clean in cleaned_torv_df (already done in previous step, but double-check)
        if 'CNTYVTD' in cleaned_torv_df.columns:
             cleaned_torv_df['CNTYVTD'] = cleaned_torv_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')

             # Identify columns to merge from cleaned_torv_df, excluding keys and geo columns already merged from demographics
             torv_cols_to_merge = [col for col in cleaned_torv_df.columns if col not in ['CNTYVTD', 'Precinct', 'County', 'Intersecting Cities']]

             # Drop potential duplicate columns in master_vtd_gdf before merging TORV data
             # Use the column names from the cleaned_torv_df that are being merged
             master_vtd_gdf.drop(columns=[col for col in torv_cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
             print(f"Dropped potential duplicate TORV columns from master_vtd_gdf before TORV merge.")


             master_vtd_gdf = master_vtd_gdf.merge(
                 cleaned_torv_df[['CNTYVTD'] + torv_cols_to_merge],
                 on='CNTYVTD',
                 how='left'
             )
             print(f"✅ Merged cleaned TORV data. Rows after merge: {len(master_vtd_gdf)}")
        else:
             print("❌ 'CNTYVTD' not found in cleaned_torv_df. Skipping merge.")
    else:
        print("❌ cleaned_torv_df not available or empty. Skipping merge.")


    # --- Merge Primary Votes Data ---
    if 'primary_votes_cleaned_df' in locals() and primary_votes_cleaned_df is not None and not primary_votes_cleaned_df.empty:
        print("\n--- Merging Primary Votes Data ---")
        # Ensure key is clean in primary_votes_cleaned_df (already done in previous step, but double-check)
        if 'CNTYVTD' in primary_votes_cleaned_df.columns:
             primary_votes_cleaned_df['CNTYVTD'] = primary_votes_cleaned_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')

             # Drop potential duplicate columns in master_vtd_gdf before merging primary votes
             vote_cols_to_merge = [col for col in primary_votes_cleaned_df.columns if col != 'CNTYVTD']
             master_vtd_gdf.drop(columns=[col for col in vote_cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
             print(f"Dropped potential duplicate vote columns from master_vtd_gdf before vote merge.")


             master_vtd_gdf = master_vtd_gdf.merge(
                 primary_votes_cleaned_df,
                 on='CNTYVTD',
                 how='left'
             )
             print(f"✅ Merged primary votes data. Rows after merge: {len(master_vtd_gdf)}")
        else:
             print("❌ 'CNTYVTD' not found in primary_votes_cleaned_df. Skipping merge.")
    else:
        print("❌ primary_votes_cleaned_df not available or empty. Skipping merge.")


    # --- Final Inspection ---
    print("\n--- Final Merged GeoDataFrame Inspection ---")
    print(f"Merged GeoDataFrame shape: {master_vtd_gdf.shape}")
    print("\nHead of merged master_vtd_gdf:")
    display(master_vtd_gdf.head())
    print("\nColumns of merged master_vtd_gdf:")
    print(master_vtd_gdf.columns.tolist())
else:
    print("\n❌ Merge process skipped due to missing or invalid base GeoDataFrame.")

## Prepare data for tooltip

### Subtask:
Select and format the County, Precinct, and Cities columns for the hover tooltip. Ensure these columns exist and are handled gracefully if missing in some VTDs.

**Reasoning**:
Check if the master_vtd_gdf GeoDataFrame exists and is not empty. If it is not available or empty, print an error message and finish the task. Otherwise, prepare the County, Precinct, and Cities columns for the tooltip as requested in the instructions.

In [ ]:
import pandas as pd

# 1. Check if the master_vtd_gdf GeoDataFrame exists and is not empty.
if 'master_vtd_gdf' not in locals() or master_vtd_gdf is None or master_vtd_gdf.empty:
    print("❌ master_vtd_gdf GeoDataFrame is not available or empty. Cannot prepare tooltip columns.")
    # No further steps can be completed without the master GeoDataFrame.
else:
    print("✅ master_vtd_gdf GeoDataFrame is available for tooltip preparation.")

    # Define a list of column names expected for the tooltip.
    # Use the exact column names resulting from the previous merge steps.
    tooltip_cols_raw = ['County Name (from TORV)', 'Best Matching Precinct Name (Max Overlap)', 'Cities (from TORV)']

    # Map the raw column names to the desired display names for the tooltip
    tooltip_col_map = {
        'County Name (from TORV)': 'County',
        'Best Matching Precinct Name (Max Overlap)': 'Precinct',
        'Cities (from TORV)': 'Cities'
    }

    # Ensure the target columns for the tooltip map exist in the master_vtd_gdf
    # Create placeholder columns if they don't exist, then apply the rename
    for raw_col, display_col in tooltip_col_map.items():
        if raw_col not in master_vtd_gdf.columns:
            print(f"⚠️ Tooltip column '{raw_col}' not found. Creating a placeholder column '{display_col}'.")
            master_vtd_gdf[display_col] = 'N/A' # Create with target display name directly
        else:
            # If the column exists, prepare it (convert to string and fill NaNs)
            master_vtd_gdf[display_col] = master_vtd_gdf[raw_col].astype(str).fillna('N/A')
            # If the display column name is different from the raw, we rename later or just use the display name as the final column name


    # Select the final columns to be used in the tooltip (using display names)
    final_tooltip_cols = list(tooltip_col_map.values())
    # Ensure the final columns are in the GeoDataFrame
    for col in final_tooltip_cols:
        if col not in master_vtd_gdf.columns:
             # This shouldn't happen if the logic above worked, but as a safeguard
             master_vtd_gdf[col] = 'N/A' # Create if somehow missed


    # 6. Create a new column in master_vtd_gdf named 'tooltip_text'.
    master_vtd_gdf['tooltip_text'] = master_vtd_gdf.apply(
        lambda row: f"""
<b>County:</b> {row.get('County', 'N/A')}<br>
<b>Precinct:</b> {row.get('Precinct', 'N/A')}<br>
<b>Cities:</b> {row.get('Cities', 'N/A')}
        """,
        axis=1
    )
    print("✅ Created 'tooltip_text' column for VTD hover tooltips.")

    # 7. Print the head of master_vtd_gdf showing the original and newly created tooltip columns.
    print("\n📋 Head of master_vtd_gdf with tooltip columns:")
    # Display the original raw columns if they exist, and the final display columns and tooltip_text
    display_cols = [col for col in tooltip_cols_raw + final_tooltip_cols + ['tooltip_text'] if col in master_vtd_gdf.columns]
    display(master_vtd_gdf[display_cols].head())

    print("\nColumns of master_vtd_gdf after tooltip preparation:")
    print(master_vtd_gdf.columns.tolist())

**Reasoning**:
Prepare the demographic data for the popup dropdown. This involves selecting the relevant demographic columns and formatting them into an HTML snippet.

In [ ]:
import pandas as pd

# Ensure master_vtd_gdf and demographics_df are available
if 'master_vtd_gdf' not in locals() or master_vtd_gdf is None or master_vtd_gdf.empty:
    print("❌ master_vtd_gdf GeoDataFrame is not available or empty. Cannot prepare demographic data for popup.")
elif 'demographics_df' not in locals() or demographics_df is None or demographics_df.empty:
    print("❌ demographics_df DataFrame is not available or empty. Cannot prepare demographic data for popup.")
else:
    print("✅ master_vtd_gdf and demographics_df are available for demographic popup data preparation.")

    # Define demographic columns from DemographicsFinal.csv for the popup
    demographic_cols_raw = [
        'VTD Identifier',
        'Total Population (2020)', 'Total Population (2022)', 'Total Population (2024)',
        'Anglo Population (2020)', 'Anglo Population (2022)', 'Anglo Population (2024)',
        'Non-Anglo Population (2020)', 'Non-Anglo Population (2022)', 'Non-Anglo Population (2024)',
        'Asian Population (2020)', 'Asian Population (2022)', 'Asian Population (2024)',
        'Black Population (2020)', 'Black Population (2022)', 'Black Population (2024)',
        'Hispanic Population (2020)', 'Hispanic Population (2022)', 'Hispanic Population (2024)',
        'Black + Hispanic Population (2020)', 'Black + Hispanic Population (2022)', 'Black + Hispanic Population (2024)',
        'Voting Age Population (2020)', 'Voting Age Population (2022)', 'Voting Age Population (2024)',
        'Anglo VAP (2020)', 'Anglo VAP (2022)', 'Anglo VAP (2024)',
        'Non-Anglo VAP (2020)', 'Non-Anglo VAP (2022)', 'Non-Anglo VAP (2024)',
        'Asian VAP (2020)', 'Asian VAP (2022)', 'Asian VAP (2024)',
        'Black VAP (2020)', 'Black VAP (2022)', 'Black VAP (2024)',
        'Hispanic VAP (2020)', 'Hispanic VAP (2022)', 'Hispanic VAP (2024)',
        'Black + Hispanic VAP (2020)', 'Black + Hispanic VAP (2022)', 'Black + Hispanic VAP (2024)',
        'Total Pop % Change (2020-2022)', 'Total Pop % Change (2022-2024)',
        'Anglo Pop % Change (2020-2022)', 'Anglo Pop % Change (2022-2024)',
        'Non-Anglo Pop % Change (2020-2022)', 'Non-Anglo Pop % Change (2022-2024)',
        'Asian Pop % Change (2020-2022)', 'Asian Pop % Change (2022-2024)',
        'Black Pop % Change (2020-2022)', 'Black Pop % Change (2022-2024)',
        'Hispanic Pop % Change (2020-2022)', 'Hispanic Pop % Change (2022-2024)',
        'Black + Hispanic Pop % Change (2020-2022)', 'Black + Hispanic Pop % Change (2022-2024)',
        'VAP % Change (2020-2022)', 'VAP % Change (2022-2024)',
        'Anglo VAP % Change (2020-2022)', 'Anglo VAP % Change (2022-2024)',
        'Non-Anglo VAP % Change (2020-2022)', 'Non-Anglo VAP % Change (2022-2024)',
        'Asian VAP % Change (2020-2022)', 'Asian VAP % Change (2022-2024)',
        'Black VAP % Change (2020-2022)', 'Black VAP % Change (2022-2024)',
        'Hispanic VAP % Change (2020-2022)', 'Hispanic VAP % Change (2022-2024)',
        'Black + Hispanic VAP % Change (2020-2022)', 'Black + Hispanic VAP % Change (2022-2024)'
    ]

    # Select the columns that are present in demographics_df
    demographic_cols_present = [col for col in demographic_cols_raw if col in demographics_df.columns]

    if 'VTD Identifier' in demographics_df.columns:
        # Prepare demographics data for merging, cleaning the key
        demographics_popup_df = demographics_df[demographic_cols_present].copy()
        demographics_popup_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
        demographics_popup_df['CNTYVTD'] = demographics_popup_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')

        # Handle potential duplicates
        if not demographics_popup_df['CNTYVTD'].is_unique:
            print("⚠️ Duplicate CNTYVTDs found in demographics data for popup. Aggregating (taking first).")
            agg_funcs_demo_popup = {col: 'first' for col in demographics_popup_df.columns if col != 'CNTYVTD'}
            demographics_popup_df = demographics_popup_df.groupby('CNTYVTD', as_index=False).agg(agg_funcs_demo_popup)

        print(f"✅ Prepared demographic data for popup for {len(demographics_popup_df)} VTDs.")

        # Merge the demographic data into the master GeoDataFrame
        # Identify demographic columns to merge, excluding the key
        demo_cols_to_merge = [col for col in demographics_popup_df.columns if col != 'CNTYVTD']

        # Drop potential existing demographic columns in master_vtd_gdf before merging
        master_vtd_gdf.drop(columns=[col for col in demo_cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
        print("Dropped potential existing demographic columns from master_vtd_gdf.")

        master_vtd_gdf = master_vtd_gdf.merge(
            demographics_popup_df,
            on='CNTYVTD',
            how='left'
        )
        print(f"✅ Merged demographic data for popup. Rows after merge: {len(master_vtd_gdf)}")

        # --- Format Demographic Data for Popup HTML ---
        print("\n--- Formatting Demographic Data for Popup HTML ---")

        # Define a helper function to format numeric values and percentages, handling NaNs
        def format_value(value, is_percentage=False):
            if pd.notnull(value):
                if is_percentage:
                    # Format percentages, handling potential non-numeric conversion issues earlier
                    try:
                         return f"{float(value):.1f}%"
                    except:
                         return str(value) # Return as string if conversion fails
                else:
                    # Format numeric values with commas, handling potential non-numeric conversion earlier
                    try:
                         return f"{int(float(value)):,}"
                    except:
                         return str(value) # Return as string if conversion fails
            return 'N/A'

        # Create the demographic HTML snippet for each VTD
        master_vtd_gdf['demographics_html'] = master_vtd_gdf.apply(lambda row: f"""
        <b>Demographic Data:</b><br>
        <table style="width:100%;">
          <tr>
            <th>Category</th>
            <th>2020</th>
            <th>2022</th>
            <th>2024</th>
            <th>Change (20-22)</th>
            <th>% Change (20-22)</th>
            <th>Change (22-24)</th>
            <th>% Change (22-24)</th>
          </tr>
          <tr>
            <td>Total Pop</td>
            <td>{format_value(row.get('Total Population (2020)'))}</td>
            <td>{format_value(row.get('Total Population (2022)'))}</td>
            <td>{format_value(row.get('Total Population (2024)'))}</td>
            <td>N/A</td><td>N/A</td>
            <td>N/A</td><td>N/A</td>
          </tr>
          <tr>
            <td>Anglo Pop</td>
            <td>{format_value(row.get('Anglo Population (2020)'))}</td>
            <td>{format_value(row.get('Anglo Population (2022)'))}</td>
            <td>{format_value(row.get('Anglo Population (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Anglo Pop % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Anglo Pop % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
           <tr>
            <td>Non-Anglo Pop</td>
            <td>{format_value(row.get('Non-Anglo Population (2020)'))}</td>
            <td>{format_value(row.get('Non-Anglo Population (2022)'))}</td>
            <td>{format_value(row.get('Non-Anglo Population (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Non-Anglo Pop % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Non-Anglo Pop % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
          <tr>
            <td>Asian Pop</td>
            <td>{format_value(row.get('Asian Population (2020)'))}</td>
            <td>{format_value(row.get('Asian Population (2022)'))}</td>
            <td>{format_value(row.get('Asian Population (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Asian Pop % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Asian Pop % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
          <tr>
            <td>Black Pop</td>
            <td>{format_value(row.get('Black Population (2020)'))}</td>
            <td>{format_value(row.get('Black Population (2022)'))}</td>
            <td>{format_value(row.get('Black Population (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Black Pop % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Black Pop % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
           <tr>
            <td>Hispanic Pop</td>
            <td>{format_value(row.get('Hispanic Population (2020)'))}</td>
            <td>{format_value(row.get('Hispanic Population (2022)'))}</td>
            <td>{format_value(row.get('Hispanic Population (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Hispanic Pop % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Hispanic Pop % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
           <tr>
            <td>Black + Hispanic Pop</td>
            <td>{format_value(row.get('Black + Hispanic Population (2020)'))}</td>
            <td>{format_value(row.get('Black + Hispanic Population (2022)'))}</td>
            <td>{format_value(row.get('Black + Hispanic Population (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Black + Hispanic Pop % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Black + Hispanic Pop % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
           <tr>
            <td>VAP</td>
            <td>{format_value(row.get('Voting Age Population (2020)'))}</td>
            <td>{format_value(row.get('Voting Age Population (2022)'))}</td>
            <td>{format_value(row.get('Voting Age Population (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('VAP % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('VAP % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
           <tr>
            <td>Anglo VAP</td>
            <td>{format_value(row.get('Anglo VAP (2020)'))}</td>
            <td>{format_value(row.get('Anglo VAP (2022)'))}</td>
            <td>{format_value(row.get('Anglo VAP (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Anglo VAP % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Anglo VAP % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
          <tr>
            <td>Non-Anglo VAP</td>
            <td>{format_value(row.get('Non-Anglo VAP (2020)'))}</td>
            <td>{format_value(row.get('Non-Anglo VAP (2022)'))}</td>
            <td>{format_value(row.get('Non-Anglo VAP (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Non-Anglo VAP % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Non-Anglo VAP % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
          <tr>
            <td>Asian VAP</td>
            <td>{format_value(row.get('Asian VAP (2020)'))}</td>
            <td>{format_value(row.get('Asian VAP (2022)'))}</td>
            <td>{format_value(row.get('Asian VAP (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Asian VAP % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Asian VAP % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
          <tr>
            <td>Black VAP</td>
            <td>{format_value(row.get('Black VAP (2020)'))}</td>
            <td>{format_value(row.get('Black VAP (2022)'))}</td>
            <td>{format_value(row.get('Black VAP (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Black VAP % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Black VAP % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
          <tr>
            <td>Hispanic VAP</td>
            <td>{format_value(row.get('Hispanic VAP (2020)'))}</td>
            <td>{format_value(row.get('Hispanic VAP (2022)'))}</td>
            <td>{format_value(row.get('Hispanic VAP (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Hispanic VAP % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Hispanic VAP % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
          <tr>
            <td>Black + Hispanic VAP</td>
            <td>{format_value(row.get('Black + Hispanic VAP (2020)'))}</td>
            <td>{format_value(row.get('Black + Hispanic VAP (2022)'))}</td>
            <td>{format_value(row.get('Black + Hispanic VAP (2024)'))}</td>
            <td>N/A</td><td>{format_value(row.get('Black + Hispanic VAP % Change (2020-2022)'), is_percentage=True)}</td>
            <td>N/A</td><td>{format_value(row.get('Black + Hispanic VAP % Change (2022-2024)'), is_percentage=True)}</td>
          </tr>
        </table>
        """, axis=1)

        print("✅ Created 'demographics_html' column for popup.")

    else:
        print("❌ Necessary demographic data for popup not available. Skipping HTML generation.")
        master_vtd_gdf['demographics_html'] = "Demographic data not available."

    # Print head and columns to verify
    print("\n📋 Head of master_vtd_gdf with 'demographics_html':")
    display(master_vtd_gdf[['CNTYVTD', 'demographics_html']].head())
    print("\nColumns of master_vtd_gdf after demographics popup preparation:")
    print(master_vtd_gdf.columns.tolist())

**Reasoning**:
Prepare the Primary Votes data for the popup dropdown. This involves selecting the relevant vote columns, ensuring they are numeric, and formatting them into an HTML snippet.

In [ ]:
import pandas as pd

# Ensure master_vtd_gdf and primary_votes_cleaned_df are available
if 'master_vtd_gdf' not in locals() or master_vtd_gdf is None or master_vtd_gdf.empty:
    print("❌ master_vtd_gdf GeoDataFrame is not available or empty. Cannot prepare primary vote data for popup.")
elif 'primary_votes_cleaned_df' not in locals() or primary_votes_cleaned_df is None or primary_votes_cleaned_df.empty:
    print("❌ primary_votes_cleaned_df DataFrame is not available or empty. Cannot prepare primary vote data for popup.")
else:
    print("✅ master_vtd_gdf and primary_votes_cleaned_df are available for primary vote popup data preparation.")

    # Define the relevant primary vote columns from primary_votes_cleaned_df
    # These are the cleaned and merged vote columns
    primary_vote_cols = [
        'BufordR_2022_Primary_Votes', 'EllzeyR_2022_Primary_Votes', 'PayneR_2022_Primary_Votes',
        'BufordR_2024_Primary_Votes', 'EllzeyR_2024_Primary_Votes', 'WileyR_22024_Primary_Votes' # Corrected typo WileyR_2024
    ]

    # Ensure these columns exist in master_vtd_gdf (they should after the merge in step 3)
    # and are numeric (should be after cleaning in step 2)
    for col in primary_vote_cols:
        if col not in master_vtd_gdf.columns:
            print(f"⚠️ Primary vote column '{col}' not found in master_vtd_gdf. Creating a placeholder.")
            master_vtd_gdf[col] = pd.NA # Use pandas NA for nullable integer

        # Ensure numeric type, coercing errors to NaN
        master_vtd_gdf[col] = pd.to_numeric(master_vtd_gdf[col], errors='coerce')


    # Define a helper function to format vote counts, handling NaNs
    def format_vote_count(count):
        if pd.notnull(count):
            try:
                return f"{int(count):,}"
            except:
                return str(count) # Return as string if conversion fails
        return 'N/A'


    # Create the Primary Votes HTML snippet for each VTD
    master_vtd_gdf['primary_votes_html'] = master_vtd_gdf.apply(lambda row: f"""
    <b>Primary Vote Data:</b><br>
    <table style="width:100%;">
      <tr>
        <th>Candidate</th>
        <th>2022 Primary</th>
        <th>2024 Primary</th>
      </tr>
      <tr>
        <td>Buford (R)</td>
        <td>{format_vote_count(row.get('BufordR_2022_Primary_Votes'))}</td>
        <td>{format_vote_count(row.get('BufordR_2024_Primary_Votes'))}</td>
      </tr>
      <tr>
        <td>Ellzey (R)</td>
        <td>{format_vote_count(row.get('EllzeyR_2022_Primary_Votes'))}</td>
        <td>{format_vote_count(row.get('EllzeyR_2024_Primary_Votes'))}</td>
      </tr>
      <tr>
        <td>Payne (R)</td>
        <td>{format_vote_count(row.get('PayneR_2022_Primary_Votes'))}</td>
        <td>N/A</td> <!-- Payne was not in the 2024 primary -->
      </tr>
      <tr>
        <td>Wiley (R)</td>
        <td>N/A</td> <!-- Wiley was not in the 2022 primary -->
        <td>{format_vote_count(row.get('WileyR_22024_Primary_Votes'))}</td> # Corrected typo WileyR_2024
      </tr>
    </table>
    """, axis=1)

    print("✅ Created 'primary_votes_html' column for popup.")

    # Print head and columns to verify
    print("\n📋 Head of master_vtd_gdf with 'primary_votes_html':")
    display(master_vtd_gdf[['CNTYVTD', 'primary_votes_html']].head())
    print("\nColumns of master_vtd_gdf after primary votes popup preparation:")
    print(master_vtd_gdf.columns.tolist())

# START START START DELETE ABOVE IF WORKS
Task
Create a Python script using the provided data files ("vtds_cd6.geojson", "cleaned_torv_data.csv", "DemographicsFinal.csv", "PrimaryVotesFinal.csv") to generate an interactive Folium map of CD6. The map should display VTDs with popups containing detailed information (geographic, demographic, primary vote data, and VR/Turnout data) and tooltips showing the 'CNTYVTD'. Include layers for county boundaries and the CD6 boundary, and a layer control for toggling layers. The script should be structured into four blocks: 1) Data loading and inspection, 2) Attribute data preparation and merging, 3) Html snippet generation, and 4) Folium map creation and layer addition. Ensure 'CNTYVTD' is standardized across all dataframes for merging.

## Data loading and inspection

### Subtask:
Load all necessary data files (`vtds_cd6.geojson`, `cleaned_torv_data.csv`, `DemographicsFinal.csv`, and `PrimaryVotesFinal.csv`), display their heads and column names.


**Reasoning**:
Load and display the head and columns of the four required data files.



In [ ]:
import pandas as pd
import geopandas as gpd
import os
from IPython.display import display

print("--- Loading and Inspecting Data Files ---")

# 1. Define the file paths
vtds_geojson_path = 'vtds_cd6.geojson'
cleaned_torv_path = 'cleaned_torv_data.csv'
demographics_file_path = "DemographicsFinal.csv"
primary_votes_file_path = "PrimaryVotesFinal.csv" # Corrected file name based on previous output

# 2. For each file:

# a. vtds_cd6.geojson
if os.path.exists(vtds_geojson_path):
    try:
        vtds_cd6_gdf = gpd.read_file(vtds_geojson_path)
        print(f"\n✅ Loaded {vtds_geojson_path}")
        print(f"\n📋 Head of {vtds_geojson_path}:")
        display(vtds_cd6_gdf.head())
        print("\nColumns:")
        print(vtds_cd6_gdf.columns.tolist())
    except Exception as e:
        print(f"❌ Error loading {vtds_geojson_path}: {e}")
        vtds_cd6_gdf = None
else:
    print(f"\n❌ {vtds_geojson_path} not found.")
    vtds_cd6_gdf = None


# b. cleaned_torv_data.csv
if os.path.exists(cleaned_torv_path):
    try:
        cleaned_torv_df = pd.read_csv(cleaned_torv_path)
        print(f"\n✅ Loaded {cleaned_torv_path}")
        print(f"\n📋 Head of {cleaned_torv_path}:")
        display(cleaned_torv_df.head())
        print("\nColumns:")
        print(cleaned_torv_df.columns.tolist())
    except Exception as e:
        print(f"❌ Error loading {cleaned_torv_path}: {e}")
        cleaned_torv_df = None
else:
    print(f"\n❌ {cleaned_torv_path} not found.")
    cleaned_torv_df = None


# c. DemographicsFinal.csv
if os.path.exists(demographics_file_path):
    try:
        demographics_df = pd.read_csv(demographics_file_path)
        print(f"\n✅ Loaded {demographics_file_path}")
        print(f"\n📋 Head of {demographics_file_path}:")
        display(demographics_df.head())
        print("\nColumns:")
        print(demographics_df.columns.tolist())
    except Exception as e:
        print(f"❌ Error loading {demographics_file_path}: {e}")
        demographics_df = None
else:
    print(f"\n❌ {demographics_file_path} not found.")
    demographics_df = None


# d. PrimaryVotesFinal.csv
if os.path.exists(primary_votes_file_path):
    try:
        primary_votes_df = pd.read_csv(primary_votes_file_path)
        print(f"\n✅ Loaded {primary_votes_file_path}")
        print(f"\n📋 Head of {primary_votes_file_path}:")
        display(primary_votes_df.head())
        print("\nColumns:")
        print(primary_votes_df.columns.tolist())
    except Exception as e:
        print(f"❌ Error loading {primary_votes_file_path}: {e}")
        primary_votes_df = None
else:
    print(f"\n❌ {primary_votes_file_path} not found.")
    primary_votes_df = None

print("\n--- Data Loading and Inspection Complete ---")

## Attribute data preparation and merging

### Subtask:
Clean and standardize 'CNTYVTD' in all attribute dataframes. Create `primary_votes_cleaned_df` by processing `PrimaryVotesFinal.csv`. Merge `cleaned_torv_data.csv`, `DemographicsFinal.csv` (for geographic and demographic data), and the cleaned primary votes data onto the base VTD GeoDataFrame using 'CNTYVTD'. Calculate vote changes and percentages.


**Reasoning**:
Clean and standardize CNTYVTD in the loaded dataframes, process primary votes data, and merge the attribute data onto the VTD GeoDataFrame. This combines steps 2 through 7 from the instructions.



In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

# 1. Define a consistent projected CRS (e.g., EPSG:3083)
proj_crs = "EPSG:3083" # Using the projected CRS defined in previous steps

print("--- Data Preparation and Merging ---")

# 2. Ensure vtds_cd6_gdf is available and has the cleaned 'CNTYVTD'
master_vtd_gdf = None
merge_failed = False # Flag to track if the base GeoDataFrame is ready for merging

if 'vtds_cd6_gdf' in locals() and vtds_cd6_gdf is not None and not vtds_cd6_gdf.empty:
    print("✅ vtds_cd6_gdf is available.")

    # Create or standardize the 'CNTYVTD' column
    if 'CNTY_x' in vtds_cd6_gdf.columns and 'VTD_x' in vtds_cd6_gdf.columns:
        vtds_cd6_gdf['CNTYVTD'] = vtds_cd6_gdf['CNTY_x'].astype(str).str.strip() + vtds_cd6_gdf['VTD_x'].astype(str).str.strip()
        print("✅ Created 'CNTYVTD' in vtds_cd6_gdf from CNTY_x and VTD_x.")
    elif 'CNTYVTD_x' in vtds_cd6_gdf.columns:
         vtds_cd6_gdf.rename(columns={'CNTYVTD_x': 'CNTYVTD'}, inplace=True)
         print("✅ Using existing 'CNTYVTD_x' as 'CNTYVTD' in vtds_cd6_gdf.")
    elif 'CNTYVTD_y' in vtds_cd6_gdf.columns: # Check for _y if it was merged from pop data
         vtds_cd6_gdf.rename(columns={'CNTYVTD_y': 'CNTYVTD'}, inplace=True)
         print("✅ Using existing 'CNTYVTD_y' as 'CNTYVTD' in vtds_cd6_gdf.")
    elif 'CNTYVTD' in vtds_cd6_gdf.columns:
         print("✅ Using existing 'CNTYVTD' in vtds_cd6_gdf.")
    else:
         print("❌ Could not find a suitable VTD identifier column in vtds_cd6_gdf to create CNTYVTD. Cannot merge attributes.")
         merge_failed = True # Flag merge failure

    if not merge_failed and 'CNTYVTD' in vtds_cd6_gdf.columns:
        # Ensure CNTYVTD is string and left-strip leading zeros
        vtds_cd6_gdf['CNTYVTD'] = vtds_cd6_gdf['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
        print("✅ Cleaned 'CNTYVTD' in vtds_cd6_gdf.")

        # Reproject to the target projected CRS if needed (assuming it might not be in the target_crs after initial load)
        if vtds_cd6_gdf.crs is None:
             print(f"⚠️ vtds_cd6_gdf CRS missing, assuming EPSG:3857 and setting to {proj_crs}")
             vtds_cd6_gdf.set_crs("EPSG:3857", inplace=True) # Assume a common web CRS if missing

        if vtds_cd6_gdf.crs != proj_crs:
             vtds_cd6_gdf_proj = vtds_cd6_gdf.to_crs(proj_crs)
             print(f"✅ Reprojected vtds_cd6_gdf to {proj_crs} for spatial operations.")
        else:
             vtds_cd6_gdf_proj = vtds_cd6_gdf.copy()
             print(f"✅ vtds_cd6_gdf is already in {proj_crs}.")


        # Create the master GeoDataFrame with essential columns and geometry
        master_vtd_gdf = vtds_cd6_gdf_proj[['CNTYVTD', 'geometry']].copy()
        print(f"✅ Started master GeoDataFrame with {len(master_vtd_gdf)} VTDs.")

    else:
         print("❌ CNTYVTD column not available in vtds_cd6_gdf. Cannot create master GeoDataFrame.")
         merge_failed = True

else:
    print("❌ vtds_cd6_gdf is not available or empty. Cannot merge attributes.")
    # Create an empty GeoDataFrame with the expected columns to avoid errors
    master_vtd_gdf = gpd.GeoDataFrame({'CNTYVTD': [], 'geometry': []}, crs=proj_crs)
    merge_failed = True # Flag merge failure


# 3. Ensure attribute dataframes are available
if 'cleaned_torv_df' not in locals() or cleaned_torv_df is None or cleaned_torv_df.empty:
    print("⚠️ cleaned_torv_df not available or empty. TORV data will be missing.")
    cleaned_torv_df = pd.DataFrame({'CNTYVTD': []}) # Create empty df to prevent errors
if 'demographics_df' not in locals() or demographics_df is None or demographics_df.empty:
    print("⚠️ demographics_df not available or empty. Demographic data will be missing.")
    demographics_df = pd.DataFrame({'VTD Identifier': []}) # Create empty df to prevent errors
if 'primary_votes_df' not in locals() or primary_votes_df is None or primary_votes_df.empty:
    print("⚠️ primary_votes_df not available or empty. Primary vote data will be missing.")
    primary_votes_df = pd.DataFrame({'VTD Identifier': [], 'CNTYVTD': [], 'CNTYVTD.1': []}) # Create empty df with expected key columns


# 4. Clean and standardize 'CNTYVTD' in cleaned_torv_df
if 'CNTYVTD' in cleaned_torv_df.columns:
    cleaned_torv_df['CNTYVTD'] = cleaned_torv_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
    print("✅ Cleaned 'CNTYVTD' in cleaned_torv_df.")
else:
    print("⚠️ 'CNTYVTD' column not found in cleaned_torv_df. Skipping CNTYVTD cleaning.")


# 5. Clean and select columns for demographics_geo_df and demographics_popup_df
demographics_geo_df = pd.DataFrame({'CNTYVTD': []}) # Initialize empty
demographics_popup_df = pd.DataFrame({'CNTYVTD': []}) # Initialize empty

if 'VTD Identifier' in demographics_df.columns:
    try:
        # Prepare demographics_geo_df
        geo_cols_to_select = ['VTD Identifier', 'Best Matching Precinct Name (Max Overlap)', 'County Name (from TORV)', 'Cities (from TORV)']
        geo_cols_present = [col for col in geo_cols_to_select if col in demographics_df.columns]
        if len(geo_cols_present) > 1: # Need at least VTD Identifier and one other geo column
            demographics_geo_df = demographics_df[geo_cols_present].copy()
            demographics_geo_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
            demographics_geo_df['CNTYVTD'] = demographics_geo_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
            if not demographics_geo_df['CNTYVTD'].is_unique:
                print("⚠️ Duplicate CNTYVTDs in demographics geo data. Aggregating (taking first).")
                agg_funcs_geo = {col: 'first' for col in demographics_geo_df.columns if col != 'CNTYVTD'}
                demographics_geo_df = demographics_geo_df.groupby('CNTYVTD', as_index=False).agg(agg_funcs_geo)
            print(f"✅ Prepared demographics_geo_df with {len(demographics_geo_df)} unique CNTYVTDs.")
        else:
            print("⚠️ Not enough geographic columns found in demographics_df for demographics_geo_df.")


        # Prepare demographics_popup_df (all demographic columns)
        demo_cols_to_select_popup = [col for col in demographics_df.columns if col != 'VTD Identifier']
        if 'VTD Identifier' in demographics_df.columns: # Ensure 'VTD Identifier' exists before selecting
             demographics_popup_df = demographics_df[['VTD Identifier'] + demo_cols_to_select_popup].copy()
             demographics_popup_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
             demographics_popup_df['CNTYVTD'] = demographics_popup_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
             if not demographics_popup_df['CNTYVTD'].is_unique:
                  print("⚠️ Duplicate CNTYVTDs in demographics popup data. Aggregating (taking first).")
                  agg_funcs_demo_popup = {col: 'first' for col in demographics_popup_df.columns if col != 'CNTYVTD'}
                  demographics_popup_df = demographics_popup_df.groupby('CNTYVTD', as_index=False).agg(agg_funcs_demo_popup)
             print(f"✅ Prepared demographics_popup_df with {len(demographics_popup_df)} unique CNTYVTDs.")
        else:
             print("⚠️ 'VTD Identifier' not found in demographics_df for popup data.")


    except Exception as e:
        print(f"❌ Error processing demographics_df: {e}")
        demographics_geo_df = pd.DataFrame({'CNTYVTD': []}) # Reset to empty on error
        demographics_popup_df = pd.DataFrame({'CNTYVTD': []}) # Reset to empty on error

else:
    print("⚠️ 'VTD Identifier' column not found in demographics_df. Cannot prepare demographic dataframes.")


# 6. Process primary_votes_df - FIXING KEY SELECTION
primary_votes_cleaned_df = pd.DataFrame({'CNTYVTD': []}) # Initialize empty
if not primary_votes_df.empty:
    # Debug: Print columns of primary_votes_df before selecting
    print("\nDebug: Columns in primary_votes_df:", primary_votes_df.columns.tolist())

    vote_cols_2022_raw = ['CNTYVTD', 'BufordR_22P_U.S. Rep 6', 'EllzeyR_22P_U.S. Rep 6', 'PayneR_22P_U.S. Rep 6']
    vote_cols_2024_raw = ['CNTYVTD.1', 'BufordR_24P_U.S. Rep 6', 'EllzeyR_24P_U.S. Rep 6', 'WileyR_24P_U.S. Rep 6']

    # Select columns, only keeping those that exist
    votes_2022_df = primary_votes_df[[col for col in vote_cols_2022_raw if col in primary_votes_df.columns]].copy()
    votes_2024_df = primary_votes_df[[col for col in vote_cols_2024_raw if col in primary_votes_df.columns]].copy()

    # Debug: Print columns of votes_2022_df and votes_2024_df after selection
    print("Debug: Columns in votes_2022_df after selection:", votes_2022_df.columns.tolist())
    print("Debug: Columns in votes_2024_df after selection:", votes_2024_df.columns.tolist())


    # Clean keys and rename for merging
    if 'CNTYVTD' in votes_2022_df.columns:
        votes_2022_df['CNTYVTD'] = votes_2022_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
        print("✅ Cleaned 'CNTYVTD' in votes_2022_df.")
    else:
        # If 'CNTYVTD' is not in 2022, check for 'VTD Identifier' which might be the key for 2022 data in this file
        if 'VTD Identifier' in votes_2022_df.columns:
            votes_2022_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
            votes_2022_df['CNTYVTD'] = votes_2022_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
            print("✅ Used and cleaned 'VTD Identifier' as 'CNTYVTD' in votes_2022_df.")
        else:
             print("❌ Neither 'CNTYVTD' nor 'VTD Identifier' found in 2022 vote data columns. Cannot process 2022 votes.")
             votes_2022_df = pd.DataFrame({'CNTYVTD': []}) # Create empty to prevent merge errors


    if 'CNTYVTD.1' in votes_2024_df.columns:
        votes_2024_df.rename(columns={'CNTYVTD.1': 'CNTYVTD'}, inplace=True)
        print("✅ Renamed 'CNTYVTD.1' to 'CNTYVTD' in votes_2024_df.")
        if 'CNTYVTD' in votes_2024_df.columns: # Ensure it exists after rename
             votes_2024_df['CNTYVTD'] = votes_2024_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
             print("✅ Cleaned 'CNTYVTD' in votes_2024_df.")
    else:
         # If 'CNTYVTD.1' is not in 2024, check for 'CNTYVTD' or 'VTD Identifier' as a fallback
         if 'CNTYVTD' in votes_2024_df.columns:
             votes_2024_df['CNTYVTD'] = votes_2024_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
             print("✅ Used and cleaned existing 'CNTYVTD' in votes_2024_df.")
         elif 'VTD Identifier' in votes_2024_df.columns:
              votes_2024_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
              votes_2024_df['CNTYVTD'] = votes_2024_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
              print("✅ Used and cleaned 'VTD Identifier' as 'CNTYVTD' in votes_2024_df.")
         else:
              print("❌ Neither 'CNTYVTD.1', 'CNTYVTD', nor 'VTD Identifier' found in 2024 vote data columns. Cannot process 2024 votes.")
              votes_2024_df = pd.DataFrame({'CNTYVTD': []}) # Create empty to prevent merge errors


    # Merge 2022 and 2024 data only if both have the CNTYVTD key
    if 'CNTYVTD' in votes_2022_df.columns and 'CNTYVTD' in votes_2024_df.columns:
        primary_votes_merged = pd.merge(
            votes_2022_df,
            votes_2024_df,
            on='CNTYVTD',
            how='outer'
        )

        # Aggregate potential duplicates
        if not primary_votes_merged.empty and 'CNTYVTD' in primary_votes_merged.columns and primary_votes_merged['CNTYVTD'].duplicated().any():
             print("⚠️ Duplicates found on CNTYVTD after initial primary vote merge. Aggregating (taking first).")
             agg_funcs_votes = {col: 'first' for col in primary_votes_merged.columns if col != 'CNTYVTD'}
             primary_votes_cleaned_df = primary_votes_merged.groupby('CNTYVTD', as_index=False).agg(agg_funcs_votes)
        else:
             primary_votes_cleaned_df = primary_votes_merged.copy()

        # Ensure all vote columns are numeric
        if not primary_votes_cleaned_df.empty:
            vote_cols_to_numeric = [col for col in primary_votes_cleaned_df.columns if col != 'CNTYVTD']
            for col in vote_cols_to_numeric:
                primary_votes_cleaned_df[col] = pd.to_numeric(primary_votes_cleaned_df[col], errors='coerce')
            print("✅ Processed and cleaned primary votes data.")
        else:
             print("⚠️ primary_votes_cleaned_df is empty after processing.")

    else:
        print("❌ Cannot merge 2022 and 2024 primary vote data because 'CNTYVTD' is missing in one or both dataframes.")


# 7. Merge attribute data onto the master GeoDataFrame
if not merge_failed and master_vtd_gdf is not None and not master_vtd_gdf.empty:
    print("\n--- Merging Attribute Data onto Master GeoDataFrame ---")

    # 7a. Merge demographics_geo_df
    if not demographics_geo_df.empty:
        print("Merging demographics_geo_df...")
        # Identify geo columns to merge from demographics_geo_df
        geo_cols_to_merge = [col for col in demographics_geo_df.columns if col != 'CNTYVTD']
        # Drop potential duplicate columns in master_vtd_gdf before merging
        master_vtd_gdf.drop(columns=[col for col in geo_cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
        master_vtd_gdf = master_vtd_gdf.merge(demographics_geo_df, on='CNTYVTD', how='left')
        print(f"✅ Merged demographics geo info. Rows after merge: {len(master_vtd_gdf)}")
    else:
        print("⚠️ demographics_geo_df is empty. Skipping merge.")
        # Add placeholder columns if merge is skipped
        geo_cols_placeholder = ['Best Matching Precinct Name (Max Overlap)', 'County Name (from TORV)', 'Cities (from TORV)']
        for col in geo_cols_placeholder:
            if col not in master_vtd_gdf.columns:
                master_vtd_gdf[col] = np.nan # Use NaN for missing data


    # 7b. Merge cleaned_torv_df
    if not cleaned_torv_df.empty:
        print("Merging cleaned_torv_df...")
        # Identify TORV columns to merge, excluding the geo columns already handled
        torv_cols_to_merge = [col for col in cleaned_torv_df.columns if col not in ['CNTYVTD', 'Precinct', 'County', 'Intersecting Cities', 'Best Matching Precinct Name (Max Overlap)', 'County Name (from TORV)', 'Cities (from TORV)']]
        # Drop potential duplicate columns in master_vtd_gdf before merging TORV data
        master_vtd_gdf.drop(columns=[col for col in torv_cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')

        # Select only the relevant columns from cleaned_torv_df for the merge
        cols_from_torv = ['CNTYVTD'] + torv_cols_to_merge
        master_vtd_gdf = master_vtd_gdf.merge(cleaned_torv_df[cols_from_torv], on='CNTYVTD', how='left')
        print(f"✅ Merged cleaned TORV data. Rows after merge: {len(master_vtd_gdf)}")
    else:
        print("⚠️ cleaned_torv_df is empty. Skipping merge.")
        # Add placeholder columns for key TORV metrics if merge is skipped
        torv_placeholder_cols = [col for col in cleaned_torv_df.columns if col != 'CNTYVTD'] # Use all original TORV cols except key
        for col in torv_placeholder_cols:
            if col not in master_vtd_gdf.columns:
                master_vtd_gdf[col] = np.nan


    # 7c. Merge primary_votes_cleaned_df
    if not primary_votes_cleaned_df.empty:
        print("Merging primary_votes_cleaned_df...")
        # Identify vote columns to merge
        vote_cols_to_merge = [col for col in primary_votes_cleaned_df.columns if col != 'CNTYVTD']
         # Drop potential duplicate columns in master_vtd_gdf before merging primary votes
        master_vtd_gdf.drop(columns=[col for col in vote_cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
        master_vtd_gdf = master_vtd_gdf.merge(primary_votes_cleaned_df, on='CNTYVTD', how='left')
        print(f"✅ Merged primary votes data. Rows after merge: {len(master_vtd_gdf)}")
    else:
        print("⚠️ primary_votes_cleaned_df is empty. Skipping merge.")
        # Add placeholder columns for key primary vote metrics if merge is skipped
        # Use the expected final column names
        vote_placeholder_cols = [
            'BufordR_22P_U.S. Rep 6', 'EllzeyR_22P_U.S. Rep 6', 'PayneR_22P_U.S. Rep 6',
            'BufordR_24P_U.S. Rep 6', 'EllzeyR_24P_U.S. Rep 6', 'WileyR_24P_U.S. Rep 6'
        ]
        for col in vote_placeholder_cols:
             if col not in master_vtd_gdf.columns:
                  master_vtd_gdf[col] = np.nan


    # 7d. Calculate Vote Changes and Percentages (only if necessary columns exist)
    print("\n--- Calculating Vote Changes and Percentages ---")
    vote_cols_for_calc = {
        'BufordR': {'2022': 'BufordR_22P_U.S. Rep 6', '2024': 'BufordR_24P_U.S. Rep 6'},
        'EllzeyR': {'2022': 'EllzeyR_22P_U.S. Rep 6', '2024': 'EllzeyR_24P_U.S. Rep 6'},
        'PayneR': {'2022': 'PayneR_22P_U.S. Rep 6', '2024': None}, # Payne only in 2022
        'WileyR': {'2022': None, '2024': 'WileyR_24P_U.S. Rep 6'} # Wiley only in 2024
    }

    for candidate, years in vote_cols_for_calc.items():
        col_2022 = years['2022']
        col_2024 = years['2024']

        # Ensure columns exist and are numeric
        col_2022_exists = col_2022 in master_vtd_gdf.columns
        col_2024_exists = col_2024 in master_vtd_gdf.columns

        if col_2022_exists:
             master_vtd_gdf[col_2022] = pd.to_numeric(master_vtd_gdf[col_2022], errors='coerce').fillna(0)
        if col_2024_exists:
             master_vtd_gdf[col_2024] = pd.to_numeric(master_vtd_gdf[col_2024], errors='coerce').fillna(0)


        if col_2022_exists and col_2024_exists:
            # Calculate Amount Change
            amount_change_col = f'{candidate}_Amount_Change_2022_2024'
            master_vtd_gdf[amount_change_col] = master_vtd_gdf[col_2024] - master_vtd_gdf[col_2022]
            print(f"✅ Calculated '{amount_change_col}'.")

            # Calculate Percentage Change
            pct_change_col = f'{candidate}_Pct_Change_2022_2024'
            denominator = master_vtd_gdf[col_2022].replace(0, np.nan)
            master_vtd_gdf[pct_change_col] = ((master_vtd_gdf[col_2024] - master_vtd_gdf[col_2022]) / denominator) * 100
            master_vtd_gdf[pct_change_col].replace([np.inf, -np.inf], np.nan, inplace=True)
            print(f"✅ Calculated '{pct_change_col}'.")
        elif col_2022_exists and col_2024 is None:
             print(f"⚠️ Only 2022 data available for {candidate}. Skipping 2022-2024 change calculations.")
        elif col_2022 is None and col_2024_exists:
             print(f"⚠️ Only 2024 data available for {candidate}. Skipping 2022-2024 change calculations.")
        else:
             print(f"❌ Necessary vote columns for {candidate} not found for change calculations. Skipping.")
             # Add placeholder columns for changes if calculation was skipped
             master_vtd_gdf[f'{candidate}_Amount_Change_2022_2024'] = np.nan
             master_vtd_gdf[f'{candidate}_Pct_Change_2022_2024'] = np.nan


    # 7e. Calculate Demographic Changes and Percentages (only if necessary columns exist)
    print("\n--- Calculating Demographic Changes and Percentages ---")
    demographic_years = ['2020', '2022', '2024']
    demographic_categories = [
        'Total Population', 'Anglo Population', 'Non-Anglo Population',
        'Asian Population', 'Black Population', 'Hispanic Population', 'Black + Hispanic Population',
        'Voting Age Population', 'Anglo VAP', 'Non-Anglo VAP',
        'Asian VAP', 'Black VAP', 'Hispanic VAP', 'Black + Hispanic VAP'
    ]

    for category in demographic_categories:
        col_2020 = f'{category} ({demographic_years[0]})'
        col_2022 = f'{category} ({demographic_years[1]})'
        col_2024 = f'{category} ({demographic_years[2]})'

        # Ensure columns exist and are numeric
        col_2020_exists = col_2020 in master_vtd_gdf.columns
        col_2022_exists = col_2022 in master_vtd_gdf.columns
        col_2024_exists = col_2024 in master_vtd_gdf.columns

        if col_2020_exists:
             master_vtd_gdf[col_2020] = pd.to_numeric(master_vtd_gdf[col_2020], errors='coerce')
        if col_2022_exists:
             master_vtd_gdf[col_2022] = pd.to_numeric(master_vtd_gdf[col_2022], errors='coerce')
        if col_2024_exists:
             master_vtd_gdf[col_2024] = pd.to_numeric(master_vtd_gdf[col_2024], errors='coerce')


        # Calculate 2020-2022 Changes
        if col_2020_exists and col_2022_exists:
            change_col_2020_2022 = f'{category} Change 2020-2022'
            pct_change_col_2020_2022 = f'{category} % Change 2020-2022'

            master_vtd_gdf[change_col_2020_2022] = master_vtd_gdf[col_2022] - master_vtd_gdf[col_2020]
            denominator = master_vtd_gdf[col_2020].replace(0, np.nan)
            master_vtd_gdf[pct_change_col_2020_2022] = ((master_vtd_gdf[col_2022] - master_vtd_gdf[col_2020]) / denominator) * 100
            master_vtd_gdf[pct_change_col_2020_2022].replace([np.inf, -np.inf], np.nan, inplace=True)
            print(f"✅ Calculated '{change_col_2020_2022}' and '{pct_change_col_2020_2022}'.")
        else:
            print(f"❌ Missing columns for {category} Change 2020-2022. Skipping calculation.")
            master_vtd_gdf[f'{category} Change 2020-2022'] = np.nan
            master_vtd_gdf[f'{category} % Change 2020-2022'] = np.nan


        # Calculate 2022-2024 Changes
        if col_2022_exists and col_2024_exists:
            change_col_2022_2024 = f'{category} Change 2022-2024'
            pct_change_col_2022_2024 = f'{category} % Change 2022-2024'

            master_vtd_gdf[change_col_2022_2024] = master_vtd_gdf[col_2024] - master_vtd_gdf[col_2022]
            denominator = master_vtd_gdf[col_2022].replace(0, np.nan)
            master_vtd_gdf[pct_change_col_2022_2024] = ((master_vtd_gdf[col_2024] - master_vtd_gdf[col_2022]) / denominator) * 100
            master_vtd_gdf[pct_change_col_2022_2024].replace([np.inf, -np.inf], np.nan, inplace=True)
            print(f"✅ Calculated '{change_col_2022_2024}' and '{pct_change_col_2022_2024}'.")
        else:
            print(f"❌ Missing columns for {category} Change 2022-2024. Skipping calculation.")
            master_vtd_gdf[f'{category} Change 2022-2024'] = np.nan
            master_vtd_gdf[f'{category} % Change 2022-2024'] = np.nan


    # 8. Print the shape, head, and column names of the final master_vtd_gdf
    print("\n--- Final Merged GeoDataFrame Inspection ---")
    print(f"Merged GeoDataFrame shape: {master_vtd_gdf.shape}")
    print("\nHead of merged master_vtd_gdf:")
    display(master_vtd_gdf.head())
    print("\nColumns of merged master_vtd_gdf:")
    print(master_vtd_gdf.columns.tolist())

else:
    print("\n❌ Merge process skipped due to missing or invalid base GeoDataFrame.")

**Reasoning**:
The traceback indicates that the 2022 and 2024 primary vote data did not contain the expected key columns after selection. The debug output shows that the original `primary_votes_df` uses "VTD Identifier" for the key. The selection logic for `votes_2022_df` and `votes_2024_df` needs to be adjusted to explicitly select "VTD Identifier" and then rename it to "CNTYVTD" in both dataframes before attempting the merge. The calculation of demographic changes also failed due to missing columns, although the debug output from the previous cell shows the demographic columns were present in `master_vtd_gdf` after merging. I need to re-examine the column names being used for the demographic calculations and ensure they match the columns present in `master_vtd_gdf` after the merges. I will regenerate the data preparation and merging code block again, fixing the primary vote data processing and the demographic change calculations.



In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

# 1. Define a consistent projected CRS (e.g., EPSG:3083)
proj_crs = "EPSG:3083" # Using the projected CRS defined in previous steps

print("--- Data Preparation and Merging ---")

# 2. Ensure vtds_cd6_gdf is available and has the cleaned 'CNTYVTD'
master_vtd_gdf = None
merge_failed = False # Flag to track if the base GeoDataFrame is ready for merging

if 'vtds_cd6_gdf' in locals() and vtds_cd6_gdf is not None and not vtds_cd6_gdf.empty:
    print("✅ vtds_cd6_gdf is available.")

    # Create or standardize the 'CNTYVTD' column
    if 'CNTY_x' in vtds_cd6_gdf.columns and 'VTD_x' in vtds_cd6_gdf.columns:
        vtds_cd6_gdf['CNTYVTD'] = vtds_cd6_gdf['CNTY_x'].astype(str).str.strip() + vtds_cd6_gdf['VTD_x'].astype(str).str.strip()
        print("✅ Created 'CNTYVTD' in vtds_cd6_gdf from CNTY_x and VTD_x.")
    elif 'CNTYVTD_x' in vtds_cd6_gdf.columns:
         vtds_cd6_gdf.rename(columns={'CNTYVTD_x': 'CNTYVTD'}, inplace=True)
         print("✅ Using existing 'CNTYVTD_x' as 'CNTYVTD' in vtds_cd6_gdf.")
    elif 'CNTYVTD_y' in vtds_cd6_gdf.columns: # Check for _y if it was merged from pop data
         vtds_cd6_gdf.rename(columns={'CNTYVTD_y': 'CNTYVTD'}, inplace=True)
         print("✅ Using existing 'CNTYVTD_y' as 'CNTYVTD' in vtds_cd6_gdf.")
    elif 'CNTYVTD' in vtds_cd6_gdf.columns:
         print("✅ Using existing 'CNTYVTD' in vtds_cd6_gdf.")
    else:
         print("❌ Could not find a suitable VTD identifier column in vtds_cd6_gdf to create CNTYVTD. Cannot merge attributes.")
         merge_failed = True # Flag merge failure

    if not merge_failed and 'CNTYVTD' in vtds_cd6_gdf.columns:
        # Ensure CNTYVTD is string and left-strip leading zeros
        vtds_cd6_gdf['CNTYVTD'] = vtds_cd6_gdf['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
        print("✅ Cleaned 'CNTYVTD' in vtds_cd6_gdf.")

        # Reproject to the target projected CRS if needed (assuming it might not be in the target_crs after initial load)
        if vtds_cd6_gdf.crs is None:
             print(f"⚠️ vtds_cd6_gdf CRS missing, assuming EPSG:3857 and setting to {proj_crs}")
             vtds_cd6_gdf.set_crs("EPSG:3857", inplace=True) # Assume a common web CRS if missing

        if vtds_cd6_gdf.crs != proj_crs:
             vtds_cd6_gdf_proj = vtds_cd6_gdf.to_crs(proj_crs)
             print(f"✅ Reprojected vtds_cd6_gdf to {proj_crs} for spatial operations.")
        else:
             vtds_cd6_gdf_proj = vtds_cd6_gdf.copy()
             print(f"✅ vtds_cd6_gdf is already in {proj_crs}.")


        # Create the master GeoDataFrame with essential columns and geometry
        master_vtd_gdf = vtds_cd6_gdf_proj[['CNTYVTD', 'geometry']].copy()
        print(f"✅ Started master GeoDataFrame with {len(master_vtd_gdf)} VTDs.")

    else:
         print("❌ CNTYVTD column not available in vtds_cd6_gdf. Cannot create master GeoDataFrame.")
         merge_failed = True

else:
    print("❌ vtds_cd6_gdf is not available or empty. Cannot merge attributes.")
    # Create an empty GeoDataFrame with the expected columns to avoid errors
    master_vtd_gdf = gpd.GeoDataFrame({'CNTYVTD': [], 'geometry': []}, crs=proj_crs)
    merge_failed = True # Flag merge failure


# 3. Ensure attribute dataframes are available
if 'cleaned_torv_df' not in locals() or cleaned_torv_df is None or cleaned_torv_df.empty:
    print("⚠️ cleaned_torv_df not available or empty. TORV data will be missing.")
    cleaned_torv_df = pd.DataFrame({'CNTYVTD': []}) # Create empty df to prevent errors
if 'demographics_df' not in locals() or demographics_df is None or demographics_df.empty:
    print("⚠️ demographics_df not available or empty. Demographic data will be missing.")
    demographics_df = pd.DataFrame({'VTD Identifier': []}) # Create empty df to prevent errors
if 'primary_votes_df' not in locals() or primary_votes_df is None or primary_votes_df.empty:
    print("⚠️ primary_votes_df not available or empty. Primary vote data will be missing.")
    primary_votes_df = pd.DataFrame({'VTD Identifier': [], 'CNTYVTD': [], 'CNTYVTD.1': []}) # Create empty df with expected key columns


# 4. Clean and standardize 'CNTYVTD' in cleaned_torv_df
if 'CNTYVTD' in cleaned_torv_df.columns:
    cleaned_torv_df['CNTYVTD'] = cleaned_torv_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
    print("✅ Cleaned 'CNTYVTD' in cleaned_torv_df.")
else:
    print("⚠️ 'CNTYVTD' column not found in cleaned_torv_df. Skipping CNTYVTD cleaning.")


# 5. Clean and select columns for demographics_geo_df and demographics_popup_df
demographics_geo_df = pd.DataFrame({'CNTYVTD': []}) # Initialize empty
demographics_popup_df = pd.DataFrame({'CNTYVTD': []}) # Initialize empty

if 'VTD Identifier' in demographics_df.columns:
    try:
        # Prepare demographics_geo_df
        geo_cols_to_select = ['VTD Identifier', 'Best Matching Precinct Name (Max Overlap)', 'County Name (from TORV)', 'Cities (from TORV)']
        geo_cols_present = [col for col in geo_cols_to_select if col in demographics_df.columns]
        if len(geo_cols_present) > 1: # Need at least VTD Identifier and one other geo column
            demographics_geo_df = demographics_df[geo_cols_present].copy()
            demographics_geo_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
            demographics_geo_df['CNTYVTD'] = demographics_geo_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
            if not demographics_geo_df['CNTYVTD'].is_unique:
                print("⚠️ Duplicate CNTYVTDs in demographics geo data. Aggregating (taking first).")
                agg_funcs_geo = {col: 'first' for col in demographics_geo_df.columns if col != 'CNTYVTD'}
                demographics_geo_df = demographics_geo_df.groupby('CNTYVTD', as_index=False).agg(agg_funcs_geo)
            print(f"✅ Prepared demographics_geo_df with {len(demographics_geo_df)} unique CNTYVTDs.")
        else:
            print("⚠️ Not enough geographic columns found in demographics_df for demographics_geo_df.")


        # Prepare demographics_popup_df (all demographic columns)
        demo_cols_to_select_popup = [col for col in demographics_df.columns if col != 'VTD Identifier']
        if 'VTD Identifier' in demographics_df.columns: # Ensure 'VTD Identifier' exists before selecting
             demographics_popup_df = demographics_df[['VTD Identifier'] + demo_cols_to_select_popup].copy()
             demographics_popup_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
             demographics_popup_df['CNTYVTD'] = demographics_popup_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
             if not demographics_popup_df['CNTYVTD'].is_unique:
                  print("⚠️ Duplicate CNTYVTDs in demographics popup data. Aggregating (taking first).")
                  agg_funcs_demo_popup = {col: 'first' for col in demographics_popup_df.columns if col != 'CNTYVTD'}
                  demographics_popup_df = demographics_popup_df.groupby('CNTYVTD', as_index=False).agg(agg_funcs_demo_popup)
             print(f"✅ Prepared demographics_popup_df with {len(demographics_popup_df)} unique CNTYVTDs.")
        else:
             print("⚠️ 'VTD Identifier' not found in demographics_df for popup data.")


    except Exception as e:
        print(f"❌ Error processing demographics_df: {e}")
        demographics_geo_df = pd.DataFrame({'CNTYVTD': []}) # Reset to empty on error
        demographics_popup_df = pd.DataFrame({'CNTYVTD': []}) # Reset to empty on error

else:
    print("⚠️ 'VTD Identifier' column not found in demographics_df. Cannot prepare demographic dataframes.")


# 6. Process primary_votes_df - Fixing key selection again
primary_votes_cleaned_df = pd.DataFrame({'CNTYVTD': []}) # Initialize empty
if not primary_votes_df.empty:
    # Debug: Print columns of primary_votes_df before selecting
    print("\nDebug: Columns in primary_votes_df:", primary_votes_df.columns.tolist())

    # Define columns to select, including 'VTD Identifier' and vote columns
    vote_cols_2022_select = ['VTD Identifier', 'Ellzey (R) 2022 Primary Votes', 'Buford (R) 2022 Primary Votes', 'Payne (R) 2022 Primary Votes']
    vote_cols_2024_select = ['VTD Identifier', 'Ellzey (R) 2024 Primary Votes', 'Buford (R) 2024 Primary Votes', 'Wiley (R) 2024 Primary Votes']

    # Select columns, only keeping those that exist
    votes_2022_df = primary_votes_df[[col for col in vote_cols_2022_select if col in primary_votes_df.columns]].copy()
    votes_2024_df = primary_votes_df[[col for col in vote_cols_2024_select if col in primary_votes_df.columns]].copy()

    # Debug: Print columns of votes_2022_df and votes_2024_df after selection
    print("Debug: Columns in votes_2022_df after selection:", votes_2022_df.columns.tolist())
    print("Debug: Columns in votes_2024_df after selection:", votes_2024_df.columns.tolist())

    # Rename 'VTD Identifier' to 'CNTYVTD' in both dataframes
    if 'VTD Identifier' in votes_2022_df.columns:
        votes_2022_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
        votes_2022_df['CNTYVTD'] = votes_2022_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
        print("✅ Renamed and cleaned 'VTD Identifier' to 'CNTYVTD' in votes_2022_df.")
    else:
        print("❌ 'VTD Identifier' not found in 2022 vote data columns. Cannot process 2022 votes.")
        votes_2022_df = pd.DataFrame({'CNTYVTD': []}) # Create empty to prevent merge errors

    if 'VTD Identifier' in votes_2024_df.columns:
        votes_2024_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
        votes_2024_df['CNTYVTD'] = votes_2024_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
        print("✅ Renamed and cleaned 'VTD Identifier' to 'CNTYVTD' in votes_2024_df.")
    else:
        print("❌ 'VTD Identifier' not found in 2024 vote data columns. Cannot process 2024 votes.")
        votes_2024_df = pd.DataFrame({'CNTYVTD': []}) # Create empty to prevent merge errors


    # Merge 2022 and 2024 data only if both have the CNTYVTD key
    if 'CNTYVTD' in votes_2022_df.columns and 'CNTYVTD' in votes_2024_df.columns:
        # Drop vote columns from 2024 that are duplicates of 2022 columns but with different names if they exist
        # (e.g. the Ellzey Votes Change columns) - assuming we calculate our own changes
        cols_to_drop_from_2024 = ['Ellzey Votes Change (2022-2024)', 'Ellzey Votes % Change (2022-2024)']
        votes_2024_df.drop(columns=[col for col in cols_to_drop_from_2024 if col in votes_2024_df.columns], inplace=True, errors='ignore')


        primary_votes_merged = pd.merge(
            votes_2022_df,
            votes_2024_df,
            on='CNTYVTD',
            how='outer'
        )

        # Aggregate potential duplicates
        if not primary_votes_merged.empty and 'CNTYVTD' in primary_votes_merged.columns and primary_votes_merged['CNTYVTD'].duplicated().any():
             print("⚠️ Duplicates found on CNTYVTD after initial primary vote merge. Aggregating (taking first).")
             agg_funcs_votes = {col: 'first' for col in primary_votes_merged.columns if col != 'CNTYVTD'}
             primary_votes_cleaned_df = primary_votes_merged.groupby('CNTYVTD', as_index=False).agg(agg_funcs_votes)
        else:
             primary_votes_cleaned_df = primary_votes_merged.copy()

        # Ensure all vote columns are numeric
        if not primary_votes_cleaned_df.empty:
            vote_cols_to_numeric = [col for col in primary_votes_cleaned_df.columns if col != 'CNTYVTD']
            for col in vote_cols_to_numeric:
                primary_votes_cleaned_df[col] = pd.to_numeric(primary_votes_cleaned_df[col], errors='coerce')
            print("✅ Processed and cleaned primary votes data.")
        else:
             print("⚠️ primary_votes_cleaned_df is empty after processing.")

    else:
        print("❌ Cannot merge 2022 and 2024 primary vote data because 'CNTYVTD' is missing in one or both dataframes.")


# 7. Merge attribute data onto the master GeoDataFrame
if not merge_failed and master_vtd_gdf is not None and not master_vtd_gdf.empty:
    print("\n--- Merging Attribute Data onto Master GeoDataFrame ---")

    # 7a. Merge demographics_geo_df
    if not demographics_geo_df.empty:
        print("Merging demographics_geo_df...")
        # Identify geo columns to merge from demographics_geo_df
        geo_cols_to_merge = [col for col in demographics_geo_df.columns if col != 'CNTYVTD']
        # Drop potential duplicate columns in master_vtd_gdf before merging
        master_vtd_gdf.drop(columns=[col for col in geo_cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
        master_vtd_gdf = master_vtd_gdf.merge(demographics_geo_df, on='CNTYVTD', how='left')
        print(f"✅ Merged demographics geo info. Rows after merge: {len(master_vtd_gdf)}")
    else:
        print("⚠️ demographics_geo_df is empty. Skipping merge.")
        # Add placeholder columns if merge is skipped
        geo_cols_placeholder = ['Best Matching Precinct Name (Max Overlap)', 'County Name (from TORV)', 'Cities (from TORV)']
        for col in geo_cols_placeholder:
            if col not in master_vtd_gdf.columns:
                master_vtd_gdf[col] = np.nan # Use NaN for missing data


    # 7b. Merge cleaned_torv_df
    if not cleaned_torv_df.empty:
        print("Merging cleaned_torv_df...")
        # Identify TORV columns to merge, excluding the geo columns already handled
        torv_cols_to_merge = [col for col in cleaned_torv_df.columns if col not in ['CNTYVTD', 'Precinct', 'County', 'Intersecting Cities', 'Best Matching Precinct Name (Max Overlap)', 'County Name (from TORV)', 'Cities (from TORV)']]
        # Drop potential duplicate columns in master_vtd_gdf before merging TORV data
        master_vtd_gdf.drop(columns=[col for col in torv_cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')

        # Select only the relevant columns from cleaned_torv_df for the merge
        cols_from_torv = ['CNTYVTD'] + torv_cols_to_merge
        master_vtd_gdf = master_vtd_gdf.merge(cleaned_torv_df[cols_from_torv], on='CNTYVTD', how='left')
        print(f"✅ Merged cleaned TORV data. Rows after merge: {len(master_vtd_gdf)}")
    else:
        print("⚠️ cleaned_torv_df is empty. Skipping merge.")
        # Add placeholder columns for key TORV metrics if merge is skipped
        torv_placeholder_cols = [col for col in cleaned_torv_df.columns if col != 'CNTYVTD'] # Use all original TORV cols except key
        for col in torv_placeholder_cols:
            if col not in master_vtd_gdf.columns:
                master_vtd_gdf[col] = np.nan


    # 7c. Merge primary_votes_cleaned_df
    if not primary_votes_cleaned_df.empty:
        print("Merging primary_votes_cleaned_df...")
        # Identify vote columns to merge
        vote_cols_to_merge = [col for col in primary_votes_cleaned_df.columns if col != 'CNTYVTD']
         # Drop potential duplicate columns in master_vtd_gdf before merging primary votes
        master_vtd_gdf.drop(columns=[col for col in vote_cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
        master_vtd_gdf = master_vtd_gdf.merge(primary_votes_cleaned_df, on='CNTYVTD', how='left')
        print(f"✅ Merged primary votes data. Rows after merge: {len(master_vtd_gdf)}")
    else:
        print("⚠️ primary_votes_cleaned_df is empty. Skipping merge.")
        # Add placeholder columns for key primary vote metrics if merge is skipped
        # Use the expected final column names based on how they appear in the merged data
        vote_placeholder_cols = [
            'Ellzey (R) 2022 Primary Votes', 'Buford (R) 2022 Primary Votes', 'Payne (R) 2022 Primary Votes',
            'Ellzey (R) 2024 Primary Votes', 'Buford (R) 2024 Primary Votes', 'Wiley (R) 2024 Primary Votes'
        ]
        for col in vote_placeholder_cols:
             if col not in master_vtd_gdf.columns:
                  master_vtd_gdf[col] = np.nan


    # 7d. Calculate Vote Changes and Percentages (only if necessary columns exist)
    print("\n--- Calculating Vote Changes and Percentages ---")
    # Use the actual column names after merging
    vote_cols_for_calc = {
        'BufordR': {'2022': 'Buford (R) 2022 Primary Votes', '2024': 'Buford (R) 2024 Primary Votes'},
        'EllzeyR': {'2022': 'Ellzey (R) 2022 Primary Votes', '2024': 'Ellzey (R) 2024 Primary Votes'},
        'PayneR': {'2022': 'Payne (R) 2022 Primary Votes', '2024': None}, # Payne only in 2022
        'WileyR': {'2022': None, '2024': 'Wiley (R) 2024 Primary Votes'} # Wiley only in 2024
    }

    for candidate, years in vote_cols_for_calc.items():
        col_2022 = years['2022']
        col_2024 = years['2024']

        # Ensure columns exist and are numeric
        col_2022_exists = col_2022 in master_vtd_gdf.columns
        col_2024_exists = col_2024 in master_vtd_gdf.columns

        if col_2022_exists:
             master_vtd_gdf[col_2022] = pd.to_numeric(master_vtd_gdf[col_2022], errors='coerce').fillna(0)
        if col_2024_exists:
             master_vtd_gdf[col_2024] = pd.to_numeric(master_vtd_gdf[col_2024], errors='coerce').fillna(0)


        if col_2022_exists and col_2024_exists:
            # Calculate Amount Change
            amount_change_col = f'{candidate}_Amount_Change_2022_2024'
            master_vtd_gdf[amount_change_col] = master_vtd_gdf[col_2024] - master_vtd_gdf[col_2022]
            print(f"✅ Calculated '{amount_change_col}'.")

            # Calculate Percentage Change
            pct_change_col = f'{candidate}_Pct_Change_2022_2024'
            denominator = master_vtd_gdf[col_2022].replace(0, np.nan)
            master_vtd_gdf[pct_change_col] = ((master_vtd_gdf[col_2024] - master_vtd_gdf[col_2022]) / denominator) * 100
            master_vtd_gdf[pct_change_col].replace([np.inf, -np.inf], np.nan, inplace=True)
            print(f"✅ Calculated '{pct_change_col}'.")
        elif col_2022_exists and col_2024 is None:
             print(f"⚠️ Only 2022 data available for {candidate}. Skipping 2022-2024 change calculations.")
             master_vtd_gdf[f'{candidate}_Amount_Change_2022_2024'] = np.nan
             master_vtd_gdf[f'{candidate}_Pct_Change_2022_2024'] = np.nan
        elif col_2022 is None and col_2024_exists:
             print(f"⚠️ Only 2024 data available for {candidate}. Skipping 2022-2024 change calculations.")
             master_vtd_gdf[f'{candidate}_Amount_Change_2022_2024'] = np.nan
             master_vtd_gdf[f'{candidate}_Pct_Change_2022_2024'] = np.nan
        else:
             print(f"❌ Necessary vote columns for {candidate} not found for change calculations. Skipping.")
             # Add placeholder columns for changes if calculation was skipped
             master_vtd_gdf[f'{candidate}_Amount_Change_2022_2024'] = np.nan
             master_vtd_gdf[f'{candidate}_Pct_Change_2022_2024'] = np.nan


    # 7e. Calculate Demographic Changes and Percentages (only if necessary columns exist)
    print("\n--- Calculating Demographic Changes and Percentages ---")
    demographic_years = ['2020', '2022', '2024']
    demographic_categories = [
        'Total Population', 'Anglo Population', 'Non-Anglo Population',
        'Asian Population', 'Black Population', 'Hispanic Population', 'Black + Hispanic Population',
        'Voting Age Population', 'Anglo VAP', 'Non-Anglo VAP',
        'Asian VAP', 'Black VAP', 'Hispanic VAP', 'Black + Hispanic VAP'
    ]

    for category in demographic_categories:
        col_2020 = f'{category} ({demographic_years[0]})'
        col_2022 = f'{category} ({demographic_years[1]})'
        col_2024 = f'{category} ({demographic_years[2]})'

        # Ensure columns exist and are numeric
        col_2020_exists = col_2020 in master_vtd_gdf.columns
        col_2022_exists = col_2022 in master_vtd_gdf.columns
        col_2024_exists = col_2024 in master_vtd_gdf.columns

        # Calculate 2020-2022 Changes
        if col_2020_exists and col_2022_exists:
            change_col_2020_2022 = f'{category} Change 2020-2022'
            pct_change_col_2020_2022 = f'{category} % Change 2020-2022'

            # Ensure columns are numeric before calculation
            master_vtd_gdf[col_2020] = pd.to_numeric(master_vtd_gdf[col_2020], errors='coerce')
            master_vtd_gdf[col_2022] = pd.to_numeric(master_vtd_gdf[col_2022], errors='coerce')


            master_vtd_gdf[change_col_2020_2022] = master_vtd_gdf[col_2022] - master_vtd_gdf[col_2020]
            denominator = master_vtd_gdf[col_2020].replace(0, np.nan)
            master_vtd_gdf[pct_change_col_2020_2022] = ((master_vtd_gdf[col_2022] - master_vtd_gdf[col_2020]) / denominator) * 100
            master_vtd_gdf[pct_change_col_2020_2022].replace([np.inf, -np.inf], np.nan, inplace=True)
            print(f"✅ Calculated '{change_col_2020_2022}' and '{pct_change_col_2020_2022}'.")
        else:
            print(f"❌ Missing columns for {category} Change 2020-2022. Skipping calculation.")
            master_vtd_gdf[f'{category} Change 2020-2022'] = np.nan
            master_vtd_gdf[f'{category} % Change 2020-2022'] = np.nan


        # Calculate 2022-2024 Changes
        if col_2022_exists and col_2024_exists:
            change_col_2022_2024 = f'{category} Change 2022-2024'
            pct_change_col_2022_2024 = f'{category} % Change 2022-2024'

            # Ensure columns are numeric before calculation
            master_vtd_gdf[col_2022] = pd.to_numeric(master_vtd_gdf[col_2022], errors='coerce')
            master_vtd_gdf[col_2024] = pd.to_numeric(master_vtd_gdf[col_2024], errors='coerce')

            master_vtd_gdf[change_col_2022_2024] = master_vtd_gdf[col_2024] - master_vtd_gdf[col_2022]
            denominator = master_vtd_gdf[col_2022].replace(0, np.nan)
            master_vtd_gdf[pct_change_col_2022_2024] = ((master_vtd_gdf[col_2024] - master_vtd_gdf[col_2022]) / denominator) * 100
            master_vtd_gdf[pct_change_col_2022_2024].replace([np.inf, -np.inf], np.nan, inplace=True)
            print(f"✅ Calculated '{change_col_2022_2024}' and '{pct_change_col_2022_2024}'.")
        else:
            print(f"❌ Missing columns for {category} Change 2022-2024. Skipping calculation.")
            master_vtd_gdf[f'{category} Change 2022-2024'] = np.nan
            master_vtd_gdf[f'{category} % Change 2022-2024'] = np.nan


    # 8. Print the shape, head, and column names of the final master_vtd_gdf
    print("\n--- Final Merged GeoDataFrame Inspection ---")
    print(f"Merged GeoDataFrame shape: {master_vtd_gdf.shape}")
    print("\nHead of merged master_vtd_gdf:")
    display(master_vtd_gdf.head())
    print("\nColumns of merged master_vtd_gdf:")
    print(master_vtd_gdf.columns.tolist())

else:
    print("\n❌ Merge process skipped due to missing or invalid base GeoDataFrame.")

**Reasoning**:
The previous step successfully merged the data and calculated vote and demographic changes. The `master_vtd_gdf` now contains all the necessary attribute data. The next step is to prepare the data for the tooltip by selecting and formatting the County, Precinct, and Cities columns. This corresponds to step 4 from the overall task plan, but is broken down into a subtask for clarity. However, demogrpahic info is not loading correctly.




In [ ]:
import pandas as pd # Ensure pandas is imported if not already

# 1. Check if the master_vtd_gdf GeoDataFrame exists and is not empty.
if 'master_vtd_gdf' not in locals() or master_vtd_gdf is None or master_vtd_gdf.empty:
    print("❌ master_vtd_gdf GeoDataFrame is not available or empty. Cannot prepare tooltip columns.")
    # No further steps can be completed without the master GeoDataFrame.
else:
    print("✅ master_vtd_gdf GeoDataFrame is available for tooltip preparation.")

    # 2. Define a list of column names expected for the tooltip.
    # Use the exact column names resulting from the previous merge steps.
    tooltip_cols_raw = ['County Name (from TORV)', 'Best Matching Precinct Name (Max Overlap)', 'Cities (from TORV)']

    # Map the raw column names to the desired display names for the tooltip
    tooltip_col_map = {
        'County Name (from TORV)': 'County',
        'Best Matching Precinct Name (Max Overlap)': 'Precinct',
        'Cities (from TORV)': 'Cities'
    }

    # Ensure the target columns for the tooltip map exist in the master_vtd_gdf
    # Create placeholder columns if they don't exist, then apply the rename
    for raw_col, display_col in tooltip_col_map.items():
        if raw_col not in master_vtd_gdf.columns:
            print(f"⚠️ Tooltip column '{raw_col}' not found. Creating a placeholder column '{display_col}'.")
            master_vtd_gdf[display_col] = 'N/A' # Create with target display name directly
        else:
            # If the column exists, prepare it (convert to string and fill NaNs)
            master_vtd_gdf[display_col] = master_vtd_gdf[raw_col].astype(str).fillna('N/A')
            # If the display column name is different from the raw, we rename later or just use the display name as the final column name


    # Select the final columns to be used in the tooltip (using display names)
    final_tooltip_cols = list(tooltip_col_map.values())
    # Ensure the final columns are in the GeoDataFrame
    for col in final_tooltip_cols:
        if col not in master_vtd_gdf.columns:
             # This shouldn't happen if the logic above worked, but as a safeguard
             master_vtd_gdf[col] = 'N/A' # Create if somehow missed


    # 6. Create a new column in master_vtd_gdf named 'tooltip_text'.
    master_vtd_gdf['tooltip_text'] = master_vtd_gdf.apply(
        lambda row: f"""
<b>County:</b> {row.get('County', 'N/A')}<br>
<b>Precinct:</b> {row.get('Precinct', 'N/A')}<br>
<b>Cities:</b> {row.get('Cities', 'N/A')}
        """,
        axis=1
    )
    print("✅ Created 'tooltip_text' column for VTD hover tooltips.")

    # 7. Print the head of master_vtd_gdf showing the original and newly created tooltip columns.
    print("\n📋 Head of master_vtd_gdf with tooltip columns:")
    # Display the original raw columns if they exist, and the final display columns and tooltip_text
    display_cols = [col for col in tooltip_cols_raw + final_tooltip_cols + ['tooltip_text'] if col in master_vtd_gdf.columns]
    display(master_vtd_gdf[display_cols].head())

    print("\nColumns of master_vtd_gdf after tooltip preparation:")
    print(master_vtd_gdf.columns.tolist())

**Reasoning**:
The tooltip data has been prepared. The next step is to prepare the demographic data for the popup dropdown by selecting the relevant demographic columns from `master_vtd_gdf` and formatting them into an HTML snippet, including calculated changes. This corresponds to part of step 4 from the overall task plan.



In [ ]:
import pandas as pd
import numpy as np # Ensure numpy is imported for NaN handling

# Ensure master_vtd_gdf is available
if 'master_vtd_gdf' not in locals() or master_vtd_gdf is None or master_vtd_gdf.empty:
    print("❌ master_vtd_gdf GeoDataFrame is not available or empty. Cannot prepare demographic data for popup.")
else:
    print("✅ master_vtd_gdf is available for demographic popup data preparation.")

    # Define demographic columns present in master_vtd_gdf after merging
    demographic_cols_present = [
        'Total Population (2020)', 'Total Population (2022)', 'Total Population (2024)',
        'Anglo Population (2020)', 'Anglo Population (2022)', 'Anglo Population (2024)',
        'Non-Anglo Population (2020)', 'Non-Anglo Population (2022)', 'Non-Anglo Population (2024)',
        'Asian Population (2020)', 'Asian Population (2022)', 'Asian Population (2024)',
        'Black Population (2020)', 'Black Population (2022)', 'Black Population (2024)',
        'Hispanic Population (2020)', 'Hispanic Population (2022)', 'Hispanic Population (2024)',
        'Black + Hispanic Population (2020)', 'Black + Hispanic Population (2022)', 'Black + Hispanic Population (2024)',
        'Voting Age Population (2020)', 'Voting Age Population (2022)', 'Voting Age Population (2024)',
        'Anglo VAP (2020)', 'Anglo VAP (2022)', 'Anglo VAP (2024)',
        'Non-Anglo VAP (2020)', 'Non-Anglo VAP (2022)', 'Non-Anglo VAP (2024)',
        'Asian VAP (2020)', 'Asian VAP (2022)', 'Asian VAP (2024)',
        'Black VAP (2020)', 'Black VAP (2022)', 'Black VAP (2024)',
        'Hispanic VAP (2020)', 'Hispanic VAP (2022)', 'Hispanic VAP (2024)',
        'Black + Hispanic VAP (2020)', 'Black + Hispanic VAP (2022)', 'Black + Hispanic VAP (2024)',
        'Total Population Change 2020-2022', 'Total Population % Change 2020-2022',
        'Total Population Change 2022-2024', 'Total Population % Change 2022-2024',
        'Anglo Population Change 2020-2022', 'Anglo Population % Change 2020-2022',
        'Anglo Population Change 2022-2024', 'Anglo Population % Change 2022-2024',
        'Non-Anglo Population Change 2020-2022', 'Non-Anglo Population % Change 2020-2022',
        'Non-Anglo Population Change 2022-2024', 'Non-Anglo Population % Change 2022-2024',
        'Asian Population Change 2020-2022', 'Asian Population % Change 2020-2022',
        'Asian Population Change 2022-2024', 'Asian Population % Change 2022-2024',
        'Black Population Change 2020-2022', 'Black Population % Change 2020-2022',
        'Black Population Change 2022-2024', 'Black Population % Change 2022-2024',
        'Hispanic Population Change 2020-2022', 'Hispanic Population % Change 2020-2022',
        'Hispanic Population Change 2022-2024', 'Hispanic Population % Change 2022-2024',
        'Black + Hispanic Population Change 2020-2022', 'Black + Hispanic Population % Change 2020-2022',
        'Black + Hispanic Population Change 2022-2024', 'Black + Hispanic Population % Change 2022-2024',
        'Voting Age Population Change 2020-2022', 'Voting Age Population % Change 2020-2022',
        'Voting Age Population Change 2022-2024', 'Voting Age Population % Change 2022-2024',
        'Anglo VAP Change 2020-2022', 'Anglo VAP % Change 2020-2022',
        'Anglo VAP Change 2022-2024', 'Anglo VAP % Change 2022-2024',
        'Non-Anglo VAP Change 2020-2022', 'Non-Anglo VAP % Change 2020-2022',
        'Non-Anglo VAP Change 2022-2024', 'Non-Anglo VAP % Change 2022-2024',
        'Asian VAP Change 2020-2022', 'Asian VAP % Change 2020-2022',
        'Asian VAP Change 2022-2024', 'Asian VAP % Change 2022-2024',
        'Black VAP Change 2020-2022', 'Black VAP % Change 2020-2022',
        'Black VAP Change 2022-2024', 'Black VAP % Change 2022-2024',
        'Hispanic VAP Change 2020-2022', 'Hispanic VAP % Change 2020-2022',
        'Hispanic VAP Change 2022-2024', 'Hispanic VAP % Change 2022-2024',
        'Black + Hispanic VAP Change 2020-2022', 'Black + Hispanic VAP % Change 2020-2022',
        'Black + Hispanic VAP Change 2022-2024', 'Black + Hispanic VAP % Change 2022-2024'
    ]

    # Define a helper function to format numeric values and percentages, handling NaNs
    def format_value_html(value, is_percentage=False):
        if pd.notnull(value):
            try:
                if is_percentage:
                     # Format percentages to one decimal place with % sign
                    return f"{float(value):.1f}%"
                else:
                    # Format numeric values with commas
                    return f"{int(float(value)):,}"
            except (ValueError, TypeError):
                # Handle cases where conversion to float/int fails (e.g., non-numeric strings)
                return str(value)
        return 'N/A'

    # Create the demographic HTML snippet for each VTD
    master_vtd_gdf['demographics_html'] = master_vtd_gdf.apply(lambda row: f"""
    <b>Demographic Data:</b><br>
    <table style="width:100%; border-collapse: collapse; border: 1px solid black;">
      <thead>
        <tr style="border-bottom: 1px solid black;">
          <th>Category</th>
          <th>2020</th>
          <th>2022</th>
          <th>2024</th>
          <th>Change (20-22)</th>
          <th>% Change (20-22)</th>
          <th>Change (22-24)</th>
          <th>% Change (22-24)</th>
        </tr>
      </thead>
      <tbody>
        <tr>
          <td>Total Pop</td>
          <td>{format_value_html(row.get('Total Population (2020)'))}</td>
          <td>{format_value_html(row.get('Total Population (2022)'))}</td>
          <td>{format_value_html(row.get('Total Population (2024)'))}</td>
          <td>{format_value_html(row.get('Total Population Change 2020-2022'))}</td>
          <td>{format_value_html(row.get('Total Population % Change 2020-2022'), is_percentage=True)}</td>
          <td>{format_value_html(row.get('Total Population Change 2022-2024'))}</td>
          <td>{format_value_html(row.get('Total Population % Change 2022-2024'), is_percentage=True)}</td>
        </tr>
        <tr>
          <td>Anglo Pop</td>
          <td>{format_value_html(row.get('Anglo Population (2020)'))}</td>
          <td>{format_value_html(row.get('Anglo Population (2022)'))}</td>
          <td>{format_value_html(row.get('Anglo Population (2024)'))}</td>
          <td>{format_value_html(row.get('Anglo Population Change 2020-2022'))}</td>
          <td>{format_value_html(row.get('Anglo Population % Change 2020-2022'), is_percentage=True)}</td>
          <td>{format_value_html(row.get('Anglo Population Change 2022-2024'))}</td>
          <td>{format_value_html(row.get('Anglo Population % Change 2022-2024'), is_percentage=True)}</td>
        </tr>
         <tr>
          <td>Non-Anglo Pop</td>
          <td>{format_value_html(row.get('Non-Anglo Population (2020)'))}</td>
          <td>{format_value_html(row.get('Non-Anglo Population (2022)'))}</td>
          <td>{format_value_html(row.get('Non-Anglo Population (2024)'))}</td>
          <td>{format_value_html(row.get('Non-Anglo Population Change 2020-2022'))}</td>
          <td>{format_value_html(row.get('Non-Anglo Population % Change 2020-2022'), is_percentage=True)}</td>
          <td>{format_value_html(row.get('Non-Anglo Population Change 2022-2024'))}</td>
          <td>{format_value_html(row.get('Non-Anglo Population % Change 2022-2024'), is_percentage=True)}</td>
        </tr>
        <tr>
          <td>Asian Pop</td>
          <td>{format_value_html(row.get('Asian Population (2020)'))}</td>
          <td>{format_value_html(row.get('Asian Population (2022)'))}</td>
          <td>{format_value_html(row.get('Asian Population (2024)'))}</td>
          <td>{format_value_html(row.get('Asian Population Change 2020-2022'))}</td>
          <td>{format_value_html(row.get('Asian Population % Change 2020-2022'), is_percentage=True)}</td>
          <td>{format_value_html(row.get('Asian Population Change 2022-2024'))}</td>
          <td>{format_value_html(row.get('Asian Population % Change 2022-2024'), is_percentage=True)}</td>
        </tr>
        <tr>
          <td>Black Pop</td>
          <td>{format_value_html(row.get('Black Population (2020)'))}</td>
          <td>{format_value_html(row.get('Black Population (2022)'))}</td>
          <td>{format_value_html(row.get('Black Population (2024)'))}</td>
          <td>{format_value_html(row.get('Black Population Change 2020-2022'))}</td>
          <td>{format_value_html(row.get('Black Population % Change 2020-2022'), is_percentage=True)}</td>
          <td>{format_value_html(row.get('Black Population Change 2022-2024'))}</td>
          <td>{format_value_html(row.get('Black Population % Change 2022-2024'), is_percentage=True)}</td>
        </tr>
         <tr>
          <td>Hispanic Pop</td>
          <td>{format_value_html(row.get('Hispanic Population (2020)'))}</td>
          <td>{format_value_html(row.get('Hispanic Population (2022)'))}</td>
          <td>{format_value_html(row.get('Hispanic Population (2024)'))}</td>
          <td>{format_value_html(row.get('Hispanic Population Change 2020-2022'))}</td>
          <td>{format_value_html(row.get('Hispanic Population % Change 2020-2022'), is_percentage=True)}</td>
          <td>{format_value_html(row.get('Hispanic Population Change 2022-2024'))}</td>
          <td>{format_value_html(row.get('Hispanic Population % Change 2022-2024'), is_percentage=True)}</td>
        </tr>
         <tr>
          <td>Black + Hispanic Pop</td>
          <td>{format_value_html(row.get('Black + Hispanic Population (2020)'))}</td>
          <td>{format_value_html(row.get('Black + Hispanic Population (2022)'))}</td>
          <td>{format_value_html(row.get('Black + Hispanic Population (2024)'))}</td>
          <td>{format_value_html(row.get('Black + Hispanic Population Change 2020-2022'))}</td>
          <td>{format_value_html(row.get('Black + Hispanic Population % Change 2020-2022'), is_percentage=True)}</td>
          <td>{format_value_html(row.get('Black + Hispanic Population Change 2022-2024'))}</td>
          <td>{format_value_html(row.get('Black + Hispanic Population % Change 2022-2024'), is_percentage=True)}</td>
        </tr>
         <tr>
          <td>VAP</td>
          <td>{format_value_html(row.get('Voting Age Population (2020)'))}</td>
          <td>{format_value_html(row.get('Voting Age Population (2022)'))}</td>
          <td>{format_value_html(row.get('Voting Age Population (2024)'))}</td>
           <td>{format_value_html(row.get('Voting Age Population Change 2020-2022'))}</td>
          <td>{format_value_html(row.get('Voting Age Population % Change 2020-2022'), is_percentage=True)}</td>
           <td>{format_value_html(row.get('Voting Age Population Change 2022-2024'))}</td>
          <td>{format_value_html(row.get('Voting Age Population % Change 2022-2024'), is_percentage=True)}</td>
        </tr>
         <tr>
          <td>Anglo VAP</td>
          <td>{format_value_html(row.get('Anglo VAP (2020)'))}</td>
          <td>{format_value_html(row.get('Anglo VAP (2022)'))}</td>
          <td>{format_value_html(row.get('Anglo VAP (2024)'))}</td>
           <td>{format_value_html(row.get('Anglo VAP Change 2020-2022'))}</td>
          <td>{format_value_html(row.get('Anglo VAP % Change 2020-2022'), is_percentage=True)}</td>
           <td>{format_value_html(row.get('Anglo VAP Change 2022-2024'))}</td>
          <td>{format_value_html(row.get('Anglo VAP % Change 2022-2024'), is_percentage=True)}</td>
        </tr>
        <tr>
          <td>Non-Anglo VAP</td>
          <td>{format_value_html(row.get('Non-Anglo VAP (2020)'))}</td>
          <td>{format_value_html(row.get('Non-Anglo VAP (2022)'))}</td>
          <td>{format_value_html(row.get('Non-Anglo VAP (2024)'))}</td>
          <td>{format_value_html(row.get('Non-Anglo VAP Change 2020-2022'))}</td>
          <td>{format_value_html(row.get('Non-Anglo VAP % Change 2020-2022'), is_percentage=True)}</td>
          <td>{format_value_html(row.get('Non-Anglo VAP Change 2022-2024'))}</td>
          <td>{format_value_html(row.get('Non-Anglo VAP % Change 2022-2024'), is_percentage=True)}</td>
        </tr>
        <tr>
          <td>Asian VAP</td>
          <td>{format_value_html(row.get('Asian VAP (2020)'))}</td>
          <td>{format_value_html(row.get('Asian VAP (2022)'))}</td>
          <td>{format_value_html(row.get('Asian VAP (2024)'))}</td>
          <td>{format_value_html(row.get('Asian VAP Change 2020-2022'))}</td>
          <td>{format_value_html(row.get('Asian VAP % Change 2020-2022'), is_percentage=True)}</td>
          <td>{format_value_html(row.get('Asian VAP Change 2022-2024'))}</td>
          <td>{format_value_html(row.get('Asian VAP % Change 2022-2024'), is_percentage=True)}</td>
        </tr>
        <tr>
          <td>Black VAP</td>
          <td>{format_value_html(row.get('Black VAP (2020)'))}</td>
          <td>{format_value_html(row.get('Black VAP (2022)'))}</td>
          <td>{format_value_html(row.get('Black VAP (2024)'))}</td>
          <td>{format_value_html(row.get('Black VAP Change 2020-2022'))}</td>
          <td>{format_value_html(row.get('Black VAP % Change 2020-2022'), is_percentage=True)}</td>
          <td>{format_value_html(row.get('Black VAP Change 2022-2024'))}</td>
          <td>{format_value_html(row.get('Black VAP % Change 2022-2024'), is_percentage=True)}</td>
        </tr>
        <tr>
          <td>Hispanic VAP</td>
          <td>{format_value_html(row.get('Hispanic VAP (2020)'))}</td>
          <td>{format_value_html(row.get('Hispanic VAP (2022)'))}</td>
          <td>{format_value_html(row.get('Hispanic VAP (2024)'))}</td>
          <td>{format_value_html(row.get('Hispanic VAP Change 2020-2022'))}</td>
          <td>{format_value_html(row.get('Hispanic VAP % Change 2020-2022'), is_percentage=True)}</td>
          <td>{format_value_html(row.get('Hispanic VAP Change 2022-2024'))}</td>
          <td>{format_value_html(row.get('Hispanic VAP % Change 2022-2024'), is_percentage=True)}</td>
        </tr>
        <tr>
          <td>Black + Hispanic VAP</td>
          <td>{format_value_html(row.get('Black + Hispanic VAP (2020)'))}</td>
          <td>{format_value_html(row.get('Black + Hispanic VAP (2022)'))}</td>
          <td>{format_value_html(row.get('Black + Hispanic VAP (2024)'))}</td>
          <td>{format_value_html(row.get('Black + Hispanic VAP Change 2020-2022'))}</td>
          <td>{format_value_html(row.get('Black + Hispanic VAP % Change 2020-2022'), is_percentage=True)}</td>
          <td>{format_value_html(row.get('Black + Hispanic VAP Change 2022-2024'))}</td>
          <td>{format_value_html(row.get('Black + Hispanic VAP % Change 2022-2024'), is_percentage=True)}</td>
        </tr>
      </tbody>
    </table>
    """, axis=1)

    print("✅ Created 'demographics_html' column for popup.")

    # Print head and columns to verify
    print("\n📋 Head of master_vtd_gdf with 'demographics_html':")
    display(master_vtd_gdf[['CNTYVTD', 'demographics_html']].head())
    print("\nColumns of master_vtd_gdf after demographics popup preparation:")
    print(master_vtd_gdf.columns.tolist())

## Data loading and inspection

### Subtask:
Load all necessary data files (`vtds_cd6.geojson`, `cleaned_torv_data.csv`, `DemographicsFinal.csv`, and `PrimaryVotesFinal.csv`), display their heads and column names.

**Reasoning**:
Load and display the head and columns of the four required data files.

In [ ]:
import pandas as pd
import geopandas as gpd
import os
from IPython.display import display

print("--- Loading and Inspecting Data Files ---")

# 1. Define the file paths
vtds_geojson_path = 'vtds_cd6.geojson'
cleaned_torv_path = 'cleaned_torv_data.csv'
demographics_file_path = "DemographicsFinal.csv"
primary_votes_file_path = "PrimaryVotesFinal.csv" # Corrected file name based on previous output

# 2. For each file:

# a. vtds_cd6.geojson
if os.path.exists(vtds_geojson_path):
    try:
        vtds_cd6_gdf = gpd.read_file(vtds_geojson_path)
        print(f"\n✅ Loaded {vtds_geojson_path}")
        print(f"\n📋 Head of {vtds_geojson_path}:")
        display(vtds_cd6_gdf.head())
        print("\nColumns:")
        print(vtds_cd6_gdf.columns.tolist())
    except Exception as e:
        print(f"❌ Error loading {vtds_geojson_path}: {e}")
        vtds_cd6_gdf = None
else:
    print(f"\n❌ {vtds_geojson_path} not found.")
    vtds_cd6_gdf = None


# b. cleaned_torv_data.csv
if os.path.exists(cleaned_torv_path):
    try:
        cleaned_torv_df = pd.read_csv(cleaned_torv_path)
        print(f"\n✅ Loaded {cleaned_torv_path}")
        print(f"\n📋 Head of {cleaned_torv_path}:")
        display(cleaned_torv_df.head())
        print("\nColumns:")
        print(cleaned_torv_df.columns.tolist())
    except Exception as e:
        print(f"❌ Error loading {cleaned_torv_path}: {e}")
        cleaned_torv_df = None
else:
    print(f"\n❌ {cleaned_torv_path} not found.")
    cleaned_torv_df = None


# c. DemographicsFinal.csv
if os.path.exists(demographics_file_path):
    try:
        demographics_df = pd.read_csv(demographics_file_path)
        print(f"\n✅ Loaded {demographics_file_path}")
        print(f"\n📋 Head of {demographics_file_path}:")
        display(demographics_df.head())
        print("\nColumns:")
        print(demographics_df.columns.tolist())
    except Exception as e:
        print(f"❌ Error loading {demographics_file_path}: {e}")
        demographics_df = None
else:
    print(f"\n❌ {demographics_file_path} not found.")
    demographics_df = None


# d. PrimaryVotesFinal.csv
if os.path.exists(primary_votes_file_path):
    try:
        primary_votes_df = pd.read_csv(primary_votes_file_path)
        print(f"\n✅ Loaded {primary_votes_file_path}")
        print(f"\n📋 Head of {primary_votes_file_path}:")
        display(primary_votes_df.head())
        print("\nColumns:")
        print(primary_votes_df.columns.tolist())
    except Exception as e:
        print(f"❌ Error loading {primary_votes_file_path}: {e}")
        primary_votes_df = None
else:
    print(f"\n❌ {primary_votes_file_path} not found.")
    primary_votes_df = None

print("\n--- Data Loading and Inspection Complete ---")

## Attribute data preparation and merging

### Subtask:
Clean and standardize 'CNTYVTD' in all attribute dataframes. Create `primary_votes_cleaned_df` by processing `PrimaryVotesFinal.csv`. Merge `cleaned_torv_data.csv`, `DemographicsFinal.csv` (for geographic and demographic data), and the cleaned primary votes data onto the base VTD GeoDataFrame using 'CNTYVTD'. Calculate vote changes and percentages.

**Reasoning**:
Clean and standardize CNTYVTD in the loaded dataframes, process primary votes data, and merge the attribute data onto the VTD GeoDataFrame. This combines steps 2 through 7 from the instructions.

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

# 1. Define a consistent projected CRS (e.g., EPSG:3083)
proj_crs = "EPSG:3083" # Using the projected CRS defined in previous steps

print("--- Data Preparation and Merging ---")

# 2. Ensure vtds_cd6_gdf is available and has the cleaned 'CNTYVTD'
master_vtd_gdf = None
merge_failed = False # Flag to track if the base GeoDataFrame is ready for merging

if 'vtds_cd6_gdf' in locals() and vtds_cd6_gdf is not None and not vtds_cd6_gdf.empty:
    print("✅ vtds_cd6_gdf is available.")

    # Create or standardize the 'CNTYVTD' column
    if 'CNTY_x' in vtds_cd6_gdf.columns and 'VTD_x' in vtds_cd6_gdf.columns:
        vtds_cd6_gdf['CNTYVTD'] = vtds_cd6_gdf['CNTY_x'].astype(str).str.strip() + vtds_cd6_gdf['VTD_x'].astype(str).str.strip()
        print("✅ Created 'CNTYVTD' in vtds_cd6_gdf from CNTY_x and VTD_x.")
    elif 'CNTYVTD_x' in vtds_cd6_gdf.columns:
         vtds_cd6_gdf.rename(columns={'CNTYVTD_x': 'CNTYVTD'}, inplace=True)
         print("✅ Using existing 'CNTYVTD_x' as 'CNTYVTD' in vtds_cd6_gdf.")
    elif 'CNTYVTD_y' in vtds_cd6_gdf.columns: # Check for _y if it was merged from pop data
         vtds_cd6_gdf.rename(columns={'CNTYVTD_y': 'CNTYVTD'}, inplace=True)
         print("✅ Using existing 'CNTYVTD_y' as 'CNTYVTD' in vtds_cd6_gdf.")
    elif 'CNTYVTD' in vtds_cd6_gdf.columns:
         print("✅ Using existing 'CNTYVTD' in vtds_cd6_gdf.")
    else:
         print("❌ Could not find a suitable VTD identifier column in vtds_cd6_gdf to create CNTYVTD. Cannot merge attributes.")
         merge_failed = True # Flag merge failure

    if not merge_failed and 'CNTYVTD' in vtds_cd6_gdf.columns:
        # Ensure CNTYVTD is string and left-strip leading zeros
        vtds_cd6_gdf['CNTYVTD'] = vtds_cd6_gdf['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
        print("✅ Cleaned 'CNTYVTD' in vtds_cd6_gdf.")

        # Reproject to the target projected CRS if needed (assuming it might not be in the target_crs after initial load)
        if vtds_cd6_gdf.crs is None:
             print(f"⚠️ vtds_cd6_gdf CRS missing, assuming EPSG:3857 and setting to {proj_crs}")
             vtds_cd6_gdf.set_crs("EPSG:3857", inplace=True) # Assume a common web CRS if missing

        if vtds_cd6_gdf.crs != proj_crs:
             vtds_cd6_gdf_proj = vtds_cd6_gdf.to_crs(proj_crs)
             print(f"✅ Reprojected vtds_cd6_gdf to {proj_crs} for spatial operations.")
        else:
             vtds_cd6_gdf_proj = vtds_cd6_gdf.copy()
             print(f"✅ vtds_cd6_gdf is already in {proj_crs}.")


        # Create the master GeoDataFrame with essential columns and geometry
        master_vtd_gdf = vtds_cd6_gdf_proj[['CNTYVTD', 'geometry']].copy()
        print(f"✅ Started master GeoDataFrame with {len(master_vtd_gdf)} VTDs.")

    else:
         print("❌ CNTYVTD column not available in vtds_cd6_gdf. Cannot create master GeoDataFrame.")
         merge_failed = True

else:
    print("❌ vtds_cd6_gdf is not available or empty. Cannot merge attributes.")
    # Create an empty GeoDataFrame with the expected columns to avoid errors
    master_vtd_gdf = gpd.GeoDataFrame({'CNTYVTD': [], 'geometry': []}, crs=proj_crs)
    merge_failed = True # Flag merge failure


# 3. Ensure attribute dataframes are available
if 'cleaned_torv_df' not in locals() or cleaned_torv_df is None or cleaned_torv_df.empty:
    print("⚠️ cleaned_torv_df not available or empty. TORV data will be missing.")
    cleaned_torv_df = pd.DataFrame({'CNTYVTD': []}) # Create empty df to prevent errors
if 'demographics_df' not in locals() or demographics_df is None or demographics_df.empty:
    print("⚠️ demographics_df not available or empty. Demographic data will be missing.")
    demographics_df = pd.DataFrame({'VTD Identifier': []}) # Create empty df to prevent errors
if 'primary_votes_df' not in locals() or primary_votes_df is None or primary_votes_df.empty:
    print("⚠️ primary_votes_df not available or empty. Primary vote data will be missing.")
    primary_votes_df = pd.DataFrame({'VTD Identifier': [], 'CNTYVTD': [], 'CNTYVTD.1': []}) # Create empty df with expected key columns


# 4. Clean and standardize 'CNTYVTD' in cleaned_torv_df
if 'CNTYVTD' in cleaned_torv_df.columns:
    cleaned_torv_df['CNTYVTD'] = cleaned_torv_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
    print("✅ Cleaned 'CNTYVTD' in cleaned_torv_df.")
else:
    print("⚠️ 'CNTYVTD' column not found in cleaned_torv_df. Skipping CNTYVTD cleaning.")


# 5. Clean and select columns for demographics_geo_df and demographics_popup_df
demographics_geo_df = pd.DataFrame({'CNTYVTD': []}) # Initialize empty
demographics_popup_df = pd.DataFrame({'CNTYVTD': []}) # Initialize empty

if 'VTD Identifier' in demographics_df.columns:
    try:
        # Prepare demographics_geo_df
        geo_cols_to_select = ['VTD Identifier', 'Best Matching Precinct Name (Max Overlap)', 'County Name (from TORV)', 'Cities (from TORV)']
        geo_cols_present = [col for col in geo_cols_to_select if col in demographics_df.columns]
        if len(geo_cols_present) > 1: # Need at least VTD Identifier and one other geo column
            demographics_geo_df = demographics_df[geo_cols_present].copy()
            demographics_geo_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
            demographics_geo_df['CNTYVTD'] = demographics_geo_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
            if not demographics_geo_df['CNTYVTD'].is_unique:
                print("⚠️ Duplicate CNTYVTDs in demographics geo data. Aggregating (taking first).")
                agg_funcs_geo = {col: 'first' for col in demographics_geo_df.columns if col != 'CNTYVTD'}
                demographics_geo_df = demographics_geo_df.groupby('CNTYVTD', as_index=False).agg(agg_funcs_geo)
            print(f"✅ Prepared demographics_geo_df with {len(demographics_geo_df)} unique CNTYVTDs.")
        else:
            print("⚠️ Not enough geographic columns found in demographics_df for demographics_geo_df.")


        # Prepare demographics_popup_df (all demographic columns)
        demo_cols_to_select_popup = [col for col in demographics_df.columns if col != 'VTD Identifier']
        if 'VTD Identifier' in demographics_df.columns: # Ensure 'VTD Identifier' exists before selecting
             demographics_popup_df = demographics_df[['VTD Identifier'] + demo_cols_to_select_popup].copy()
             demographics_popup_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
             demographics_popup_df['CNTYVTD'] = demographics_popup_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
             if not demographics_popup_df['CNTYVTD'].is_unique:
                  print("⚠️ Duplicate CNTYVTDs in demographics popup data. Aggregating (taking first).")
                  agg_funcs_demo_popup = {col: 'first' for col in demographics_popup_df.columns if col != 'CNTYVTD'}
                  demographics_popup_df = demographics_popup_df.groupby('CNTYVTD', as_index=False).agg(agg_funcs_demo_popup)
             print(f"✅ Prepared demographics_popup_df with {len(demographics_popup_df)} unique CNTYVTDs.")
        else:
             print("⚠️ 'VTD Identifier' not found in demographics_df for popup data.")


    except Exception as e:
        print(f"❌ Error processing demographics_df: {e}")
        demographics_geo_df = pd.DataFrame({'CNTYVTD': []}) # Reset to empty on error
        demographics_popup_df = pd.DataFrame({'CNTYVTD': []}) # Reset to empty on error

else:
    print("⚠️ 'VTD Identifier' column not found in demographics_df. Cannot prepare demographic dataframes.")


# 6. Process primary_votes_df - FIXING KEY SELECTION again
primary_votes_cleaned_df = pd.DataFrame({'CNTYVTD': []}) # Initialize empty
if not primary_votes_df.empty:
    # Debug: Print columns of primary_votes_df before selecting
    print("\nDebug: Columns in primary_votes_df:", primary_votes_df.columns.tolist())

    # Define columns to select, including 'VTD Identifier' and vote columns
    # Using the actual column names from the file inspection
    vote_cols_2022_select = ['VTD Identifier', 'Ellzey (R) 2022 Primary Votes', 'Buford (R) 2022 Primary Votes', 'Payne (R) 2022 Primary Votes']
    vote_cols_2024_select = ['VTD Identifier', 'Ellzey (R) 2024 Primary Votes', 'Buford (R) 2024 Primary Votes', 'Wiley (R) 2024 Primary Votes']

    # Select columns, only keeping those that exist
    votes_2022_df = primary_votes_df[[col for col in vote_cols_2022_select if col in primary_votes_df.columns]].copy()
    votes_2024_df = primary_votes_df[[col for col in vote_cols_2024_select if col in primary_votes_df.columns]].copy()

    # Debug: Print columns of votes_2022_df and votes_2024_df after selection
    print("Debug: Columns in votes_2022_df after selection:", votes_2022_df.columns.tolist())
    print("Debug: Columns in votes_2024_df after selection:", votes_2024_df.columns.tolist())

    # Rename 'VTD Identifier' to 'CNTYVTD' in both dataframes
    if 'VTD Identifier' in votes_2022_df.columns:
        votes_2022_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
        votes_2022_df['CNTYVTD'] = votes_2022_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
        print("✅ Renamed and cleaned 'VTD Identifier' to 'CNTYVTD' in votes_2022_df.")
    else:
        print("❌ 'VTD Identifier' not found in 2022 vote data columns. Cannot process 2022 votes.")
        votes_2022_df = pd.DataFrame({'CNTYVTD': []}) # Create empty to prevent merge errors

    if 'VTD Identifier' in votes_2024_df.columns:
        votes_2024_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
        votes_2024_df['CNTYVTD'] = votes_2024_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
        print("✅ Renamed and cleaned 'VTD Identifier' to 'CNTYVTD' in votes_2024_df.")
    else:
        print("❌ 'VTD Identifier' not found in 2024 vote data columns. Cannot process 2024 votes.")
        votes_2024_df = pd.DataFrame({'CNTYVTD': []}) # Create empty to prevent merge errors


    # Merge 2022 and 2024 data only if both have the CNTYVTD key
    if 'CNTYVTD' in votes_2022_df.columns and 'CNTYVTD' in votes_2024_df.columns:
        # Drop vote columns from 2024 that are duplicates of 2022 columns but with different names if they exist
        # (e.g. the Ellzey Votes Change columns) - assuming we calculate our own changes
        cols_to_drop_from_2024 = ['Ellzey Votes Change (2022-2024)', 'Ellzey Votes % Change (2022-2024)']
        votes_2024_df.drop(columns=[col for col in cols_to_drop_from_2024 if col in votes_2024_df.columns], inplace=True, errors='ignore')


        primary_votes_merged = pd.merge(
            votes_2022_df,
            votes_2024_df,
            on='CNTYVTD',
            how='outer'
        )

        # Aggregate potential duplicates
        if not primary_votes_merged.empty and 'CNTYVTD' in primary_votes_merged.columns and primary_votes_merged['CNTYVTD'].duplicated().any():
             print("⚠️ Duplicates found on CNTYVTD after initial primary vote merge. Aggregating (taking first).")
             agg_funcs_votes = {col: 'first' for col in primary_votes_merged.columns if col != 'CNTYVTD'}
             primary_votes_cleaned_df = primary_votes_merged.groupby('CNTYVTD', as_index=False).agg(agg_funcs_votes)
        else:
             primary_votes_cleaned_df = primary_votes_merged.copy()

        # Ensure all vote columns are numeric
        if not primary_votes_cleaned_df.empty:
            vote_cols_to_numeric = [col for col in primary_votes_cleaned_df.columns if col != 'CNTYVTD']
            for col in vote_cols_to_numeric:
                primary_votes_cleaned_df[col] = pd.to_numeric(primary_votes_cleaned_df[col], errors='coerce')
            print("✅ Processed and cleaned primary votes data.")
        else:
             print("⚠️ primary_votes_cleaned_df is empty after processing.")

    else:
        print("❌ Cannot merge 2022 and 2024 primary vote data because 'CNTYVTD' is missing in one or both dataframes.")


# 7. Merge attribute data onto the master GeoDataFrame
if not merge_failed and master_vtd_gdf is not None and not master_vtd_gdf.empty:
    print("\n--- Merging Attribute Data onto Master GeoDataFrame ---")

    # 7a. Merge demographics_geo_df
    if not demographics_geo_df.empty:
        print("Merging demographics_geo_df...")
        # Identify geo columns to merge from demographics_geo_df
        geo_cols_to_merge = [col for col in demographics_geo_df.columns if col != 'CNTYVTD']
        # Drop potential duplicate columns in master_vtd_gdf before merging
        master_vtd_gdf.drop(columns=[col for col in geo_cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
        master_vtd_gdf = master_vtd_gdf.merge(demographics_geo_df, on='CNTYVTD', how='left')
        print(f"✅ Merged demographics geo info. Rows after merge: {len(master_vtd_gdf)}")
    else:
        print("⚠️ demographics_geo_df is empty. Skipping merge.")
        # Add placeholder columns if merge is skipped
        geo_cols_placeholder = ['Best Matching Precinct Name (Max Overlap)', 'County Name (from TORV)', 'Cities (from TORV)']
        for col in geo_cols_placeholder:
            if col not in master_vtd_gdf.columns:
                master_vtd_gdf[col] = np.nan # Use NaN for missing data

    # 7a - NEW: Merge demographics_popup_df to include all demographic columns
    if not demographics_popup_df.empty:
        print("Merging demographics_popup_df...")
        # Identify demographic columns to merge from demographics_popup_df, excluding the key
        demo_popup_cols_to_merge = [col for col in demographics_popup_df.columns if col != 'CNTYVTD']
        # Drop potential duplicate columns in master_vtd_gdf before merging
        master_vtd_gdf.drop(columns=[col for col in demo_popup_cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
        master_vtd_gdf = master_vtd_gdf.merge(demographics_popup_df, on='CNTYVTD', how='left')
        print(f"✅ Merged demographics popup data. Rows after merge: {len(master_vtd_gdf)}")
    else:
        print("⚠️ demographics_popup_df is empty. Skipping merge.")
        # Add placeholder columns for key demographic metrics if merge is skipped
        # Use the expected final column names based on how they appear in demographics_df
        demo_popup_placeholder_cols = [
            'Total Population (2020)', 'Total Population (2022)', 'Total Population (2024)',
            'Anglo Population (2020)', 'Anglo Population (2022)', 'Anglo Population (2024)',
            'Non-Anglo Population (2020)', 'Non-Anglo Population (2022)', 'Non-Anglo Population (2024)',
            'Asian Population (2020)', 'Asian Population (2022)', 'Asian Population (2024)',
            'Black Population (2020)', 'Black Population (2022)', 'Black Population (2024)',
            'Hispanic Population (2020)', 'Hispanic Population (2022)', 'Hispanic Population (2024)',
            'Black + Hispanic Population (2020)', 'Black + Hispanic Population (2022)', 'Black + Hispanic Population (2024)',
            'Voting Age Population (2020)', 'Voting Age Population (2022)', 'Voting Age Population (2024)',
            'Anglo VAP (2020)', 'Anglo VAP (2022)', 'Anglo VAP (2024)',
            'Non-Anglo VAP (2020)', 'Non-Anglo VAP (2022)', 'Non-Anglo VAP (2024)',
            'Asian VAP (2020)', 'Asian VAP (2022)', 'Asian VAP (2024)',
            'Black VAP (2020)', 'Black VAP (2022)', 'Black VAP (2024)',
            'Hispanic VAP (2020)', 'Hispanic VAP (2022)', 'Hispanic VAP (2024)',
            'Black + Hispanic VAP (2020)', 'Black + Hispanic VAP (2022)', 'Black + Hispanic VAP (2024)',
            'Total Pop % Change (2020-2022)', 'Total Pop % Change (2022-2024)',
            'Anglo Pop % Change (2020-2022)', 'Anglo Pop % Change (2022-2024)',
            'Non-Anglo Pop % Change (2020-2022)', 'Non-Anglo Pop % Change (2022-2024)',
            'Asian Pop % Change (2020-2022)', 'Asian Pop % Change (2022-2024)',
            'Black Pop % Change (2020-2022)', 'Black Pop % Change (2022-2024)',
            'Hispanic Pop % Change (2020-2022)', 'Hispanic Pop % Change (2022-2024)',
            'Black + Hispanic Pop % Change (2020-2022)', 'Black + Hispanic Pop % Change (2022-2024)',
            'VAP % Change (2020-2022)', 'VAP % Change (2022-2024)',
            'Anglo VAP % Change (2020-2022)', 'Anglo VAP % Change (2022-2024)',
            'Non-Anglo VAP % Change (2020-2022)', 'Non-Anglo VAP % Change (2022-2024)',
            'Asian VAP % Change (2020-2022)', 'Asian VAP % Change (2022-2024)',
            'Black VAP % Change (2020-2022)', 'Black VAP % Change (2022-2024)',
            'Hispanic VAP % Change (2020-2022)', 'Hispanic VAP % Change (2022-2024)',
            'Black + Hispanic VAP % Change (2020-2022)', 'Black + Hispanic VAP % Change (2022-2024)'
        ]
        for col in demo_popup_placeholder_cols:
             if col not in master_vtd_gdf.columns:
                  master_vtd_gdf[col] = np.nan


    # 7b. Merge cleaned_torv_df
    if not cleaned_torv_df.empty:
        print("Merging cleaned_torv_df...")
        # Identify TORV columns to merge, excluding the keys and geo columns already handled (either from demographics_geo_df or potentially if they exist in cleaned_torv_df)
        torv_cols_to_merge = [col for col in cleaned_torv_df.columns if col not in ['CNTYVTD', 'Precinct', 'County', 'Intersecting Cities', 'Best Matching Precinct Name (Max Overlap)', 'County Name (from TORV)', 'Cities (from TORV)']]
        # Drop potential duplicate columns in master_vtd_gdf before merging TORV data
        master_vtd_gdf.drop(columns=[col for col in torv_cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')

        # Select only the relevant columns from cleaned_torv_df for the merge
        cols_from_torv = ['CNTYVTD'] + torv_cols_to_merge
        master_vtd_gdf = master_vtd_gdf.merge(cleaned_torv_df[cols_from_torv], on='CNTYVTD', how='left')
        print(f"✅ Merged cleaned TORV data. Rows after merge: {len(master_vtd_gdf)}")
    else:
        print("⚠️ cleaned_torv_df is empty. Skipping merge.")
        # Add placeholder columns for key TORV metrics if merge is skipped
        # Use the original TORV columns (excluding key) for placeholders
        torv_placeholder_cols = [col for col in cleaned_torv_df.columns if col != 'CNTYVTD']
        for col in torv_placeholder_cols:
            if col not in master_vtd_gdf.columns:
                master_vtd_gdf[col] = np.nan


    # 7c. Merge primary_votes_cleaned_df
    if not primary_votes_cleaned_df.empty:
        print("Merging primary_votes_cleaned_df...")
        # Identify vote columns to merge
        vote_cols_to_merge = [col for col in primary_votes_cleaned_df.columns if col != 'CNTYVTD']
         # Drop potential duplicate columns in master_vtd_gdf before merging primary votes
        master_vtd_gdf.drop(columns=[col for col in vote_cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
        master_vtd_gdf = master_vtd_gdf.merge(primary_votes_cleaned_df, on='CNTYVTD', how='left')
        print(f"✅ Merged primary votes data. Rows after merge: {len(master_vtd_gdf)}")
    else:
        print("⚠️ primary_votes_cleaned_df is empty. Skipping merge.")
        # Add placeholder columns for key primary vote metrics if merge is skipped
        # Use the expected final column names based on how they appear in the merged data
        vote_placeholder_cols = [
            'Ellzey (R) 2022 Primary Votes', 'Buford (R) 2022 Primary Votes', 'Payne (R) 2022 Primary Votes',
            'Ellzey (R) 2024 Primary Votes', 'Buford (R) 2024 Primary Votes', 'Wiley (R) 2024 Primary Votes'
        ]
        for col in vote_placeholder_cols:
             if col not in master_vtd_gdf.columns:
                  master_vtd_gdf[col] = np.nan


    # 7d. Calculate Vote Changes and Percentages (only if necessary columns exist)
    print("\n--- Calculating Vote Changes and Percentages ---")
    # Use the actual column names after merging
    vote_cols_for_calc = {
        'BufordR': {'2022': 'Buford (R) 2022 Primary Votes', '2024': 'Buford (R) 2024 Primary Votes'},
        'EllzeyR': {'2022': 'Ellzey (R) 2022 Primary Votes', '2024': 'Ellzey (R) 2024 Primary Votes'},
        'PayneR': {'2022': 'Payne (R) 2022 Primary Votes', '2024': None}, # Payne only in 2022
        'WileyR': {'2022': None, '2024': 'Wiley (R) 2024 Primary Votes'} # Wiley only in 2024
    }

    for candidate, years in vote_cols_for_calc.items():
        col_2022 = years['2022']
        col_2024 = years['2024']

        # Ensure columns exist and are numeric
        col_2022_exists = col_2022 in master_vtd_gdf.columns
        col_2024_exists = col_2024 in master_vtd_gdf.columns

        if col_2022_exists:
             master_vtd_gdf[col_2022] = pd.to_numeric(master_vtd_gdf[col_2022], errors='coerce').fillna(0)
        if col_2024_exists:
             master_vtd_gdf[col_2024] = pd.to_numeric(master_vtd_gdf[col_2024], errors='coerce').fillna(0)


        if col_2022_exists and col_2024_exists:
            # Calculate Amount Change
            amount_change_col = f'{candidate}_Amount_Change_2022_2024'
            master_vtd_gdf[amount_change_col] = master_vtd_gdf[col_2024] - master_vtd_gdf[col_2022]
            print(f"✅ Calculated '{amount_change_col}'.")

            # Calculate Percentage Change
            pct_change_col = f'{candidate}_Pct_Change_2022_2024'
            denominator = master_vtd_gdf[col_2022].replace(0, np.nan)
            master_vtd_gdf[pct_change_col] = ((master_vtd_gdf[col_2024] - master_vtd_gdf[col_2022]) / denominator) * 100
            master_vtd_gdf[pct_change_col].replace([np.inf, -np.inf], np.nan, inplace=True)
            print(f"✅ Calculated '{pct_change_col}'.")
        elif col_2022_exists and col_2024 is None:
             print(f"⚠️ Only 2022 data available for {candidate}. Skipping 2022-2024 change calculations.")
             master_vtd_gdf[f'{candidate}_Amount_Change_2022_2024'] = np.nan
             master_vtd_gdf[f'{candidate}_Pct_Change_2022_2024'] = np.nan
        elif col_2022 is None and col_2024_exists:
             print(f"⚠️ Only 2024 data available for {candidate}. Skipping 2022-2024 change calculations.")
             master_vtd_gdf[f'{candidate}_Amount_Change_2022_2024'] = np.nan
             master_vtd_gdf[f'{candidate}_Pct_Change_2022_2024'] = np.nan
        else:
             print(f"❌ Necessary vote columns for {candidate} not found for change calculations. Skipping.")
             # Add placeholder columns for changes if calculation was skipped
             master_vtd_gdf[f'{candidate}_Amount_Change_2022_2024'] = np.nan
             master_vtd_gdf[f'{candidate}_Pct_Change_2022_2024'] = np.nan


    # 7e. Calculate Demographic Changes and Percentages (only if necessary columns exist)
    print("\n--- Calculating Demographic Changes and Percentages ---")
    demographic_years = ['2020', '2022', '2024']
    demographic_categories = [
        'Total Population', 'Anglo Population', 'Non-Anglo Population',
        'Asian Population', 'Black Population', 'Hispanic Population', 'Black + Hispanic Population',
        'Voting Age Population', 'Anglo VAP', 'Non-Anglo VAP',
        'Asian VAP', 'Black VAP', 'Hispanic VAP', 'Black + Hispanic VAP'
    ]

    for category in demographic_categories:
        col_2020 = f'{category} ({demographic_years[0]})'
        col_2022 = f'{category} ({demographic_years[1]})'
        col_2024 = f'{category} ({demographic_years[2]})'

        # Ensure columns exist and are numeric in master_vtd_gdf after merging demographics_df
        col_2020_exists = col_2020 in master_vtd_gdf.columns
        col_2022_exists = col_2022 in master_vtd_gdf.columns
        col_2024_exists = col_2024 in master_vtd_gdf.columns

        # Calculate 2020-2022 Changes
        if col_2020_exists and col_2022_exists:
            change_col_2020_2022 = f'{category} Change 2020-2022'
            pct_change_col_2020_2022 = f'{category} % Change 2020-2022'

            # Ensure columns are numeric before calculation
            master_vtd_gdf[col_2020] = pd.to_numeric(master_vtd_gdf[col_2020], errors='coerce')
            master_vtd_gdf[col_2022] = pd.to_numeric(master_vtd_gdf[col_2022], errors='coerce')


            master_vtd_gdf[change_col_2020_2022] = master_vtd_gdf[col_2022] - master_vtd_gdf[col_2020]
            denominator = master_vtd_gdf[col_2020].replace(0, np.nan)
            master_vtd_gdf[pct_change_col_2020_2022] = ((master_vtd_gdf[col_2022] - master_vtd_gdf[col_2020]) / denominator) * 100
            master_vtd_gdf[pct_change_col_2020_2022].replace([np.inf, -np.inf], np.nan, inplace=True)
            print(f"✅ Calculated '{change_col_2020_2022}' and '{pct_change_col_2020_2022}'.")
        else:
            print(f"❌ Missing columns for {category} Change 2020-2022. Skipping calculation.")
            master_vtd_gdf[f'{category} Change 2020-2022'] = np.nan
            master_vtd_gdf[f'{category} % Change 2020-2022'] = np.nan


        # Calculate 2022-2024 Changes
        if col_2022_exists and col_2024_exists:
            change_col_2022_2024 = f'{category} Change 2022-2024'
            pct_change_col_2022_2024 = f'{category} % Change 2022-2024'

            # Ensure columns are numeric before calculation
            master_vtd_gdf[col_2022] = pd.to_numeric(master_vtd_gdf[col_2022], errors='coerce')
            master_vtd_gdf[col_2024] = pd.to_numeric(master_vtd_gdf[col_2024], errors='coerce')

            master_vtd_gdf[change_col_2022_2024] = master_vtd_gdf[col_2024] - master_vtd_gdf[col_2022]
            denominator = master_vtd_gdf[col_2022].replace(0, np.nan)
            master_vtd_gdf[pct_change_col_2022_2024] = ((master_vtd_gdf[col_2024] - master_vtd_gdf[col_2022]) / denominator) * 100
            master_vtd_gdf[pct_change_col_2022_2024].replace([np.inf, -np.inf], np.nan, inplace=True)
            print(f"✅ Calculated '{change_col_2022_2024}' and '{pct_change_col_2022_2024}'.")
        else:
            print(f"❌ Missing columns for {category} Change 2022-2024. Skipping calculation.")
            master_vtd_gdf[f'{category} Change 2022-2024'] = np.nan
            master_vtd_gdf[f'{category} % Change 2022-2024'] = np.nan


    # 8. Print the shape, head, and column names of the final master_vtd_gdf
    print("\n--- Final Merged GeoDataFrame Inspection ---")
    print(f"Merged GeoDataFrame shape: {master_vtd_gdf.shape}")
    print("\nHead of merged master_vtd_gdf:")
    display(master_vtd_gdf.head())
    print("\nColumns of merged master_vtd_gdf:")
    print(master_vtd_gdf.columns.tolist())

else:
    print("\n❌ Merge process skipped due to missing or invalid base GeoDataFrame.")

## Prepare data for popup HTML

### Subtask:
Prepare the HTML snippets for the popup, including basic information, demographic data, primary vote data, and VR/Turnout data. Generate the full interactive popup HTML with the dropdown functionality.

**Reasoning**:
Generate the HTML snippets for the basic info, demographic, primary votes, and VR/Turnout data, and then combine them into a single HTML string for the interactive popup with dropdown functionality.

In [ ]:
import pandas as pd
import numpy as np # Ensure numpy is imported for NaN handling

# Ensure master_vtd_gdf is available
if 'master_vtd_gdf' not in locals() or master_vtd_gdf is None or master_vtd_gdf.empty:
    print("❌ master_vtd_gdf GeoDataFrame is not available or empty. Cannot prepare HTML columns.")
else:
    print("✅ master_vtd_gdf is available for HTML column preparation.")

    # Define a helper function to format numeric values and percentages, handling NaNs
    def format_value_robust(value):
        if pd.notnull(value):
            try:
                # Check if the value is a string representation of a percentage (e.g., "10.5%")
                if isinstance(value, str) and '%' in value:
                    # Attempt to convert to float and format, keep original if fails
                    try:
                        return f"{float(value.replace('%', '').strip()):.1f}%"
                    except:
                        return str(value)
                else:
                    # Format numeric values with commas
                    return f"{int(float(value)):,}"
            except (ValueError, TypeError):
                # Handle cases where conversion to float/int fails (e.g., non-numeric strings)
                return str(value)
        return 'N/A'

    # Define a helper function to format vote counts and add color span for change
    def format_vote_change(change, percentage):
        color = 'green' if pd.notnull(change) and change > 0 else ('red' if pd.notnull(change) and change < 0 else 'black')
        # Format percentage to one decimal place, handle NaN
        pct_str = f"{percentage:+.1f}%" if pd.notnull(percentage) else 'N/A'
        return f"<span style='color:{color};'>({format_value_robust(change)} / {pct_str})</span>" if pd.notnull(change) else 'N/A'


    # Define a helper function to format percentages and add color span based on thresholds
    # We need thresholds for RV % and Turnout % which will be determined later.
    # For now, we'll use a basic color based on positive/negative change or default to black.
    def format_percentage_color(percentage):
         if pd.notnull(percentage):
             try:
                  # Attempt to convert to float if it's a string like "50.5%"
                  percentage_float = float(str(percentage).replace('%', '').strip())
                  color = 'green' if percentage_float > 0 else ('red' if percentage_float < 0 else 'black')
                  return f"<span style='color:{color};'>{percentage:.1f}%</span>"
             except:
                  return str(percentage) # Return as string if conversion or comparison fails
         return 'N/A'


    # --- Prepare Tooltip HTML Snippet ---
    print("--- Preparing Tooltip HTML Snippet ---")
    # Use the columns available after merging (prioritizing TORV/cleaned names if they exist)
    # Include PREC, PlanH2316, PlanS2168 from the latest merge
    master_vtd_gdf['tooltip_text'] = master_vtd_gdf.apply(lambda row: f"""
<b>VTD Identifier:</b> {row.get('VTDKEY', row.get('CNTYVTD', 'N/A'))}<br>
<b>County:</b> {row.get('County', row.get('County Name (from TORV)', 'N/A'))}<br>
<b>Precinct(s):</b> {row.get('PREC', row.get('Best Matching Precinct Name (Max Overlap)', 'N/A'))}<br>
<b>Cities:</b> {row.get('Intersecting Cities', row.get('Cities (from TORV)', 'N/A'))}<br>
<b>Congressional District:</b> {row.get('CD', 'N/A')}<br>
<b>House District:</b> {row.get('PlanH2316', 'N/A')}<br>
<b>Senate District:</b> {row.get('PlanS2168', 'N/A')}
    """, axis=1)
    print("✅ Created 'tooltip_text' column for VTD hover tooltips.")


    # --- Prepare Popup HTML Snippets ---
    print("\n--- Preparing Popup HTML Snippets ---")

    # Prepare basic info HTML snippet - Use the columns from DemographicsFinal.csv or cleaned_torv_data.csv
    # Prioritize columns from cleaned_torv_data.csv if available, otherwise use DemographicsFinal.csv merged columns
    # Use VTDKEY and the newly merged County and PREC
    master_vtd_gdf['Basic_Info_HTML'] = master_vtd_gdf.apply(lambda row: f"""
    <b>VTD Identifier:</b> {row.get('VTDKEY', row.get('CNTYVTD', 'N/A'))}<br>
    <b>Precinct(s):</b> {row.get('PREC', row.get('Best Matching Precinct Name (Max Overlap)', 'N/A'))}<br>
    <b>County:</b> {row.get('County', row.get('County Name (from TORV)', 'N/A'))}<br>
    <b>Cities:</b> {row.get('Intersecting Cities', row.get('Cities (from TORV)', 'N/A'))}
    """, axis=1)
    print("✅ Prepared 'Basic_Info_HTML'.")


    # Prepare vote performance HTML snippet for each VTD (using calculated change columns)
    master_vtd_gdf['Vote_Performance_HTML'] = master_vtd_gdf.apply(lambda row: f"""
    <b>Primary Vote Performance:</b><br>
    <table style="width:100%;">
      <tr>
        <th style="text-align:left;">Candidate</th>
        <th style="text-align:left;">2022 Primary</th>
        <th style="text-align:left;">2024 Primary</th>
      </tr>
      <tr>
        <td>Buford (R)</td>
        <td>{format_value_robust(row.get('Buford (R) 2022 Primary Votes', np.nan))}</td>
        <td>{format_value_robust(row.get('Buford (R) 2024 Primary Votes', np.nan))} {format_vote_change(row.get('BufordR_Amount_Change_2022_2024', np.nan), row.get('BufordR_Pct_Change_2022_2024', np.nan))}</td>
      </tr>
      <tr>
        <td>Ellzey (R)</td>
        <td>{format_value_robust(row.get('Ellzey (R) 2022 Primary Votes', np.nan))}</td>
        <td>{format_value_robust(row.get('Ellzey (R) 2024 Primary Votes', np.nan))} {format_vote_change(row.get('EllzeyR_Amount_Change_2022_2024', np.nan), row.get('EllzeyR_Pct_Change_2022_2024', np.nan))}</td>
      </tr>
      <tr>
        <td>Payne (R)</td>
        <td>{format_value_robust(row.get('Payne (R) 2022 Primary Votes', np.nan))}</td>
        <td>N/A</td> <!-- Payne was not in the 2024 primary -->
      </tr>
      <tr>
        <td>Wiley (R)</td>
        <td>N/A</td> <!-- Wiley was not in the 2022 primary -->
        <td>{format_value_robust(row.get('Wiley (R) 2024 Primary Votes', np.nan))} {format_vote_change(row.get('WileyR_Amount_Change_2022_2024', np.nan), row.get('WileyR_Pct_Change_2022_2024', np.nan))}</td>
      </tr>
    </table>
    """, axis=1)
    print("✅ Prepared 'Vote_Performance_HTML'.")


    # Prepare Voter Registration and Turnout HTML snippet
    master_vtd_gdf['RV_Turnout_HTML'] = master_vtd_gdf.apply(lambda row: f"""
    <b>Voter Registration & Turnout Data:</b><br>
    <table style="width:100%;">
      <tr>
        <th>Metric</th>
        <th>2020</th>
        <th>2022</th>
        <th>2024</th>
        <th>Change (20-22)</th>
        <th>% Change (20-22)</th>
        <th>Change (22-24)</th>
        <th>% Change (22-24)</th>
      </tr>
      <tr>
        <td>Registered Voters</td>
        <td>{format_value_robust(row.get('Voter Registration 2020', np.nan))}</td>
        <td>{format_value_robust(row.get('Voter Registration 2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Voter Registration 2024', np.nan))}</td>
        <td>{format_value_robust(row.get('VR Change 2020-2022', np.nan))}</td>
        <td>{format_percentage_color(row.get('VR % Change 2020-2022', np.nan))}</td> # Using basic color for now
        <td>{format_value_robust(row.get('VR Change 2022-2024', np.nan))}</td>
        <td>{format_percentage_color(row.get('VR % Change 2022-2024', np.nan))}</td> # Using basic color for now
      </tr>
       <tr>
        <td>Turnout</td>
        <td>{format_value_robust(row.get('Turnout 2020', np.nan))}</td>
        <td>{format_value_robust(row.get('Turnout 2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Turnout 2024', np.nan))}</td>
        <td>{format_value_robust(row.get('Turnout Change 2020-2022', np.nan))}</td>
        <td>{format_percentage_color(row.get('Turnout % Change 2020-2022', np.nan))}</td> # Using basic color for now
        <td>{format_value_robust(row.get('Turnout Change 2022-2024', np.nan))}</td>
        <td>{format_percentage_color(row.get('Turnout % Change 2022-2024', np.nan))}</td> # Using basic color for now
      </tr>
    </table>
    """, axis=1)
    print("✅ Prepared 'RV_Turnout_HTML'.")

    # Prepare Demographic Data HTML (using calculated change columns)
    master_vtd_gdf['demographics_html'] = master_vtd_gdf.apply(lambda row: f"""
    <b>Demographic Data:</b><br>
    <table style="width:100%;">
      <tr>
        <th>Category</th>
        <th>2020</th>
        <th>2022</th>
        <th>2024</th>
        <th>Change (20-22)</th>
        <th>% Change (20-22)</th>
        <th>Change (22-24)</th>
        <th>% Change (22-24)</th>
      </tr>
      <tr>
        <td>Total Pop</td>
        <td>{format_value_robust(row.get('Total Population (2020)', np.nan))}</td>
        <td>{format_value_robust(row.get('Total Population (2022)', np.nan))}</td>
        <td>{format_value_robust(row.get('Total Population (2024)', np.nan))}</td>
        <td>{format_value_robust(row.get('Total Population Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Total Population % Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Total Population Change 2022-2024', np.nan))}</td>
        <td>{format_value_robust(row.get('Total Population % Change 2022-2024', np.nan))}</td>
      </tr>
      <tr>
        <td>Anglo Pop</td>
        <td>{format_value_robust(row.get('Anglo Population (2020)', np.nan))}</td>
        <td>{format_value_robust(row.get('Anglo Population (2022)', np.nan))}</td>
        <td>{format_value_robust(row.get('Anglo Population (2024)', np.nan))}</td>
        <td>{format_value_robust(row.get('Anglo Population Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Anglo Population % Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Anglo Population Change 2022-2024', np.nan))}</td>
        <td>{format_value_robust(row.get('Anglo Population % Change 2022-2024', np.nan))}</td>
      </tr>
       <tr>
        <td>Non-Anglo Pop</td>
        <td>{format_value_robust(row.get('Non-Anglo Population (2020)', np.nan))}</td>
        <td>{format_value_robust(row.get('Non-Anglo Population (2022)', np.nan))}</td>
        <td>{format_value_robust(row.get('Non-Anglo Population (2024)', np.nan))}</td>
        <td>{format_value_robust(row.get('Non-Anglo Population Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Non-Anglo Population % Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Non-Anglo Population Change 2022-2024', np.nan))}</td>
        <td>{format_value_robust(row.get('Non-Anglo Population % Change 2022-2024', np.nan))}</td>
      </tr>
      <tr>
        <td>Asian Pop</td>
        <td>{format_value_robust(row.get('Asian Population (2020)', np.nan))}</td>
        <td>{format_value_robust(row.get('Asian Population (2022)', np.nan))}</td>
        <td>{format_value_robust(row.get('Asian Population (2024)', np.nan))}</td>
        <td>{format_value_robust(row.get('Asian Population Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Asian Population % Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Asian Population Change 2022-2024', np.nan))}</td>
        <td>{format_value_robust(row.get('Asian Population % Change 2022-2024', np.nan))}</td>
      </tr>
      <tr>
        <td>Black Pop</td>
        <td>{format_value_robust(row.get('Black Population (2020)', np.nan))}</td>
        <td>{format_value_robust(row.get('Black Population (2022)', np.nan))}</td>
        <td>{format_value_robust(row.get('Black Population (2024)', np.nan))}</td>
        <td>{format_value_robust(row.get('Black Population Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Black Population % Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Black Population Change 2022-2024', np.nan))}</td>
        <td>{format_value_robust(row.get('Black Population % Change 2022-2024', np.nan))}</td>
      </tr>
       <tr>
        <td>Hispanic Pop</td>
        <td>{format_value_robust(row.get('Hispanic Population (2020)', np.nan))}</td>
        <td>{format_value_robust(row.get('Hispanic Population (2022)', np.nan))}</td>
        <td>{format_value_robust(row.get('Hispanic Population (2024)', np.nan))}</td>
        <td>{format_value_robust(row.get('Hispanic Population Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Hispanic Population % Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Hispanic Population Change 2022-2024', np.nan))}</td>
        <td>{format_value_robust(row.get('Hispanic Population % Change 2022-2024', np.nan))}</td>
      </tr>
       <tr>
        <td>Black + Hispanic Pop</td>
        <td>{format_value_robust(row.get('Black + Hispanic Population (2020)', np.nan))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic Population (2022)', np.nan))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic Population (2024)', np.nan))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic Population Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic Population % Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic Population Change 2022-2024', np.nan))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic Population % Change 2022-2024', np.nan))}</td>
      </tr>
       <tr>
        <td>VAP</td>
        <td>{format_value_robust(row.get('Voting Age Population (2020)', np.nan))}</td>
        <td>{format_value_robust(row.get('Voting Age Population (2022)', np.nan))}</td>
        <td>{format_value_robust(row.get('Voting Age Population (2024)', np.nan))}</td>
         <td>{format_value_robust(row.get('Voting Age Population Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Voting Age Population % Change 2020-2022', np.nan))}</td>
         <td>{format_value_robust(row.get('Voting Age Population Change 2022-2024', np.nan))}</td>
        <td>{format_value_robust(row.get('Voting Age Population % Change 2022-2024', np.nan))}</td>
      </tr>
       <tr>
        <td>Anglo VAP</td>
        <td>{format_value_robust(row.get('Anglo VAP (2020)', np.nan))}</td>
        <td>{format_value_robust(row.get('Anglo VAP (2022)', np.nan))}</td>
        <td>{format_value_robust(row.get('Anglo VAP (2024)', np.nan))}</td>
         <td>{format_value_robust(row.get('Anglo VAP Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Anglo VAP % Change 2020-2022', np.nan))}</td>
         <td>{format_value_robust(row.get('Anglo VAP Change 2022-2024', np.nan))}</td>
        <td>{format_value_robust(row.get('Anglo VAP % Change 2022-2024', np.nan))}</td>
      </tr>
      <tr>
        <td>Non-Anglo VAP</td>
        <td>{format_value_robust(row.get('Non-Anglo VAP (2020)', np.nan))}</td>
        <td>{format_value_robust(row.get('Non-Anglo VAP (2022)', np.nan))}</td>
        <td>{format_value_robust(row.get('Non-Anglo VAP (2024)', np.nan))}</td>
        <td>{format_value_robust(row.get('Non-Anglo VAP Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Non-Anglo VAP % Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Non-Anglo VAP Change 2022-2024', np.nan))}</td>
        <td>{format_value_robust(row.get('Non-Anglo VAP % Change 2022-2024', np.nan))}</td>
      </tr>
      <tr>
        <td>Asian VAP</td>
        <td>{format_value_robust(row.get('Asian VAP (2020)', np.nan))}</td>
        <td>{format_value_robust(row.get('Asian VAP (2022)', np.nan))}</td>
        <td>{format_value_robust(row.get('Asian VAP (2024)', np.nan))}</td>
        <td>{format_value_robust(row.get('Asian VAP Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Asian VAP % Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Asian VAP Change 2022-2024', np.nan))}</td>
        <td>{format_value_robust(row.get('Asian VAP % Change 2022-2024', np.nan))}</td>
      </tr>
      <tr>
        <td>Black VAP</td>
        <td>{format_value_robust(row.get('Black VAP (2020)', np.nan))}</td>
        <td>{format_value_robust(row.get('Black VAP (2022)', np.nan))}</td>
        <td>{format_value_robust(row.get('Black VAP (2024)', np.nan))}</td>
        <td>{format_value_robust(row.get('Black VAP Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Black VAP % Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Black VAP Change 2022-2024', np.nan))}</td>
        <td>{format_value_robust(row.get('Black VAP % Change 2022-2024', np.nan))}</td>
      </tr>
      <tr>
        <td>Hispanic VAP</td>
        <td>{format_value_robust(row.get('Hispanic VAP (2020)', np.nan))}</td>
        <td>{format_value_robust(row.get('Hispanic VAP (2022)', np.nan))}</td>
        <td>{format_value_robust(row.get('Hispanic VAP (2024)', np.nan))}</td>
        <td>{format_value_robust(row.get('Hispanic VAP Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Hispanic VAP % Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Hispanic VAP Change 2022-2024', np.nan))}</td>
        <td>{format_value_robust(row.get('Hispanic VAP % Change 2022-2024', np.nan))}</td>
      </tr>
      <tr>
        <td>Black + Hispanic VAP</td>
        <td>{format_value_robust(row.get('Black + Hispanic VAP (2020)', np.nan))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic VAP (2022)', np.nan))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic VAP (2024)', np.nan))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic VAP Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic VAP % Change 2020-2022', np.nan))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic VAP Change 2022-2024', np.nan))}</td>
        <td>{format_value_robust(row.get('Black + Hispanic VAP % Change 2022-2024', np.nan))}</td>
      </tr>
    </table>
    """, axis=1)

    print("✅ Created 'demographics_html' column for popup.")


    # --- Generate Full Popup HTML with Checkboxes ---
    # This assumes the individual HTML snippets ('Basic_Info_HTML', 'demographics_html', 'Vote_Performance_HTML', 'RV_Turnout_HTML') have been created
    # If any are missing, the generate_full_popup_html function will use a placeholder.
    def generate_full_popup_html_checkbox(row):
        # Use row.get() with a default empty string to handle potential missing HTML columns gracefully
        basic_info = row.get('Basic_Info_HTML', '<p>Basic info not available.</p>')
        demographics_html = row.get('demographics_html', '<p>Demographic data not available.</p>')
        primary_votes_html = row.get('Vote_Performance_HTML', '<p>Primary vote data not available.</p>') # Use Vote_Performance_HTML
        vr_turnout_html = row.get('RV_Turnout_HTML', '<p>VR/Turnout data not available.</p>') # Use RV_Turnout_HTML


        # Generate the full HTML structure with checkboxes and hidden content sections
        # Embed the script directly within the HTML content
        html_content = f"""
        <div style="width: 350px;">
            {basic_info}
            <hr>
            <b>View Data:</b><br>
            <input type="checkbox" id="demo-check-{row.name}" name="data_view" value="demographics" checked>
            <label for="demo-check-{row.name}"> Demographics</label><br>
            <input type="checkbox" id="votes-check-{row.name}" name="data_view" value="primary_votes">
            <label for="votes-check-{row.name}"> Primary Votes</label><br>
            <input type="checkbox" id="vr-check-{row.name}" name="data_view" value="vr_turnout">
            <label for="vr-check-{row.name}"> VR/Turnout</label><br>
            <hr>
            <div id="demographics-section-{row.name}" class="data-section">
                {demographics_html}
            </div>
            <div id="primary_votes-section-{row.name}" class="data-section" style="display:none;">
                {primary_votes_html}
            </div>
            <div id="vr_turnout-section-{row.name}" class="data-section" style="display:none;">
                {vr_turnout_html}
            </div>
        </div>

        <script>
            // Function to show/hide sections based on checkbox state
            function updateDataSections(rowId) {{
                // Find the container div relative to the checkbox that triggered the event
                // This makes the script more robust if there are multiple popups
                var checkbox = document.getElementById('demo-check-' + rowId); // Use one of the checkboxes to find the container
                if (!checkbox) return; // Exit if checkbox not found (popup likely closed)

                var popupContent = checkbox.closest('div[style="width: 350px;"]');
                if (!popupContent) return; // Exit if popup content not found


                var demoChecked = popupContent.querySelector('#demo-check-' + rowId).checked;
                var votesChecked = popupContent.querySelector('#votes-check-' + rowId).checked;
                var vrChecked = popupContent.querySelector('#vr-check-' + rowId).checked;

                // Ensure sections are found before attempting to change display style
                var demoSection = popupContent.querySelector('#demographics-section-' + rowId);
                if (demoSection) demoSection.style.display = demoChecked ? 'block' : 'none';

                var votesSection = popupContent.querySelector('#primary_votes-section-' + rowId);
                if (votesSection) votesSection.style.display = votesChecked ? 'block' : 'none';

                var vrSection = popupContent.querySelector('#vr_turnout-section-' + rowId);
                if (vrSection) vrSection.style.display = vrChecked ? 'block' : 'none';
            }}

            // Add event listeners to checkboxes using a more reliable method for Folium popups
            // Use a MutationObserver to attach event listeners when the popup content is added to the DOM
            var observer = new MutationObserver(function(mutations, observer) {{
                mutations.forEach(function(mutation) {{
                    if (mutation.addedNodes.length) {{
                        mutation.addedNodes.forEach(function(node) {{
                            if (node.nodeType === 1) {{ // Check if it's an element node
                                // Look for the checkbox container within the added node
                                var container = node.querySelector('div[style="width: 350px;"]');
                                if (container) {{
                                    var demoCheckbox = container.querySelector('input[id^="demo-check-"]');
                                    if (demoCheckbox) {{
                                        var rowId = demoCheckbox.id.replace('demo-check-', '');
                                        var votesCheckbox = container.querySelector('#votes-check-' + rowId);
                                        var vrCheckbox = container.querySelector('#vr-check-' + rowId);

                                        if (votesCheckbox && vrCheckbox) {{ // Ensure all checkboxes are found
                                            // Add event listeners
                                            demoCheckbox.addEventListener('change', function() {{ updateDataSections(rowId); }});
                                            votesCheckbox.addEventListener('change', function() {{ updateDataSections(rowId); }});
                                            vrCheckbox.addEventListener('change', function() {{ updateDataSections(rowId); }});

                                            // Trigger initial state update after event listeners are attached
                                            updateDataSections(rowId);

                                            // Optionally disconnect observer after finding and setting up for this popup
                                            // observer.disconnect(); // Consider if you expect multiple popups simultaneously
                                        }}
                                    }}
                                }}
                            }}
                        }});
                    }}
                }});
            }});

            // Start observing the body for added nodes (where the popup content will be attached by Folium)
            // This is a general approach, more specific targeting within the iframe might be possible but complex.
            observer.observe(document.body, {{ childList: true, subtree: true }});

             // Fallback: Also try triggering after a small delay
             // This can sometimes help if the MutationObserver doesn't catch the initial load in all environments
             setTimeout(function() {{
                 var demoCheckbox = document.querySelector('input[id^="demo-check-"]');
                 if (demoCheckbox) {{
                     var rowId = demoCheckbox.id.replace('demo-check-', '');
                     var votesCheckbox = document.querySelector('#votes-check-' + rowId);
                     var vrCheckbox = document.querySelector('#vr-check-' + rowId);
                     if (votesCheckbox && vrCheckbox) {{
                         updateDataSections(rowId);
                     }}
                 }}
             }}, 250); // Increased delay slightly


        </script>
        """
        return html_content

    # Check if master_vtd_gdf has the necessary HTML columns before applying
    # The columns checked here are those needed to GENERATE the HTML snippets, not the final columns displayed
    required_html_snippet_cols = [
        'VTDKEY', # Needed for basic info and popup ID
        'County', 'County Name (from TORV)', # For Basic_Info_HTML
        'PREC', 'Best Matching Precinct Name (Max Overlap)', # For Basic_Info_HTML and tooltip
        'Intersecting Cities', 'Cities (from TORV)', # For Basic_Info_HTML and tooltip
        'CD', # For tooltip
        'PlanH2316', # For tooltip (Renamed from PlanH2316_y)
        'PlanS2168', # For tooltip (Renamed from PlanS2168_y)
        # Columns needed for Vote_Performance_HTML (adjust based on actual merged columns)
        'Buford (R) 2022 Primary Votes', 'Buford (R) 2024 Primary Votes', 'BufordR_Amount_Change_2022_2024', 'BufordR_Pct_Change_2022_2024',
        'Ellzey (R) 2022 Primary Votes', 'Ellzey (R) 2024 Primary Votes', 'EllzeyR_Amount_Change_2022_2024', 'EllzeyR_Pct_Change_2022_2024',
        'Payne (R) 2022 Primary Votes',
        'Wiley (R) 2024 Primary Votes', 'WileyR_Amount_Change_2022_2024', 'WileyR_Pct_Change_2022_2024',
        # Columns needed for RV_Turnout_HTML (adjust based on actual merged columns)
        'Voter Registration 2020', 'Voter Registration 2022', 'Voter Registration 2024', 'VR Change 2020-2022', 'VR % Change 2020-2022', 'VR Change 2022-2024', 'VR % Change 2022-2024',
        'Turnout 2020', 'Turnout 2022', 'Turnout 2024', 'Turnout Change 2020-2022', 'Turnout % Change 2020-2022', 'Turnout Change 2022-2024', 'Turnout % Change 2022-2024',
        # Columns needed for demographics_html (adjust based on actual merged columns)
        'Total Population (2020)', 'Total Population (2022)', 'Total Population (2024)', 'Total Population Change 2020-2022', 'Total Population % Change 2020-2022', 'Total Population Change 2022-2024', 'Total Population % Change 2022-2024',
        'Anglo Population (2020)', 'Anglo Population (2022)', 'Anglo Population (2024)', 'Anglo Population Change 2020-2022', 'Anglo Population % Change 2020-2022', 'Anglo Population % Change 2022-2024', 'Anglo Population % Change 2022-2024',
        'Non-Anglo Population (2020)', 'Non-Anglo Population (2022)', 'Non-Anglo Population (2024)', 'Non-Anglo Population Change 2020-2022', 'Non-Anglo Population % Change 2020-2022', 'Non-Anglo Population Change 2022-2024', 'Non-Anglo Population % Change 2022-2024',
        'Asian Population (2020)', 'Asian Population (2022)', 'Asian Population (2024)', 'Asian Population Change 2020-2022', 'Asian Population % Change 2020-2022', 'Asian Population Change 2022-2024', 'Asian Population % Change 2022-2024',
        'Black Population (2020)', 'Black Population (2022)', 'Black Population (2024)', 'Black Population Change 2020-2022', 'Black Population % Change 2020-2022', 'Black Population Change 2022-2024', 'Black Population % Change 2022-2024',
        'Hispanic Population (2020)', 'Hispanic Population (2022)', 'Hispanic Population (2024)', 'Hispanic Population Change 2020-2022', 'Hispanic Population % Change 2020-2022', 'Hispanic Population Change 2022-2024', 'Hispanic Population % Change 2022-2024',
        'Black + Hispanic Population (2020)', 'Black + Hispanic Population (2022)', 'Black + Hispanic Population (2024)', 'Black + Hispanic Population Change 2020-2022', 'Black + Hispanic Population % Change 2020-2022', 'Black + Hispanic Population Change 2022-2024', 'Black + Hispanic Population % Change 2022-2024',
        'Voting Age Population (2020)', 'Voting Age Population (2022)', 'Voting Age Population (2024)', 'Voting Age Population Change 2020-2022', 'Voting Age Population % Change 2020-2022', 'Voting Age Population Change 2022-2024', 'Voting Age Population % Change 2022-2024',
        'Anglo VAP (2020)', 'Anglo VAP (2022)', 'Anglo VAP (2024)', 'Anglo VAP Change 2020-2022', 'Anglo VAP % Change 2020-2022', 'Anglo VAP Change 2022-2024', 'Anglo VAP % Change 2022-2024',
        'Non-Anglo VAP (2020)', 'Non-Anglo VAP (2022)', 'Non-Anglo VAP (2024)', 'Non-Anglo VAP Change 2020-2022', 'Non-Anglo VAP % Change 2020-2022', 'Non-Anglo VAP Change 2022-2024', 'Non-Anglo VAP % Change 2022-2024',
        'Asian VAP (2020)', 'Asian VAP (2022)', 'Asian VAP (2024)', 'Asian VAP Change 2020-2022', 'Asian VAP % Change 2020-2022', 'Asian VAP Change 2022-2024', 'Asian VAP % Change 2022-2024',
        'Black VAP (2020)', 'Black VAP (2022)', 'Black VAP (2024)', 'Black VAP Change 2020-2022', 'Black VAP % Change 2020-2022', 'Black VAP Change 2022-2024', 'Black VAP % Change 2022-2024',
        'Hispanic VAP (2020)', 'Hispanic VAP (2022)', 'Hispanic VAP (2024)', 'Hispanic VAP Change 2020-2022', 'Hispanic VAP % Change 2020-2022', 'Hispanic VAP Change 2022-2024', 'Hispanic VAP % Change 2022-2024',
        'Black + Hispanic VAP (2020)', 'Black + Hispanic VAP (2022)', 'Black + Hispanic VAP (2024)', 'Black + Hispanic VAP Change 2020-2022', 'Black + Hispanic VAP % Change 2020-2022', 'Black + Hispanic VAP Change 2022-2024', 'Black + Hispanic VAP % Change 2022-2024',
    ]
    # Check that the necessary columns for *generating the HTML snippets* are present in master_vtd_gdf
    # This check is more comprehensive now based on the columns used in the snippet generation functions
    missing_html_snippet_cols = [col for col in required_html_snippet_cols if col not in master_vtd_gdf.columns]

    if not missing_html_snippet_cols:
         # Generate the individual HTML snippets first
         master_vtd_gdf['Basic_Info_HTML'] = master_vtd_gdf.apply(lambda row: f"""
         <b>VTD Identifier:</b> {row.get('VTDKEY', row.get('CNTYVTD', 'N/A'))}<br>
         <b>Precinct(s):</b> {row.get('PREC', row.get('Best Matching Precinct Name (Max Overlap)', 'N/A'))}<br>
         <b>County:</b> {row.get('County', row.get('County Name (from TORV)', 'N/A'))}<br>
         <b>Cities:</b> {row.get('Intersecting Cities', row.get('Cities (from TORV)', 'N/A'))}
         """, axis=1)

         master_vtd_gdf['Vote_Performance_HTML'] = master_vtd_gdf.apply(lambda row: f"""
         <b>Primary Vote Performance:</b><br>
         <table style="width:100%;">
           <tr>
             <th style="text-align:left;">Candidate</th>
             <th style="text-align:left;">2022 Primary</th>
             <th style="text-align:left;">2024 Primary</th>
           </tr>
           <tr>
             <td>Buford (R)</td>
             <td>{format_value_robust(row.get('Buford (R) 2022 Primary Votes', np.nan))}</td>
             <td>{format_value_robust(row.get('Buford (R) 2024 Primary Votes', np.nan))} {format_vote_change(row.get('BufordR_Amount_Change_2022_2024', np.nan), row.get('BufordR_Pct_Change_2022_2024', np.nan))}</td>
           </tr>
           <tr>
             <td>Ellzey (R)</td>
             <td>{format_value_robust(row.get('Ellzey (R) 2022 Primary Votes', np.nan))}</td>
             <td>{format_value_robust(row.get('Ellzey (R) 2024 Primary Votes', np.nan))} {format_vote_change(row.get('EllzeyR_Amount_Change_2022_2024', np.nan), row.get('EllzeyR_Pct_Change_2022_2024', np.nan))}</td>
           </tr>
           <tr>
             <td>Payne (R)</td>
             <td>{format_value_robust(row.get('Payne (R) 2022 Primary Votes', np.nan))}</td>
             <td>N/A</td> <!-- Payne was not in the 2024 primary -->
           </tr>
           <tr>
             <td>Wiley (R)</td>
             <td>N/A</td> <!-- Wiley was not in the 2022 primary -->
             <td>{format_value_robust(row.get('Wiley (R) 2024 Primary Votes', np.nan))} {format_vote_change(row.get('WileyR_Amount_Change_2022_2024', np.nan), row.get('WileyR_Pct_Change_2022_2024', np.nan))}</td>
           </tr>
         </table>
         """, axis=1)

         master_vtd_gdf['RV_Turnout_HTML'] = master_vtd_gdf.apply(lambda row: f"""
         <b>Voter Registration & Turnout Data:</b><br>
         <table style="width:100%;">
           <tr>
             <th>Metric</th>
             <th>2020</th>
             <th>2022</th>
             <th>2024</th>
             <th>Change (20-22)</th>
             <th>% Change (20-22)</th>
             <th>Change (22-24)</th>
             <th>% Change (22-24)</th>
           </tr>
           <tr>
             <td>Registered Voters</td>
             <td>{format_value_robust(row.get('Voter Registration 2020', np.nan))}</td>
             <td>{format_value_robust(row.get('Voter Registration 2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Voter Registration 2024', np.nan))}</td>
             <td>{format_value_robust(row.get('VR Change 2020-2022', np.nan))}</td>
             <td>{format_percentage_color(row.get('VR % Change 2020-2022', np.nan))}</td> # Using basic color for now
             <td>{format_value_robust(row.get('VR Change 2022-2024', np.nan))}</td>
             <td>{format_percentage_color(row.get('VR % Change 2022-2024', np.nan))}</td> # Using basic color for now
           </tr>
            <tr>
             <td>Turnout</td>
             <td>{format_value_robust(row.get('Turnout 2020', np.nan))}</td>
             <td>{format_value_robust(row.get('Turnout 2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Turnout 2024', np.nan))}</td>
             <td>{format_value_robust(row.get('Turnout Change 2020-2022', np.nan))}</td>
             <td>{format_percentage_color(row.get('Turnout % Change 2020-2022', np.nan))}</td> # Using basic color for now
             <td>{format_value_robust(row.get('Turnout Change 2022-2024', np.nan))}</td>
             <td>{format_percentage_color(row.get('Turnout % Change 2022-2024', np.nan))}</td> # Using basic color for now
           </tr>
         </table>
         """, axis=1)


         master_vtd_gdf['demographics_html'] = master_vtd_gdf.apply(lambda row: f"""
         <b>Demographic Data:</b><br>
         <table style="width:100%;">
           <tr>
             <th>Category</th>
             <th>2020</th>
             <th>2022</th>
             <th>2024</th>
             <th>Change (20-22)</th>
             <th>% Change (20-22)</th>
             <th>Change (22-24)</th>
             <th>% Change (22-24)</th>
           </tr>
           <tr>
             <td>Total Pop</td>
             <td>{format_value_robust(row.get('Total Population (2020)', np.nan))}</td>
             <td>{format_value_robust(row.get('Total Population (2022)', np.nan))}</td>
             <td>{format_value_robust(row.get('Total Population (2024)', np.nan))}</td>
             <td>{format_value_robust(row.get('Total Population Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Total Population % Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Total Population Change 2022-2024', np.nan))}</td>
             <td>{format_value_robust(row.get('Total Population % Change 2022-2024', np.nan))}</td>
           </tr>
           <tr>
             <td>Anglo Pop</td>
             <td>{format_value_robust(row.get('Anglo Population (2020)', np.nan))}</td>
             <td>{format_value_robust(row.get('Anglo Population (2022)', np.nan))}</td>
             <td>{format_value_robust(row.get('Anglo Population (2024)', np.nan))}</td>
             <td>{format_value_robust(row.get('Anglo Population Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Anglo Population % Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Anglo Population Change 2022-2024', np.nan))}</td>
             <td>{format_value_robust(row.get('Anglo Population % Change 2022-2024', np.nan))}</td>
           </tr>
            <tr>
             <td>Non-Anglo Pop</td>
             <td>{format_value_robust(row.get('Non-Anglo Population (2020)', np.nan))}</td>
             <td>{format_value_robust(row.get('Non-Anglo Population (2022)', np.nan))}</td>
             <td>{format_value_robust(row.get('Non-Anglo Population (2024)', np.nan))}</td>
             <td>{format_value_robust(row.get('Non-Anglo Population Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Non-Anglo Population % Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Non-Anglo Population Change 2022-2024', np.nan))}</td>
             <td>{format_value_robust(row.get('Non-Anglo Population % Change 2022-2024', np.nan))}</td>
           </tr>
           <tr>
             <td>Asian Pop</td>
             <td>{format_value_robust(row.get('Asian Population (2020)', np.nan))}</td>
             <td>{format_value_robust(row.get('Asian Population (2022)', np.nan))}</td>
             <td>{format_value_robust(row.get('Asian Population (2024)', np.nan))}</td>
             <td>{format_value_robust(row.get('Asian Population Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Asian Population % Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Asian Population Change 2022-2024', np.nan))}</td>
             <td>{format_value_robust(row.get('Asian Population % Change 2022-2024', np.nan))}</td>
           </tr>
           <tr>
             <td>Black Pop</td>
             <td>{format_value_robust(row.get('Black Population (2020)', np.nan))}</td>
             <td>{format_value_robust(row.get('Black Population (2022)', np.nan))}</td>
             <td>{format_value_robust(row.get('Black Population (2024)', np.nan))}</td>
             <td>{format_value_robust(row.get('Black Population Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Black Population % Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Black Population Change 2022-2024', np.nan))}</td>
             <td>{format_value_robust(row.get('Black Population % Change 2022-2024', np.nan))}</td>
           </tr>
            <tr>
             <td>Hispanic Pop</td>
             <td>{format_value_robust(row.get('Hispanic Population (2020)', np.nan))}</td>
             <td>{format_value_robust(row.get('Hispanic Population (2022)', np.nan))}</td>
             <td>{format_value_robust(row.get('Hispanic Population (2024)', np.nan))}</td>
             <td>{format_value_robust(row.get('Hispanic Population Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Hispanic Population % Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Hispanic Population Change 2022-2024', np.nan))}</td>
             <td>{format_value_robust(row.get('Hispanic Population % Change 2022-2024', np.nan))}</td>
           </tr>
            <tr>
             <td>Black + Hispanic Pop</td>
             <td>{format_value_robust(row.get('Black + Hispanic Population (2020)', np.nan))}</td>
             <td>{format_value_robust(row.get('Black + Hispanic Population (2022)', np.nan))}</td>
             <td>{format_value_robust(row.get('Black + Hispanic Population (2024)', np.nan))}</td>
             <td>{format_value_robust(row.get('Black + Hispanic Population Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Black + Hispanic Population % Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Black + Hispanic Population Change 2022-2024', np.nan))}</td>
             <td>{format_value_robust(row.get('Black + Hispanic Population % Change 2022-2024', np.nan))}</td>
           </tr>
            <tr>
             <td>VAP</td>
             <td>{format_value_robust(row.get('Voting Age Population (2020)', np.nan))}</td>
             <td>{format_value_robust(row.get('Voting Age Population (2022)', np.nan))}</td>
             <td>{format_value_robust(row.get('Voting Age Population (2024)', np.nan))}</td>
              <td>{format_value_robust(row.get('Voting Age Population Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Voting Age Population % Change 2020-2022', np.nan))}</td>
              <td>{format_value_robust(row.get('Voting Age Population Change 2022-2024', np.nan))}</td>
             <td>{format_value_robust(row.get('Voting Age Population % Change 2022-2024', np.nan))}</td>
           </tr>
            <tr>
             <td>Anglo VAP</td>
             <td>{format_value_robust(row.get('Anglo VAP (2020)', np.nan))}</td>
             <td>{format_value_robust(row.get('Anglo VAP (2022)', np.nan))}</td>
             <td>{format_value_robust(row.get('Anglo VAP (2024)', np.nan))}</td>
              <td>{format_value_robust(row.get('Anglo VAP Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Anglo VAP % Change 2020-2022', np.nan))}</td>
              <td>{format_value_robust(row.get('Anglo VAP Change 2022-2024', np.nan))}</td>
             <td>{format_value_robust(row.get('Anglo VAP % Change 2022-2024', np.nan))}</td>
           </tr>
           <tr>
             <td>Non-Anglo VAP</td>
             <td>{format_value_robust(row.get('Non-Anglo VAP (2020)', np.nan))}</td>
             <td>{format_value_robust(row.get('Non-Anglo VAP (2022)', np.nan))}</td>
             <td>{format_value_robust(row.get('Non-Anglo VAP (2024)', np.nan))}</td>
             <td>{format_value_robust(row.get('Non-Anglo VAP Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Non-Anglo VAP % Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Non-Anglo VAP Change 2022-2024', np.nan))}</td>
             <td>{format_value_robust(row.get('Non-Anglo VAP % Change 2022-2024', np.nan))}</td>
           </tr>
           <tr>
             <td>Asian VAP</td>
             <td>{format_value_robust(row.get('Asian VAP (2020)', np.nan))}</td>
             <td>{format_value_robust(row.get('Asian VAP (2022)', np.nan))}</td>
             <td>{format_value_robust(row.get('Asian VAP (2024)', np.nan))}</td>
             <td>{format_value_robust(row.get('Asian VAP Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Asian VAP % Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Asian VAP Change 2022-2024', np.nan))}</td>
             <td>{format_value_robust(row.get('Asian VAP % Change 2022-2024', np.nan))}</td>
           </tr>
           <tr>
             <td>Black VAP</td>
             <td>{format_value_robust(row.get('Black VAP (2020)', np.nan))}</td>
             <td>{format_value_robust(row.get('Black VAP (2022)', np.nan))}</td>
             <td>{format_value_robust(row.get('Black VAP (2024)', np.nan))}</td>
             <td>{format_value_robust(row.get('Black VAP Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Black VAP % Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Black VAP Change 2022-2024', np.nan))}</td>
             <td>{format_value_robust(row.get('Black VAP % Change 2022-2024', np.nan))}</td>
           </tr>
           <tr>
             <td>Hispanic VAP</td>
             <td>{format_value_robust(row.get('Hispanic VAP (2020)', np.nan))}</td>
             <td>{format_value_robust(row.get('Hispanic VAP (2022)', np.nan))}</td>
             <td>{format_value_robust(row.get('Hispanic VAP (2024)', np.nan))}</td>
             <td>{format_value_robust(row.get('Hispanic VAP Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Hispanic VAP % Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Hispanic VAP Change 2022-2024', np.nan))}</td>
             <td>{format_value_robust(row.get('Hispanic VAP % Change 2022-2024', np.nan))}</td>
           </tr>
           <tr>
             <td>Black + Hispanic VAP</td>
             <td>{format_value_robust(row.get('Black + Hispanic VAP (2020)', np.nan))}</td>
             <td>{format_value_robust(row.get('Black + Hispanic VAP (2022)', np.nan))}</td>
             <td>{format_value_robust(row.get('Black + Hispanic VAP (2024)', np.nan))}</td>
             <td>{format_value_robust(row.get('Black + Hispanic VAP Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Black + Hispanic VAP % Change 2020-2022', np.nan))}</td>
             <td>{format_value_robust(row.get('Black + Hispanic VAP Change 2022-2024', np.nan))}</td>
             <td>{format_value_robust(row.get('Black + Hispanic VAP % Change 2022-2024', np.nan))}</td>
           </tr>
         </table>
         """, axis=1)


         # Generate the full popup HTML using the snippets
         master_vtd_gdf['full_popup_html'] = master_vtd_gdf.apply(generate_full_popup_html_checkbox, axis=1)
         print("✅ Created 'full_popup_html' column with interactive popup HTML (checkboxes).")

    else:
         print(f"❌ master_vtd_gdf is missing required HTML snippet columns: {missing_html_snippet_cols}. Cannot generate HTML snippets and full popup HTML.")
         # Create a placeholder column
         master_vtd_gdf['full_popup_html'] = "Popup data not available due to missing data columns."
         master_vtd_gdf['tooltip_text'] = "Tooltip data not available due to missing data columns."


    # --- Final Check ---
    print("\n📋 Head of master_vtd_gdf with 'full_popup_html' and 'tooltip_text':")
    # Display a sample including the new HTML columns and key identifiers
    # Check if required columns exist before displaying
    display_cols = ['VTDKEY', 'County', 'PREC', 'PlanH2316', 'PlanS2168', 'tooltip_text', 'full_popup_html']
    display_cols_present = [col for col in display_cols if col in master_vtd_gdf.columns]
    if display_cols_present:
         display(master_vtd_gdf[display_cols_present].head())
    else:
         print("⚠️ Required display columns not found in master_vtd_gdf.")

    print("\nColumns of master_vtd_gdf after HTML column generation:")
    print(master_vtd_gdf.columns.tolist())

## Folium map creation and layer addition

### Subtask:
Initialize the Folium map. Load and reproject additional geographic layers (county boundaries, CD6 boundary) to EPSG:4326. Add all layers (VTDs with popups/tooltips, county boundaries, CD6 boundary) to the map and add a layer control.

**Reasoning**:
Initialize a Folium map centered on the CD6 area. Reproject the master VTD GeoDataFrame and any additional boundary GeoJSONs (county and CD6) to EPSG:4326 for Folium compatibility. Add the VTD layer as a GeoJson overlay, including the prepared popups and tooltips. Add the county and CD6 boundary layers. Add a Layer Control to toggle layers and finally display the map.

In [ ]:
# Ensure the map object 'm' exists
if 'm' in locals() and isinstance(m, folium.Map):
    output_html_path = 'cd6_vtd_map.html'
    try:
        m.save(output_html_path)
        print(f"✅ Map saved to {output_html_path}")
    except Exception as e:
        print(f"❌ Error saving map to HTML: {e}")
else:
    print("❌ Map object 'm' not found or is not a Folium Map. Please ensure the map was created successfully in a previous step.")

In [ ]:
import pandas as pd
import os
from IPython.display import display

print("--- Loading Heads of Relevant Files ---")

# File containing Precinct, County, and Intersecting Cities (corrected from DemographicsFinal)
demographics_file_path = "DemographicsFinal.csv"
if os.path.exists(demographics_file_path):
    try:
        demographics_df = pd.read_csv(demographics_file_path)
        print(f"\n📋 Head of {demographics_file_path}:")
        display(demographics_df.head())
        print("\nColumns:")
        print(demographics_df.columns.tolist())
    except Exception as e:
        print(f"❌ Error loading {demographics_file_path}: {e}")
else:
    print(f"\n❌ {demographics_file_path} not found.")

# File containing Primary Votes data
primary_votes_file_path = "PrimaryVotesFinal.csv" # Corrected filename
if os.path.exists(primary_votes_file_path):
    try:
        primary_votes_df = pd.read_csv(primary_votes_file_path)
        print(f"\n📋 Head of {primary_votes_file_path}:")
        display(primary_votes_df.head())
        print("\nColumns:")
        print(primary_votes_df.columns.tolist())
    except Exception as e:
        print(f"❌ Error loading {primary_votes_file_path}: {e}")
else:
    print(f"\n❌ {primary_votes_file_path} not found.")

# File containing TORV data (assuming this is the correct cleaned file name from previous steps)
cleaned_torv_file_path = "cleaned_torv_data.csv"
if os.path.exists(cleaned_torv_file_path):
    try:
        cleaned_torv_df = pd.read_csv(cleaned_torv_file_path)
        print(f"\n📋 Head of {cleaned_torv_file_path}:")
        display(cleaned_torv_df.head())
        print("\nColumns:")
        print(cleaned_torv_df.columns.tolist())
    except Exception as e:
        print(f"❌ Error loading {cleaned_torv_file_path}: {e}")
else:
    print(f"\n❌ {cleaned_torv_file_path} not found.")


print("\n--- Finished Loading Heads ---")

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Upload your three main files
from google.colab import files
uploaded = files.upload()

# Load them
demographics = pd.read_csv('DemographicsFinal.csv')
primary_votes = pd.read_csv('PrimaryVotesFinal.csv')
torv = pd.read_csv('cleaned_torv_data.csv')

# Merge files by 'VTD Identifier' / 'CNTYVTD' (normalize column names)
demographics.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
primary_votes.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
full = demographics.merge(primary_votes, on='CNTYVTD', suffixes=('', '_pv'))
full = full.merge(torv, on='CNTYVTD', suffixes=('', '_torv'))

# Preview
full.head()


In [ ]:
# Filter needed cols and calculate vote change
full['Ellzey Change'] = full['Ellzey (R) 2024 Primary Votes'] - full['Ellzey (R) 2022 Primary Votes']

fig = px.histogram(full, x='Ellzey Change', nbins=30, title='Change in Ellzey Votes by CNTYVTD')
fig.show()


In [ ]:
fig = px.scatter(
    full,
    x='Turnout Change 2022-2024',
    y='Ellzey Change',
    color='County Name',
    hover_name='CNTYVTD',
    title='Turnout Change vs Ellzey Vote Change',
    trendline='ols'
)
fig.show()


In [ ]:
fig = px.scatter(
    full,
    x='Hispanic VAP (2024)',
    y='Turnout 2024',
    color='County Name',
    hover_name='CNTYVTD',
    title='Hispanic VAP vs Turnout 2024'
)
fig.show()


In [ ]:
opportunities = full[
    (full['VR % Change 2022-2024'].str.rstrip('%').astype(float) > 3) &
    (full['Turnout % Change 2022-2024'].str.rstrip('%').astype(float) < 1)
]

fig = px.scatter(
    opportunities,
    x='VR % Change 2022-2024',
    y='Turnout % Change 2022-2024',
    color='County Name',
    hover_name='CNTYVTD',
    title='Opportunity Zones: VTDs w/ High VR Growth & Low Turnout Change'
)
fig.show()


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Select relevant columns
cluster_data = full[[
    'Ellzey (R) 2024 Primary Votes',
    'Turnout 2024',
    'Voter Registration 2024',
    'Black + Hispanic VAP (2024)',
    'Anglo VAP (2024)'
]].dropna()

# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(cluster_data)

# Fit KMeans
kmeans = KMeans(n_clusters=3, random_state=0, n_init=10).fit(X_scaled) # Added n_init for clarity and to avoid future warnings

# Assign cluster labels back to the original DataFrame based on the index of the clustered data
full['VTD Cluster'] = None # Initialize the column with None or NaN
full.loc[cluster_data.index, 'VTD Cluster'] = kmeans.labels_ # Assign labels using the index


# Visualize clusters
# Use the original 'full' DataFrame and filter for non-null cluster labels for plotting
fig = px.scatter(
    full.dropna(subset=['VTD Cluster']), # Filter out rows where clustering was not performed
    x='Voter Registration 2024',
    y='Turnout 2024',
    color='VTD Cluster', # Use the new cluster column
    title='Clustered VTDs by Registration and Turnout',
    hover_name='CNTYVTD' # Add hover_name back
)
fig.show()

# Task
The interactive report has been successfully generated and saved as `/content/ellzey_report.html`.

You can download this HTML file by clicking on the file icon in the left sidebar (file browser), navigating to `/content/`, right-clicking on `ellzey_report.html`, and selecting "Download".

## Consolidated Data Processing

### Subtask:
Prepare all geographic and attribute data for the interactive map. This includes loading, unzipping, reprojecting, cleaning, merging, and calculating all necessary metrics and HTML snippets.


**Reasoning**:
The subtask requires consolidating all data processing steps into a single block, including loading, unzipping, cleaning, reprojecting, merging, and calculating metrics. This comprehensive code block will ensure all necessary data is prepared for the interactive map and address previous issues with column availability for HTML generation.



In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import zipfile
from IPython.display import display

print("--- Starting Consolidated Data Processing ---")

# 1. Define all file paths
vtds_geojson_path = 'vtds_cd6.geojson'
cleaned_torv_path = 'cleaned_torv_data.csv'
demographics_file_path = "DemographicsFinal.csv"
primary_votes_file_path = "PrimaryVotesFinal.csv"
counties_shapefile_path = "texas_county_shapefiles/County_Boundaries.shp"
cities_shapefile_path = "texas_city_shapefiles/City.shp"
precincts_shapefile_path = "precincts_shapefiles/Precincts24G.shp"
cd6_shapefile_path = "cd6_shapefile/PLANC2193.shp"
districts_xlsx_path = "precincts24g_districts.xlsx"

# Zip file paths and their extraction directories
zip_map = {
    'TexasCounties.zip': 'texas_county_shapefiles',
    'TexasCities.zip': 'texas_city_shapefiles',
    'precincts24g.zip': 'precincts_shapefiles',
    'planc2193.zip': 'cd6_shapefile',
}

# 2. Define target CRSs
proj_crs = "EPSG:3083" # For projected data (calculations)
folium_crs = "EPSG:4326" # For Folium (web mapping)

# 3. Ensure directories exist and unzip files
for zip_file, extract_to in zip_map.items():
    if not os.path.exists(extract_to):
        os.makedirs(extract_to, exist_ok=True)
        print(f"✅ Created directory: {extract_to}")

    # Check if zip file exists before trying to unzip
    if os.path.exists(zip_file):
        # Check if shapefile components already exist in the extraction directory
        # This prevents unnecessary unzipping which can cause issues with existing files
        shp_file_base = os.path.splitext(os.path.basename(list(zip_map.keys())[list(zip_map.values()).index(extract_to)].replace('.zip', '.shp')))[0]
        if "planc2193" in shp_file_base.lower():
            shp_file_base = "PLANC2193"
        elif "texascities" in shp_file_base.lower():
            shp_file_base = "City"
        elif "texascounties" in shp_file_base.lower():
            shp_file_base = "County_Boundaries"
        elif "precincts24g" in shp_file_base.lower():
            shp_file_base = "Precincts24G"

        required_components = [f'{shp_file_base}.shp', f'{shp_file_base}.shx', f'{shp_file_base}.dbf']
        if not all(os.path.exists(os.path.join(extract_to, comp)) for comp in required_components):
            try:
                with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                    zip_ref.extractall(extract_to)
                print(f"✅ Unzipped {zip_file} to {extract_to}")
            except zipfile.BadZipFile:
                print(f"❌ Error: {zip_file} is a bad zip file.")
            except Exception as e:
                print(f"❌ Error unzipping {zip_file}: {e}")
        else:
            print(f"☑️ {zip_file} already unzipped to {extract_to}.")
    else:
        print(f"❌ {zip_file} not found. Skipping unzip.")

# Initialize GeoDataFrames and DataFrames to None
vtds_cd6_gdf = None
counties_gdf = None
cd_gdf = None
precincts_gdf_proj = None
cities_gdf_proj = None
cleaned_torv_df = None
demographics_df = None
primary_votes_df = None
districts_df = None

# 4. Load geographic data
print("\n--- Loading Geographic Data ---")

# a. vtds_cd6.geojson
if os.path.exists(vtds_geojson_path):
    try:
        vtds_cd6_gdf = gpd.read_file(vtds_geojson_path)
        print(f"✅ Loaded base VTDs from {vtds_geojson_path}.")
        if vtds_cd6_gdf.crs is None:
            print(f"⚠️ {vtds_geojson_path} CRS missing, assuming EPSG:3857 and setting to {proj_crs}")
            vtds_cd6_gdf.set_crs("EPSG:3857", inplace=True)
        if vtds_cd6_gdf.crs != proj_crs:
            vtds_cd6_gdf = vtds_cd6_gdf.to_crs(proj_crs)
            print(f"✅ Reprojected vtds_cd6_gdf to {proj_crs}.")
        else:
            print(f"✅ vtds_cd6_gdf is already in {proj_crs}.")

        # Create canonical 'CNTYVTD'
        if 'CNTYVTD' not in vtds_cd6_gdf.columns:
            if 'CNTY_x' in vtds_cd6_gdf.columns and 'VTD_x' in vtds_cd6_gdf.columns:
                vtds_cd6_gdf['CNTYVTD'] = vtds_cd6_gdf['CNTY_x'].astype(str).str.strip() + vtds_cd6_gdf['VTD_x'].astype(str).str.strip()
                print("✅ Created 'CNTYVTD' in vtds_cd6_gdf from CNTY_x and VTD_x.")
            elif 'CNTYVTD_x' in vtds_cd6_gdf.columns:
                vtds_cd6_gdf.rename(columns={'CNTYVTD_x': 'CNTYVTD'}, inplace=True)
                print("✅ Renamed 'CNTYVTD_x' to 'CNTYVTD' in vtds_cd6_gdf.")
            elif 'CNTYVTD_y' in vtds_cd6_gdf.columns:
                vtds_cd6_gdf.rename(columns={'CNTYVTD_y': 'CNTYVTD'}, inplace=True)
                print("✅ Renamed 'CNTYVTD_y' to 'CNTYVTD' in vtds_cd6_gdf.")
            else:
                print("❌ Could not find a suitable VTD identifier column in vtds_cd6_gdf.")

        if 'CNTYVTD' in vtds_cd6_gdf.columns:
            vtds_cd6_gdf['CNTYVTD'] = vtds_cd6_gdf['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
            print("✅ Cleaned 'CNTYVTD' in vtds_cd6_gdf.")

        master_vtd_gdf = vtds_cd6_gdf[['CNTYVTD', 'VTDKEY', 'geometry']].copy() # Keep VTDKEY for HTML
        print(f"✅ Initialized master_vtd_gdf with {len(master_vtd_gdf)} VTDs.")

    except Exception as e:
        print(f"❌ Error loading or processing {vtds_geojson_path}: {e}")
else:
    print(f"❌ {vtds_geojson_path} not found. Cannot proceed with map creation.")
    master_vtd_gdf = gpd.GeoDataFrame({'CNTYVTD': [], 'VTDKEY': [], 'geometry': []}, crs=proj_crs)

# b. County_Boundaries.shp
if os.path.exists(counties_shapefile_path):
    try:
        counties_gdf = gpd.read_file(counties_shapefile_path)
        print(f"✅ Loaded county boundaries from {counties_shapefile_path}.")
        if counties_gdf.crs is None:
            print(f"⚠️ {counties_shapefile_path} CRS missing, assuming EPSG:3857.")
            counties_gdf.set_crs("EPSG:3857", inplace=True)
        if counties_gdf.crs != proj_crs:
            counties_gdf = counties_gdf.to_crs(proj_crs)
            print(f"✅ Reprojected counties_gdf to {proj_crs}.")
    except Exception as e:
        print(f"❌ Error loading or processing {counties_shapefile_path}: {e}")
else:
    print(f"❌ {counties_shapefile_path} not found.")

# c. PLANC2193.shp
if os.path.exists(cd6_shapefile_path):
    try:
        cd_gdf = gpd.read_file(cd6_shapefile_path)
        print(f"✅ Loaded CD boundary data from {cd6_shapefile_path}.")
        if cd_gdf.crs is None:
            print(f"⚠️ {cd6_shapefile_path} CRS missing, assuming EPSG:3857.")
            cd_gdf.set_crs("EPSG:3857", inplace=True)
        if cd_gdf.crs != proj_crs:
            cd_gdf = cd_gdf.to_crs(proj_crs)
            print(f"✅ Reprojected cd_gdf to {proj_crs}.")
        cd6_boundary_proj = cd_gdf[cd_gdf['District'] == 6].copy() # Filter for CD6
        print("✅ Filtered for CD6 boundary.")
    except Exception as e:
        print(f"❌ Error loading or processing {cd6_shapefile_path}: {e}")
else:
    print(f"❌ {cd6_shapefile_path} not found.")

# d. Precincts24G.shp
if os.path.exists(precincts_shapefile_path):
    try:
        precincts_gdf_proj = gpd.read_file(precincts_shapefile_path)
        print(f"✅ Loaded precincts from {precincts_shapefile_path}.")
        if precincts_gdf_proj.crs is None:
            print(f"⚠️ {precincts_shapefile_path} CRS missing, assuming EPSG:3857.")
            precincts_gdf_proj.set_crs("EPSG:3857", inplace=True)
        if precincts_gdf_proj.crs != proj_crs:
            precincts_gdf_proj = precincts_gdf_proj.to_crs(proj_crs)
            print(f"✅ Reprojected precincts_gdf_proj to {proj_crs}.")
    except Exception as e:
        print(f"❌ Error loading or processing {precincts_shapefile_path}: {e}")
else:
    print(f"❌ {precincts_shapefile_path} not found.")

# e. City.shp
if os.path.exists(cities_shapefile_path):
    try:
        cities_gdf_proj = gpd.read_file(cities_shapefile_path)
        print(f"✅ Loaded cities from {cities_shapefile_path}.")
        if cities_gdf_proj.crs is None:
            print(f"⚠️ {cities_shapefile_path} CRS missing, assuming EPSG:3857.")
            cities_gdf_proj.set_crs("EPSG:3857", inplace=True)
        if cities_gdf_proj.crs != proj_crs:
            cities_gdf_proj = cities_gdf_proj.to_crs(proj_crs)
            print(f"✅ Reprojected cities_gdf_proj to {proj_crs}.")
    except Exception as e:
        print(f"❌ Error loading or processing {cities_shapefile_path}: {e}")
else:
    print(f"❌ {cities_shapefile_path} not found.")


# 5. Load attribute data
print("\n--- Loading Attribute Data ---")

# a. cleaned_torv_data.csv
if os.path.exists(cleaned_torv_path):
    try:
        cleaned_torv_df = pd.read_csv(cleaned_torv_path)
        cleaned_torv_df['CNTYVTD'] = cleaned_torv_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
        print(f"✅ Loaded and cleaned 'CNTYVTD' in {cleaned_torv_path}.")
    except Exception as e:
        print(f"❌ Error loading {cleaned_torv_path}: {e}")
        cleaned_torv_df = pd.DataFrame({'CNTYVTD': []})
else:
    print(f"❌ {cleaned_torv_path} not found.")
    cleaned_torv_df = pd.DataFrame({'CNTYVTD': []})

# b. DemographicsFinal.csv
demographics_geo_df = pd.DataFrame({'CNTYVTD': []})
demographics_popup_df = pd.DataFrame({'CNTYVTD': []})
if os.path.exists(demographics_file_path):
    try:
        demographics_df = pd.read_csv(demographics_file_path)
        print(f"✅ Loaded {demographics_file_path}.")

        if 'VTD Identifier' in demographics_df.columns:
            demographics_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
            demographics_df['CNTYVTD'] = demographics_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')

            # Separate geo info for tooltip/basic popup
            geo_cols_to_select = ['CNTYVTD', 'Best Matching Precinct Name (Max Overlap)', 'County Name (from TORV)', 'Cities (from TORV)', 'Congressional District (Max Overlap - Spatial)']
            demographics_geo_df = demographics_df[[col for col in geo_cols_to_select if col in demographics_df.columns]].copy()
            if not demographics_geo_df['CNTYVTD'].is_unique:
                demographics_geo_df = demographics_geo_df.groupby('CNTYVTD', as_index=False).first() # Aggregate duplicates
            print("✅ Prepared demographics_geo_df.")

            # Full demographic data for popup
            demographics_popup_df = demographics_df.copy()
            if not demographics_popup_df['CNTYVTD'].is_unique:
                demographics_popup_df = demographics_popup_df.groupby('CNTYVTD', as_index=False).first() # Aggregate duplicates
            print("✅ Prepared demographics_popup_df.")

        else:
            print("❌ 'VTD Identifier' not found in DemographicsFinal.csv.")
    except Exception as e:
        print(f"❌ Error loading {demographics_file_path}: {e}")
else:
    print(f"❌ {demographics_file_path} not found.")

# c. PrimaryVotesFinal.csv
primary_votes_cleaned_df = pd.DataFrame({'CNTYVTD': []})
if os.path.exists(primary_votes_file_path):
    try:
        primary_votes_df = pd.read_csv(primary_votes_file_path)
        print(f"✅ Loaded {primary_votes_file_path}.")

        if 'VTD Identifier' in primary_votes_df.columns:
            # Define columns to select for 2022 and 2024 votes
            vote_cols_2022_select = ['VTD Identifier', 'Ellzey (R) 2022 Primary Votes', 'Buford (R) 2022 Primary Votes', 'Payne (R) 2022 Primary Votes']
            vote_cols_2024_select = ['VTD Identifier', 'Ellzey (R) 2024 Primary Votes', 'Buford (R) 2024 Primary Votes', 'Wiley (R) 2024 Primary Votes']

            votes_2022_df = primary_votes_df[[col for col in vote_cols_2022_select if col in primary_votes_df.columns]].copy()
            votes_2024_df = primary_votes_df[[col for col in vote_cols_2024_select if col in primary_votes_df.columns]].copy()

            votes_2022_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)
            votes_2024_df.rename(columns={'VTD Identifier': 'CNTYVTD'}, inplace=True)

            votes_2022_df['CNTYVTD'] = votes_2022_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
            votes_2024_df['CNTYVTD'] = votes_2024_df['CNTYVTD'].astype(str).str.strip().str.lstrip('0')
            print("✅ Cleaned 'CNTYVTD' in primary vote data.")

            primary_votes_merged = pd.merge(votes_2022_df, votes_2024_df, on='CNTYVTD', how='outer', suffixes=('_2022_raw', '_2024_raw'))

            # Ensure correct final column names after merge if suffixes are applied
            # This can be tricky if names are identical and no suffix is desired for the same year
            # A better approach is to rename before merge if columns are the same but from different sources
            # For this context, assuming distinct vote columns for each year as provided.

            # If columns like 'Ellzey Votes Change (2022-2024)' exist, drop them as we'll recalculate
            cols_to_drop_from_merged = ['Ellzey Votes Change (2022-2024)', 'Ellzey Votes % Change (2022-2024)'] # These came from original primary_votes_df
            primary_votes_merged.drop(columns=[col for col in cols_to_drop_from_merged if col in primary_votes_merged.columns], inplace=True, errors='ignore')

            if not primary_votes_merged['CNTYVTD'].is_unique:
                primary_votes_cleaned_df = primary_votes_merged.groupby('CNTYVTD', as_index=False).first()
            else:
                primary_votes_cleaned_df = primary_votes_merged.copy()

            vote_cols_to_numeric = [col for col in primary_votes_cleaned_df.columns if col not in ['CNTYVTD', 'Best Matching Precinct Name', 'Cities Intersecting VTD', 'County Name']]
            for col in vote_cols_to_numeric:
                primary_votes_cleaned_df[col] = pd.to_numeric(primary_votes_cleaned_df[col], errors='coerce')
            print("✅ Prepared primary_votes_cleaned_df.")

        else:
            print("❌ 'VTD Identifier' not found in PrimaryVotesFinal.csv.")
    except Exception as e:
        print(f"❌ Error loading {primary_votes_file_path}: {e}")
else:
    print(f"❌ {primary_votes_file_path} not found.")

# d. precincts24g_districts.xlsx
agg_districts = pd.DataFrame({'CNTYVTD': []})
if os.path.exists(districts_xlsx_path):
    try:
        districts_df = pd.read_excel(districts_xlsx_path)
        print(f"✅ Loaded {districts_xlsx_path}.")

        if 'FIPS' in districts_df.columns and 'PCTKEY' in districts_df.columns:
            districts_df['CNTYVTD_temp'] = districts_df['FIPS'].astype(str).str.strip() + districts_df['PCTKEY'].astype(str).str.strip().str.lstrip('0')
            agg_districts = districts_df.groupby('CNTYVTD_temp').agg(
                PREC=('PREC', lambda x: ', '.join(x.astype(str).unique())),
                CD=('PlanC2193', 'first'),
                PlanH2316=('PlanH2316', 'first'),
                PlanS2168=('PlanS2168', 'first')
            ).reset_index()
            agg_districts.rename(columns={'CNTYVTD_temp': 'CNTYVTD'}, inplace=True)
            print("✅ Prepared aggregated districts data.")
        else:
            print("❌ 'FIPS' or 'PCTKEY' not found in districts_df.")
    except Exception as e:
        print(f"❌ Error loading {districts_xlsx_path}: {e}")
else:
    print(f"❌ {districts_xlsx_path} not found.")


# 6. Merge all attribute data onto master_vtd_gdf
print("\n--- Merging Attribute Data onto Master GeoDataFrame ---")

if master_vtd_gdf is None or master_vtd_gdf.empty:
    print("❌ master_vtd_gdf is not available or empty. Skipping all merges.")
else:
    # a. Merge demographics_geo_df
    if not demographics_geo_df.empty and 'CNTYVTD' in demographics_geo_df.columns:
        cols_to_merge = [col for col in demographics_geo_df.columns if col != 'CNTYVTD']
        master_vtd_gdf.drop(columns=[col for col in cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
        master_vtd_gdf = master_vtd_gdf.merge(demographics_geo_df, on='CNTYVTD', how='left')
        print("✅ Merged demographics_geo_df.")
    else:
        print("⚠️ demographics_geo_df not available or empty. Skipping merge.")

    # b. Merge demographics_popup_df (for all detailed demographic columns)
    if not demographics_popup_df.empty and 'CNTYVTD' in demographics_popup_df.columns:
        cols_to_merge = [col for col in demographics_popup_df.columns if col != 'CNTYVTD']
        master_vtd_gdf.drop(columns=[col for col in cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
        master_vtd_gdf = master_vtd_gdf.merge(demographics_popup_df, on='CNTYVTD', how='left')
        print("✅ Merged demographics_popup_df.")
    else:
        print("⚠️ demographics_popup_df not available or empty. Skipping merge.")

    # c. Merge cleaned_torv_df
    if not cleaned_torv_df.empty and 'CNTYVTD' in cleaned_torv_df.columns:
        cols_to_merge = [col for col in cleaned_torv_df.columns if col != 'CNTYVTD']
        master_vtd_gdf.drop(columns=[col for col in cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
        master_vtd_gdf = master_vtd_gdf.merge(cleaned_torv_df, on='CNTYVTD', how='left')
        print("✅ Merged cleaned_torv_df.")
    else:
        print("⚠️ cleaned_torv_df not available or empty. Skipping merge.")

    # d. Merge primary_votes_cleaned_df
    if not primary_votes_cleaned_df.empty and 'CNTYVTD' in primary_votes_cleaned_df.columns:
        cols_to_merge = [col for col in primary_votes_cleaned_df.columns if col != 'CNTYVTD']
        master_vtd_gdf.drop(columns=[col for col in cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
        master_vtd_gdf = master_vtd_gdf.merge(primary_votes_cleaned_df, on='CNTYVTD', how='left')
        print("✅ Merged primary_votes_cleaned_df.")
    else:
        print("⚠️ primary_votes_cleaned_df not available or empty. Skipping merge.")

    # e. Merge aggregated districts_df
    if not agg_districts.empty and 'CNTYVTD' in agg_districts.columns:
        cols_to_merge = [col for col in agg_districts.columns if col != 'CNTYVTD']
        master_vtd_gdf.drop(columns=[col for col in cols_to_merge if col in master_vtd_gdf.columns], inplace=True, errors='ignore')
        master_vtd_gdf = master_vtd_gdf.merge(agg_districts, on='CNTYVTD', how='left')
        print("✅ Merged aggregated districts data (PREC, CD, PlanH2316, PlanS2168).")
    else:
        print("⚠️ Aggregated districts data not available or empty. Skipping merge.")


# 7. Calculate derived metrics
print("\n--- Calculating Derived Metrics ---")

# a. Vote Changes and Percentages
vote_cols_for_calc = {
    'BufordR': {'2022': 'Buford (R) 2022 Primary Votes', '2024': 'Buford (R) 2024 Primary Votes'},
    'EllzeyR': {'2022': 'Ellzey (R) 2022 Primary Votes', '2024': 'Ellzey (R) 2024 Primary Votes'},
    'PayneR': {'2022': 'Payne (R) 2022 Primary Votes', '2024': None},
    'WileyR': {'2022': None, '2024': 'Wiley (R) 2024 Primary Votes'}
}

for candidate, years in vote_cols_for_calc.items():
    col_2022 = years['2022']
    col_2024 = years['2024']

    col_2022_exists = col_2022 in master_vtd_gdf.columns if col_2022 else False
    col_2024_exists = col_2024 in master_vtd_gdf.columns if col_2024 else False

    if col_2022_exists:
        master_vtd_gdf[col_2022] = pd.to_numeric(master_vtd_gdf[col_2022], errors='coerce').fillna(0)
    if col_2024_exists:
        master_vtd_gdf[col_2024] = pd.to_numeric(master_vtd_gdf[col_2024], errors='coerce').fillna(0)

    if col_2022_exists and col_2024_exists:
        amount_change_col = f'{candidate}_Amount_Change_2022_2024'
        pct_change_col = f'{candidate}_Pct_Change_2022_2024'

        master_vtd_gdf[amount_change_col] = master_vtd_gdf[col_2024] - master_vtd_gdf[col_2022]
        denominator = master_vtd_gdf[col_2022].replace(0, np.nan)
        master_vtd_gdf[pct_change_col] = ((master_vtd_gdf[col_2024] - master_vtd_gdf[col_2022]) / denominator) * 100
        master_vtd_gdf[pct_change_col].replace([np.inf, -np.inf], np.nan, inplace=True)
        print(f"✅ Calculated '{amount_change_col}' and '{pct_change_col}'.")
    else:
        if col_2022 is not None and col_2024 is not None:
             print(f"⚠️ Missing columns for {candidate} 2022-2024 change. Skipping calculation.")
        master_vtd_gdf[f'{candidate}_Amount_Change_2022_2024'] = np.nan
        master_vtd_gdf[f'{candidate}_Pct_Change_2022_2024'] = np.nan

# b. Demographic Changes and Percentages
demographic_years = ['2020', '2022', '2024']
demographic_categories = [
    'Total Population', 'Anglo Population', 'Non-Anglo Population',
    'Asian Population', 'Black Population', 'Hispanic Population', 'Black + Hispanic Population',
    'Voting Age Population', 'Anglo VAP', 'Non-Anglo VAP',
    'Asian VAP', 'Black VAP', 'Hispanic VAP', 'Black + Hispanic VAP'
]

for category in demographic_categories:
    col_2020 = f'{category} ({demographic_years[0]})'
    col_2022 = f'{category} ({demographic_years[1]})'
    col_2024 = f'{category} ({demographic_years[2]})'

    col_2020_exists = col_2020 in master_vtd_gdf.columns
    col_2022_exists = col_2022 in master_vtd_gdf.columns
    col_2024_exists = col_2024 in master_vtd_gdf.columns

    # Calculate 2020-2022 Changes
    if col_2020_exists and col_2022_exists:
        change_col_2020_2022 = f'{category} Change 2020-2022'
        pct_change_col_2020_2022 = f'{category} % Change 2020-2022'

        master_vtd_gdf[col_2020] = pd.to_numeric(master_vtd_gdf[col_2020], errors='coerce')
        master_vtd_gdf[col_2022] = pd.to_numeric(master_vtd_gdf[col_2022], errors='coerce')

        master_vtd_gdf[change_col_2020_2022] = master_vtd_gdf[col_2022] - master_vtd_gdf[col_2020]
        denominator = master_vtd_gdf[col_2020].replace(0, np.nan)
        master_vtd_gdf[pct_change_col_2020_2022] = ((master_vtd_gdf[col_2022] - master_vtd_gdf[col_2020]) / denominator) * 100
        master_vtd_gdf[pct_change_col_2020_2022].replace([np.inf, -np.inf], np.nan, inplace=True)
        print(f"✅ Calculated '{change_col_2020_2022}' and '{pct_change_col_2020_2022}'.")
    else:
        print(f"❌ Missing columns for {category} Change 2020-2022. Skipping calculation.")
        master_vtd_gdf[f'{category} Change 2020-2022'] = np.nan
        master_vtd_gdf[f'{category} % Change 2020-2022'] = np.nan

    # Calculate 2022-2024 Changes
    if col_2022_exists and col_2024_exists:
        change_col_2022_2024 = f'{category} Change 2022-2024'
        pct_change_col_2022_2024 = f'{category} % Change 2022-2024'

        master_vtd_gdf[col_2022] = pd.to_numeric(master_vtd_gdf[col_2022], errors='coerce')
        master_vtd_gdf[col_2024] = pd.to_numeric(master_vtd_gdf[col_2024], errors='coerce')

        master_vtd_gdf[change_col_2022_2024] = master_vtd_gdf[col_2024] - master_vtd_gdf[col_2022]
        denominator = master_vtd_gdf[col_2022].replace(0, np.nan)
        master_vtd_gdf[pct_change_col_2022_2024] = ((master_vtd_gdf[col_2024] - master_vtd_gdf[col_2022]) / denominator) * 100
        master_vtd_gdf[pct_change_col_2022_2024].replace([np.inf, -np.inf], np.nan, inplace=True)
        print(f"✅ Calculated '{change_col_2022_2024}' and '{pct_change_col_2022_2024}'.")
    else:
        print(f"❌ Missing columns for {category} Change 2022-2024. Skipping calculation.")
        master_vtd_gdf[f'{category} Change 2022-2024'] = np.nan
        master_vtd_gdf[f'{category} % Change 2022-2024'] = np.nan


# 8. Print the shape, head, and column names of the final master_vtd_gdf
print("\n--- Final Merged GeoDataFrame Inspection ---")
if master_vtd_gdf is not None and not master_vtd_gdf.empty:
    print(f"Merged GeoDataFrame shape: {master_vtd_gdf.shape}")
    print("\nHead of merged master_vtd_gdf:")
    display(master_vtd_gdf.head())
    print("\nColumns of merged master_vtd_gdf:")
    print(master_vtd_gdf.columns.tolist())
else:
    print("❌ master_vtd_gdf is empty or not created.")

print("--- Consolidated Data Processing Complete ---")